In [ ]:
!cat .env

In [ ]:
!pip list

In [ ]:
import json
import os
import datetime
from openpyxl import load_workbook
from copy import deepcopy
from json_logic import jsonLogic
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm
from helpers.km_converter import get_road_coordinates,km_to_coordinates
from django.contrib.gis.geos import LineString, Point
import time
import pytz
import uuid
import json
import re
import pandas as pd
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.dates import utc_to_local
from helpers.apps.reportings import get_occurrence_kind

In [ ]:
_,road=get_road_coordinates('BR-163', 854,0,Company.objects.get(name__icontains="Rota do Oeste"))
road,get_road_coordinates('BR-163', 854,0,Company.objects.get(name__icontains="Rota do Oeste"))[0][0]

In [ ]:
km_to_coordinates(road,855)[0][0],km_to_coordinates(road,855)[0][1]

In [ ]:
r=Road.objects.get(id=2041)
r.metadata['direction']['values']

In [ ]:
reps = Reporting.objects.get(company__name="Via Brasil", uuid='4f2ac80a-8639-443c-a889-7c5b947f1a5b')
reps.__dict__

In [ ]:
user = User.objects.get(username="victor.teste.csg")
user.delete()

In [ ]:
for rep in tqdm(reps):
    rep.direction = rep.direction
    rep._history_user = user
    rep.save()

In [ ]:
for a in reps:
    if a.road and a.road.lot_logic:
        print(a.km, a.road, a.road.lot_logic)

In [ ]:
r = Reporting.history.filter(number="RO-DrenI-2022.02310")
r[0].history_type

In [ ]:
reps[1]._history_user

In [ ]:
company=Company.objects.get(name="CART")
company

In [ ]:
roads=Road.objects.filter(company__name__in=["Garcia Monteiro"]).order_by('name').exclude(name__in=["SP-270","SP-300"])
roads.count()

In [ ]:
for a in tqdm(roads):
    #Remove a company especificada do campo m2m
    a.company.remove(company)
    #cria nova road sem o lot_logic
    r=Road(name=a.name,direction=a.direction,marks=a.marks,uf=a.uf,metadata=a.metadata)
    r.save()
    r.company.add(company)

In [ ]:
reps = Reporting.objects.filter(company=company)
reps.count()

In [ ]:
get_road_coordinates(reps[1000].road_name, reps[1000].km,reps[1000].direction,reps[1000].company)

In [ ]:
rep_edit=[]

for a in reps:
    ### GET_ROAD_COORDINATES RETORNAR 2 INFORMAÇÕES POINT OBJECT E A ROAD DESSAS COORDENADAS
    _,road=get_road_coordinates(a.road_name, a.km,a.direction,a.company)
    a.roads = road
    rep_edit.append(a)

In [ ]:
al = ActionLog.objects.filter(company__name="Arteris Fluminense",user='6b286419-348f-4782-ac62-21583d22018f')
al[1].__dict__

In [ ]:
company=Company.objects.get(name="CART")

In [ ]:
status = ServiceOrderActionStatus.objects.get(name="Cancelado",companies=company)
status

In [ ]:
from openpyxl import load_workbook
filename = "Email dos Usuarios Rodovias.xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[0]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
values[1]

In [ ]:
rep_edit=[]
for a in tqdm(values):
    rep=Reporting.objects.filter(number=a['Número']).first()
    rep.status=status
    rep_edit.append(rep)

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=900,user=user)

In [ ]:
company=Company.objects.get(name="CART")
user=User.objects.get(username="rlcs")
company,user

In [ ]:
from tqdm.notebook import tqdm
rep_edit=[]
errados=[]

for b in tqdm(values):
    try:
        rep=Reporting.objects.get(company=company, number=b['Número'])
        rep.form_data['reference']='Aceiros 2023'
        rep_edit.append(rep)
#         print(rep)
    except Exception as e:
        # ARRUMANDO O CAMPO PRA PODER CONSULTAR NO SISTEMA POIS ESTAVA FORA DO PADRÃO
        if str(b['Número'])[-4:] == '2021':
            number = int(str(b['Número'])[:-4])
            number = '{:05}'. format(number) + '.2021'    
        elif str(b['Número'])[-4:] == '2022':
            number = int(str(b['Número'])[:-4])
            number = '{:05}'. format(number) + '.2022'    
        elif str(b['Número'])[-4:] == '2023':
            number = int(str(b['Número'])[:-4])
            number = '{:05}'. format(number) + '.2023'    
        else:
            number = b['Número']
        rep=Reporting.objects.get(company=company, number=number)
        rep.form_data['reference']='Aceiros 2023'
        rep_edit.append(rep)
#         print(number,rep)

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=200,user=user)

In [ ]:
company=Company.objects.get(name="CART")
user=User.objects.get(username="rlcs")
new_step = ApprovalStep.objects.get(
    approval_flow__company=company, 
    name="Cancelado",
    approval_flow__target_model="reportings.Reporting"
)
company,user,new_step

In [ ]:
reps = Reporting.objects.filter(company=company,approval_step=new_step)
reps.count()

In [ ]:
rep = Reporting.objects.get(company=company, number='78708.2022')
rep.history.all().order_by('-history_date')[1]

In [ ]:
rep_edit=[]
for a in tqdm(values):
    try:
        rep = Reporting.objects.get(company=company, number=a['Número'])
        rep.approval_step = new_step
        rep.editable = False
        rep._history_user = user
        rep_edit.append(rep)
    except:
        print(a['Número'])


In [ ]:
utc = pytz.timezone('Asia/Almaty')
rep=Reporting.objects.get(company=company, number='CART-DRE-SDP-01548.2021')
rep.form_data['received_date'],utc

In [ ]:
rep_edit[0].__dict__

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=200,user=user)

In [ ]:
rep_edit=[]
errados=[]
user=User.objects.get(username="rlcs")
for b in tqdm(values):
    try:
        rep=Reporting.objects.get(company=company, number=b['Número'])
        rep.form_data['reference'] = b['Referência']
        rep._history_user = user
        rep_edit.append(rep)
    except:
        # ARRUMANDO O CAMPO PRA PODER CONSULTAR NO SISTEMA POIS ESTAVA FORA DO PADRÃO
        if str(b['Número'])[-4:] == '2021':
            number = int(str(b['Número'])[:-4])
            number = '{:05}'. format(number) + '.2021'    
        elif str(b['Número'])[-4:] == '2022':
            number = int(str(b['Número'])[:-4])
            number = '{:05}'. format(number) + '.2022'    
        elif str(b['Número'])[-4:] == '2023':
            number = int(str(b['Número'])[:-4])
            number = '{:05}'. format(number) + '.2023'    
        else:
            number = b['Número']
        try:
            rep=Reporting.objects.get(company=company, number=number)
            rep.form_data['reference'] = b['Referência']
            rep._history_user = user
            rep_edit.append(rep)
        except Exception as e:
            print(f'{b["Número"]} -> {number}')

In [ ]:
len(rep_edit),rep_edit[0].__dict__

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=230,user=user)

In [ ]:
occ = OccurrenceType.objects.filter(company__name="Rota do Oeste",active=True)

In [ ]:
for a in tqdm(occ):
    for b in a.form_fields['fields']:
        if b['apiName'] == 'anttCode':
            b['dataType'] = 'number'
            a.save()

In [ ]:
occ[0].form_fields['fields']

In [ ]:
reps=Reporting.objects.filter(company__name="Rota do Oeste",form_data__antt_code__isnull=False)
reps.count()

In [ ]:
rep_edit=[]
user=User.objects.get(username="rlcs")
for a in tqdm(reps):
    try:
        a.form_data['antt_code'] = int(a.form_data['antt_code'])
        rep_edit.append(a)
    except Exception as e:
        if a.form_data['antt_code']:
            a.form_data['antt_code'] = int(a.form_data['antt_code'][:5])
            rep_edit.append(a)
        else:
            del a.form_data['antt_code']
            rep_edit.append(a)

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=200,user=user)

In [ ]:
for a,b in obj_str.items():
    if b is not None:
        obj_str[a] = b[:5]

In [ ]:
obj_str

In [ ]:
rep_edit=[]
user=User.objects.get(username="rlcs")
for a,b in tqdm(obj_str.items()):
    
    if b:
        rep = Reporting.objects.get(company__name="Rota do Oeste",number=a.strip())
        rep.form_data['antt_code'] = int(b[:5])
        rep_edit.append(rep)

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=2000,user=user)

In [ ]:
for a in reps:
    a.occurrence_type = OccurrenceType.objects.get(name="Cerca", company__name="Eixo SP")
#     a.save()

In [ ]:
%run 'LPKNB - Funcao Traducao.ipynb'

In [ ]:
traduzirLane('Não se aplica'),traduzirDir('Ambos')

In [ ]:
company.custom_options['reporting']['fields']['direction']['select_options']['options']

# INCLUSÃO DE INFORMAÇÕES EM CLASSE VIA EXCEL 

In [ ]:
filename2 = "Pavimento - quantitativos.xlsx"
wb = load_workbook(filename2)
ws = wb['Apontamentos']

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
rep_edit=[]
user=User.objects.get(username="rlcs")
for b in tqdm(values):
    rep = Reporting.objects.get(company=company,number=b['Número'].strip())
    if 'length' in rep.form_data and 'width' in rep.form_data:
        rep.form_data['area'] = rep.form_data['length']*rep.form_data['width']
    if 'length' in rep.form_data and 'width' in rep.form_data and 'height' in rep.form_data:
        rep.form_data['volume'] = rep.form_data['length']*rep.form_data['width']*rep.form_data['height']
    rep_edit.append(rep)

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=40,user=user)

In [ ]:
c = Company.objects.get(name="Arteris Fernão Dias - Conserva")
c

In [ ]:
f = Firm.objects.filter(company=c,users__username="gustavo.souza")
for a in f:
    print(a.name)

In [ ]:
mdr = MultipleDailyReport.objects.filter(number__in=['AFD-RDO-2023.01954',
'AFD-RDO-2023.01663',
'AFD-RDO-2023.01928',
'AFD-RDO-2023.01649'
])
firm= ['Roçada 03 - Lote 02',
'Multifuncional - Lote 02',
'Roçada 02 - Lote 02',
'Defensa - Lote 02'
]
len(mdr),len(firm),Firm.objects.filter(name__in=firm)

In [ ]:
for rdo,a in zip(mdr,firm):
    try:
        rdo.firm = Firm.objects.get(name=a)
        rdo.save()
    except:
        print(rdo.number)
#     a.firm=firm
#     a.save()

In [ ]:
# mdr.delete()

# Adição ou consulta ApprovalStep

In [ ]:
users=User.objects.filter(username__in=['gessica.gomes.ter',"eduarda.beraldo",'marcoantonio.reis'])
users


In [ ]:
aps = ApprovalStep.objects.filter(approval_flow__company__name__icontains="Arteris Fernão Dias - Conserva", approval_flow__target_model__icontains="reporting")
aps[1],aps[1].responsible_users.all(),aps[1].__dict__

In [ ]:
for a in users:
    aps[5].responsible_users.add(a)

In [ ]:
for i,ap in enumerate(aps):
    print(ap,i,ap.uuid)
    for i,a in enumerate(ap.responsible_users.all()):
        print('\t'+a.first_name + " "+ a.last_name,a.username)

In [ ]:
aps[2].responsible_users.clear()

In [ ]:
apst = ApprovalStep(name="Aprovado - Revisão",approval_flow_id="5b55570a-06df-48e9-b5d0-b0802f28094a")
apst.pk

In [ ]:
apt = ApprovalTransition(name="Aprovado/Revisão",
                         origin_id="f3872ad5-fbf7-4be7-8e2a-8e13bf72c2e6",
                         destination_id="5643d203-1818-4fbe-b6df-2cbff516b506",
                        condition={'===': [{'var': 'request.action'}, 'Aprovado/Revisão']},
                        callback= {'change_fields': [{'name': 'editable', 'value': True}]},
                         button= {'icon': 'assignment', 'color': 'primary', 'variant': 'raised'})
apt

In [ ]:
apt2 = ApprovalTransition(name="Editado/Aprovado",
                         origin_id="5643d203-1818-4fbe-b6df-2cbff516b506",
                         destination_id="f3872ad5-fbf7-4be7-8e2a-8e13bf72c2e6",
                        condition={'===': [{'var': 'request.action'}, 'Editado/Aprovado']},
                        callback= {'change_fields': [{'name': 'editable', 'value': True}]},
                         button= {'icon': 'assignment', 'color': 'primary', 'variant': 'raised'})
apt2.save()

In [ ]:
apt.callback = {'change_fields': [{'name': 'editable', 'value': True}]}
apt.save()

In [ ]:
apt = ApprovalTransition.objects.get(origin__uuid="f3872ad5-fbf7-4be7-8e2a-8e13bf72c2e6")
apt,apt.__dict__


In [ ]:
Company.objects.all()

In [ ]:
MultipleDailyReport.objects.get(uuid='e503fe4a-8bc8-4bb4-8a4a-f5767e496d62')

In [ ]:
list(UserInCompany.objects.filter(user__username='freitas.flavio').values_list('company__name', flat=True).order_by('company__name'))

In [ ]:
excluir=list(UserInCompany.objects.filter(user__username='victor.martins').values_list('company', flat=True))
companies = UserInCompany.objects.filter(user__username='leonardo.ramos').exclude(company__in=excluir)
companies.count()

In [ ]:
companies[0].__dict__

In [ ]:
victor = User.objects.get(username='victor.martins')
victor

In [ ]:
for a in tqdm(companies):
    a.uuid = None
    a.user = victor
    a.save()

In [ ]:
UserInCompany.objects.filter(user__username='eduardo.mascarenhas')

In [ ]:
com = Company.objects.get(name="Demo Concessionárias")
rep = Reporting.objects.get(number="DEMO-default-2023.00004")
rep.form_data['retro'].append({'type': '3',
                               'color': '1',
                               'branch': '1',
                               'station': 'teste arrayOfObjects v3',
                               'station_km': 789})
rep.save()

In [ ]:
rep = Reporting.objects.get( occurrence_type__uuid = 'a141a451-1bb9-4f50-bced-529011871fb0')

In [ ]:
from json_logic import jsonLogic
reps = Reporting.objects.filter(company__name="Arteris Via Paulista" 

In [ ]:
job = Job.objects.filter(company__name="Arteris Via Paulista",end_date__gte='2023-01-01').order_by('-reporting_count')
job[0].firm.__dict__

In [ ]:
occ=OccurrenceType.objects.filter(company__name='Eixo Sp - Pavimento', name='Boca De Saída Drenos')
occ

In [ ]:
occ[0].delete()

In [ ]:
rep = Reporting.objects.get(form_data__artesp_code='580722',company__name='Entrevias')
rep.__dict__

In [ ]:
filename = "Apontamentos de Poda 12.04.xlsx"
wb = load_workbook(filename)
ws = wb['Apontamentos']

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
rept=[]
user=User.objects.get(username="rlcs")
for a in tqdm(values):
    try:
        rep = Reporting.objects.get(number=a['Número'],company__name='Eixo SP')
        rep.occurrence_type = OccurrenceType.objects.get(company__name="Eixo SP",uuid="c25ae9d5-4ab7-44ec-8a25-045946454578")
        rept.append(rep)
    except Exception as e:
        print(e,a['Número'])

In [ ]:
apontamentos = bulk_update_with_history(rept,Reporting, use_django_bulk=True,batch_size=50,user=user)

In [ ]:
rdo = MultipleDailyReport.objects.get(number="EIXO-RDO-2023.08907")
rdo.reportings.all()

In [ ]:
al = ActionLog.objects.filter(user__username="eduardo.bonfim1").order_by('-created_at')
al[4].__dict__

In [ ]:
rep = Reporting.objects.filter(occurrence_type=OccurrenceType.objects.get(company__name="CART",name="Fundação"))
rep[0].__dict__

In [ ]:
rep = Reporting.objects.filter(company__name="Rota do Oeste")
rep.count()

In [ ]:
counter=0
for a in rep:
    if type(a.form_data['anttCode']) is str:
        counter+=1
counter

In [ ]:
reg = OccurrenceRecord.history.filter(number__icontains='2023',history_type="-").order_by("-history_date")
reg[0].__dict__

In [ ]:
bull = Reporting.objects.filter(company__name="Arteris Fluminense - Conserva",reporting_resources__isnull=False)
bull.count()

In [ ]:
for a in bull:
    print(a.number)

In [ ]:
bull[2].reporting_resources.all()

In [ ]:
uuid="ca20ea8a-24db-4b90-84f4-78bb768e38c0"
p = ProcedureResource.objects.get(uuid='032cbae3-3a1f-4208-af2c-7d1f2affc3b7')
p.delete()

# NC_ARTESP

In [ ]:
import pytz

In [ ]:
utc = pytz.timezone('America/Sao_Paulo')
utc

In [ ]:
data=datetime.datetime(2023, 4, 13, 0, 0).replace(tzinfo=utc)
data

In [ ]:
filename = "Solicitacao Prazo ARTESP_Remanescente.xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[1]]

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [ ]:
values[400]

In [ ]:
rept=[]
user=User.objects.get(username="rlcs")
for a in tqdm(values):
    try:
        if a['Prazo Solicitado'] is not None:
            rep = Reporting.objects.get(company__name__exact="Entrevias",form_data__artesp_code=str(a['NC']))
            new_date = a['Prazo Solicitado'].replace(tzinfo=utc)
            rep.due_at = new_date
            
            rept.append(rep)
            print(rep)
    except Exception as e:
        print(e,a['NC'])
    

In [ ]:
apontamentos = bulk_update_with_history(rept,Reporting, use_django_bulk=True,batch_size=100,user=user)

In [ ]:
rept[0].form_data['artesp_code']

In [ ]:
rep = Reporting.objects.get(company__name__exact="Entrevias",form_data__artesp_code='629244')
rep.due_at,rep.form_data

In [ ]:
for b in rept:
    if b.form_data['artesp_code'] == '629244':
        print(b.due_at)

In [ ]:
rep = Reporting.objects.get(uuid="513a9a9a-690b-4d41-8134-5f187d3102b6")
rep.__dict__

In [ ]:
rep.road_name = "Sem nome 2"
rep.road_id = "1652"
# rep.save()

In [1]:
lista = [
{ "type": "Feature", "properties": { "Name": "KM 415 NORTE - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.94633381653049, -18.889667581621961, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 415 SUL - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.9463715878749, -18.889657954248928, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "km 416 NORTE - BR116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.946712023240188, -18.898265535720331, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 416 SUL - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.946750428827592, -18.89829201109864, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 417 NORTE - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.942375392662377, -18.90629726717582, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 417 SUL - BR 116 MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.942451732964749, -18.906323141975161, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 418 SUL - BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.941004272528147, -18.915220906931431, 977.62833246670948 ] } },
{ "type": "Feature", "properties": { "Name": "KM 418 NORTE - BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.940956853411507, -18.915221658527098, 977.62833246670948 ] } },
{ "type": "Feature", "properties": { "Name": "KM 419 SUL - BR-116 MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.939804583895032, -18.924186972059161, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 419 NORTE - BR-116 MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.939747677859252, -18.92418787401574, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 420 NORTE - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.94249139352732, -18.931881606937491, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 420 SUL - BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.94251889870182, -18.931827000732319, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 421 SUL - BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.948781840863333, -18.938381313666401, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 421 NORTE - BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.948734732689758, -18.938400119565021, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 422 NORTE - BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.950823800610713, -18.947133276629661, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 422 SUL- BR-116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.950851780937121, -18.947105746360471, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 423 SUL - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.958192210907441, -18.95282992153199, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 423 NORTE - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.958173716939321, -18.952857301376749, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 424 SUL - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.966336057850548, -18.95727693911974, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 424 NORTE - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.966317884331247, -18.957322370758739, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 425 SUL - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.971212826128699, -18.96421357851241, 0.0 ] } },
{ "type": "Feature", "properties": { "Name": "KM 425 NORTE - BR 116MG", "description": None, "timestamp": None, "begin": None, "end": None, "altitudeMode": None, "tessellate": -1, "extrude": 0, "visibility": -1, "drawOrder": None, "icon": None }, "geometry": { "type": "Point", "coordinates": [ -41.971165714501069, -18.964232390724309, 0.0 ] } }
]

In [3]:
import re
road_name = 'BR-116'
road_km = int(re.search(r'\d+', lista[0]['properties']['Name']).group())
latitude = lista[0]['geometry']['coordinates'][0]
longitude = lista[0]['geometry']['coordinates'][1]
road_name,road_km,latitude,longitude

('BR-116', 415, -41.94633381653049, -18.88966758162196)

In [4]:
values=[]
obj={}
key=0
values.append({'road_name': 'BR-116'})
for i,a in enumerate(lista):
    if 'NORTE' in a['properties']['Name']:
        key+=1
        obj[str(key)]={}
#         road_name = 'BR-116'
        road_km = a['properties']['Name'].split(' ')[1]
        latitude = a['geometry']['coordinates'][0]
        longitude = a['geometry']['coordinates'][1]


#         obj[str(i)]['road_name'] = road_name
        obj[str(key)]['km'] = float(road_km)
        obj[str(key)]['key'] = key
        obj[str(key)]['point'] ={}
        obj[str(key)]['point']['type'] = "Point"
        obj[str(key)]['point']['coordinates'] = [latitude,longitude]
values.append(obj)
    

In [5]:
values

[{'road_name': 'BR-116'},
 {'1': {'km': 415.0,
   'key': 1,
   'point': {'type': 'Point',
    'coordinates': [-41.94633381653049, -18.88966758162196]}},
  '2': {'km': 416.0,
   'key': 2,
   'point': {'type': 'Point',
    'coordinates': [-41.94671202324019, -18.89826553572033]}},
  '3': {'km': 417.0,
   'key': 3,
   'point': {'type': 'Point',
    'coordinates': [-41.94237539266238, -18.90629726717582]}},
  '4': {'km': 418.0,
   'key': 4,
   'point': {'type': 'Point',
    'coordinates': [-41.94095685341151, -18.9152216585271]}},
  '5': {'km': 419.0,
   'key': 5,
   'point': {'type': 'Point',
    'coordinates': [-41.93974767785925, -18.92418787401574]}},
  '6': {'km': 420.0,
   'key': 6,
   'point': {'type': 'Point',
    'coordinates': [-41.94249139352732, -18.93188160693749]}},
  '7': {'km': 421.0,
   'key': 7,
   'point': {'type': 'Point',
    'coordinates': [-41.94873473268976, -18.93840011956502]}},
  '8': {'km': 422.0,
   'key': 8,
   'point': {'type': 'Point',
    'coordinates': [-4

In [ ]:
r=Road.objects.get(id='2044')
r.__dict__

In [ ]:
list(r.marks.values())[-1]['point']['coordinates']

In [ ]:
lista = ["Arteris Planalto Sul - Conserva",
"Arteris Régis Bittencourt",
"Arteris Litoral Sul",
"Arteris Via Paulista",
"Arteris Fernão Dias",
"Arteris Intervias",
"Arteris Via Paulista",
"Eixo SP",
"Entrevias",
"CART",
"Neovia Bocaína do Sul 329/2018",
"Neovia Lages 652/2019",
"Neovia Alcinópolis",
"Neovia Tijucas",
"Neovia Araquari 886/2020",
"Neovia Vacaria",
"Neovia Planalto sul",
"Neovia General Carneiro",
"Neovia Rancho Queimado 080/2021",
"VF Gomes - BR010",
"VF Gomes - BR010",
"VF Gomes - BR158",
"ELO Rodovias Tietê",
"ELO Ecopistas",
"Elo Rodovias dos Tamoios",
"Afirma Engenharia Viária",
"Neovia Contorno Sul",
"VF Gomes - Confresa",
"Neovia Diamantino",
"Entrevias",
"Neovia Lagoa Vermelha",
"nova lacerda",
"porto espiridião",
"vila bela",
"Entrevias",
"neovia cto biguaçu",
"ELO Rota das Bandeiras",
"Dalba Guarapuava",
"Sanson",
"Neovia Régis Bittencourt",
"Dalba Foz do Iguaçu",
"via brasil",
"Elo Ecovias dos Imigrantes",
"Arteris Fluminense - Conserva",
"Arteris Régis Bittencourt",
"Arteris Litoral Sul - Conserva",
"Arteris Fernão Dias - Conserva",
"Pavicolet",
"Rota do Oeste",
"Neovia Ponte Serrada",
"Arteris Fluminense",
"Elo Via Lagos",
"Garcia Monteiro",
"Televale"]

In [ ]:
cont=0
for a in lista:
#     company=Company.objects.filter(name__icontains=a).first()
    
#     print(company.name)
#     print(a)
    try:
        company=Company.objects.filter(name__icontains=a).first()
        user=UserPermission.objects.filter(companies=company)
        print(company.name)
        for b in user:
            cont+=1
            print(f'{company.name} &\t {b.name}')
    except Exception as e:
        print(e,company,a)

In [ ]:
cont

In [ ]:
c = UserPermission.objects.filter(companies__name__in=lista)
repr(c[3].companies.permission)

In [ ]:
rep = Reporting.history.filter(km=350,history_type="-",number__startswith='EIXO-DRE',road_name='SP-294',job_id="b112acd3-9638-4509-83be-fc6e6660ca56").order_by('-updated_at')
rep.count()

In [ ]:
rep[0].__dict__

In [ ]:
rep[1].__dict__

In [ ]:
job=Job.objects.get(number="EIXO-job-2023.934")
job.uuid

In [ ]:
rep2 = Reporting.history.filter(history_type="-",number__startswith='EIXO-DRE',job_id="b112acd3-9638-4509-83be-fc6e6660ca56",history_user_id="5f2434f3-dc65-4880-8805-d144f98c6126").order_by('-updated_at')
rep2.count()

In [ ]:
rep2[0].__dict__

In [ ]:
ap = ApprovalStep.objects.get(uuid="84621272-540b-474c-9dd9-87fe5a84558b")
ap.__dict__,ap.approval_flow.company.name

In [ ]:
rep = Reporting.history.filter(number="38262.2023",form_data__number_of_persons__isnull=False,history_type="-")


In [ ]:
rep[0].form_data['number_of_persons']

In [ ]:
rep = Reporting.objects.filter(number__in=['70218.2022','38243.2023','38237.2023'],company__name="CART")

In [ ]:
rep[2].occurrence_type,rep[0].occurrence_type.uuid,rep[0].__dict__

In [ ]:
import pandas as pd

In [ ]:
company = Company.objects.get(name="Arteris Fernão Dias - Conserva")
company

In [ ]:
%run 'LPKNB - Funcao Traducao.ipynb'

In [ ]:
occ = OccurrenceType.objects.filter(company=company).values_list('occurrence_kind','name','form_fields__fields').exclude(occurrence_kind__isnull=True).order_by('occurrence_kind')
values=[list(a) for a in occ]
anterior=''
for i,a in enumerate(values):
    if traduzirNaturezaCamel(a[0],company) != None:
        values[i].insert(0,traduzirNaturezaCamel(a[0],company))

        if anterior != a[2]:
            values[i].append([a['displayName'] for a in a[3]])
            anterior=a[2]
        del a[3]

In [ ]:
a[0],occ.count(),values[0]

In [ ]:
df=pd.DataFrame(data=values).to_excel('AF2023.xlsx')

In [ ]:
teste= [a['displayName'] for a in values[0][2]]
teste

In [ ]:
firm = Firm.objects.filter(company=company).values_list('name','uuid').order_by('name')
values2=[list(a) for a in firm]
df2=pd.DataFrame(data=values2).to_excel('cart2023-2.xlsx')


In [ ]:
OccurrenceType.objects.filter(company=company).exclude(occurrence_kind__isnull=True)

In [ ]:
rep = Reporting.objects.filter(found_at__gte='2020-01-01',found_at__lte='2022-05-31',lot__in=[1,4],company__name="Eixo SP", occurrence_type__uuid='ff9f0674-fdc9-48a7-99b3-0f5a3302160a')
rep.count()

In [ ]:

for i,a in enumerate(tqdm(rep)):
    a.delete()

In [ ]:
rep = Reporting.objects.filter(company__name="CAJ", firm__isnull=True).order_by('-found_at')
rep.count()

In [ ]:
subempresa = '55d21899-c992-4dab-87dd-9b1ab5c80c98'
firm = "ac4ed564-dfe3-4373-820e-739d381e4240"
sub=SubCompany.objects.get(uuid=subempresa)
f=Firm.objects.get(uuid=firm)

In [ ]:
sub,f

In [ ]:
for a in rep:
    a.firm = f
    a.sub_company = sub
    a.save()

In [ ]:
rep = Reporting.objects.get(number="AFD-2023.00859")

In [ ]:
rep.instance.save()

In [ ]:
repfile =ReportingFile.history.model.objects.filter(reporting=rep)
repfile.count()

In [ ]:
for a in repfile:
    a.instance.save()

In [ ]:
r = Road.objects.filter(company__name="Eixo SP").order_by('name','marks__0__km')
r

In [ ]:
data={}
for i,a in enumerate(r):
    data[i]=a.name
    data[i]=a.marks['0']['km']
    data[i]=a.marks[list(a.marks)[-1]]['km']

In [ ]:
df=pd.DataFrame(data=list(data))

In [ ]:
df

In [ ]:
reps = Reporting.objects.filter(company__name="CAJ", occurrence_type__occurrence_kind=2, road=road[1])
road= Road.objects.filter(name__icontains='Sem Nome',company__name="CAJ")
road[0],reps.count()

In [ ]:
reps2 = Reporting.objects.filter(company__name="CAJ", occurrence_type__occurrence_kind=2, road=road[0])
reps2.count()

In [ ]:
for a in reps2:
    print(a.number,a.road,'|    name:',a.road_name)

In [ ]:
rep_edit=[]
errados=[]
user=User.objects.get(username="rlcs")
for b in tqdm(reps):
    try:
        b.road = road[0]
        b.road_name=road[0].name
        b.road_id=road[0].id
        rep_edit.append(b)
    except Exception as e:
        print(f'{b} | {e}')
        errados.append(b)

In [ ]:
rep_edit[0].__dict__

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=400,user=user)

In [ ]:

rep.road=road[1]
rep.road_name=road[1].name
rep.road_id=road[1].id
rep.save()

In [ ]:
target = 'reporting'
company = Company.objects.get(name__exact="Rota do Oeste")
at=ApprovalTransition.objects.filter(
    destination__approval_flow__company=company,destination__approval_flow__target_model__icontains=target)
at

In [ ]:
users=[]
obj={}
for a in at:
    print(f"{a.origin.name} > BOTÃO: {str(a).split(':')[0]} > {a.destination.name}")
    users=[str(b).split(' - ')[1] for b in a.origin.responsible_users.all()]
    obj[a.origin.name] = users
obj

In [ ]:
for a,b in obj.items():
    print(a)
    print('\t',b)

In [ ]:
reps=Reporting.objects.filter(company__name="Garcia Monteiro", occurrence_type__name="Chegada de agregado")
reps.count(),int(reps[0].number.split('.')[1]),reps[0].__dict__

In [ ]:
rep_edit=[]
user=User.objects.get(username="rlcs")
for b in tqdm(reps):
    
    try:
        rep=Reporting.objects.get(company__name="Garcia Monteiro", number=b.number)
        if int(b.number.split('.')[1]) >=680:
            print(b.number)
            b.occurrence_type_id = b.occurrence_type_id
            b.form_data['material_provider'] = b.form_data['material_provider']
            b.form_data['material_destiny'] = b.form_data['material_destiny']
            b.save()
    except Exception as e:
        print(f'{b["Número"]} -> {number}')

In [ ]:
company=Company.objects.get(name="Arteris Via Paulista - SISOAE")
company

In [ ]:
filename = "Relacao InventarioXInspecao"

In [ ]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]
wb.sheetnames, ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value

        
    values.append(obj)

In [ ]:
len(values)

In [ ]:
company=Company.objects.get(name="Arteris Via Paulista - SISOAE")
reps=["INS-2023.00192",
"INS-2023.00193",
"INS-2023.00194",
"INS-2023.00195",
"INS-2023.00196",
"INS-2023.00197",
"INS-2023.00199",
"INS-2023.00198",
"INS-2023.00200",
"INS-2023.00201"]
invs=["05.03.01.48",
"05.03.29.09.00.03",
"05.03.01.09",
"05.03.01.37",
"05.03.02.11",
"05.03.29.04.00.02",
"05.03.04.93",
"05.03.04.73",
"05.03.03.78",
"05.03.03.79"]

In [ ]:
# for a in values:
for i,a in enumerate(reps):
    try:
        rep=Reporting.objects.get(company=company,number=a)
        inv=Reporting.objects.get(company=company,number=invs[i])
        rep.parent = inv
        rep.save()
    except Exception as e:
        print(f'{e} | {rep} | {inv}')

In [ ]:
rep=Reporting.objects.filter(number='INS-2022.00887')
rep

In [ ]:
rep=Reporting.objects.get(company=company,number='INS-2022.00888')
inv=Reporting.objects.get(company=company,number='05.03.29.09.00.04')
rep.parent = inv
# rep.save()

In [ ]:
recuperar = ['c29ef86a-cbb8-4f08-a68f-56032635ef5d',
 '40d41851-88b1-4501-800a-dd04514a075a',
 '307d77ba-3b0a-45af-9a64-a2b47a833ca0']
desobstruir=['a91f5d55-cd13-450e-a3fa-057f35229579',
 '69ae5a2b-e04e-41c3-887e-3baabefe94d3']
prolongar=['e1bd87de-2a31-4dc8-8681-b7121850b220',
 '14ab5e16-1014-4c42-8289-aefb26496ccb']

In [ ]:
company = Company.objects.get(name="Arteris Via Paulista - SISOAE")

In [ ]:
rep_problema=Reporting.objects.get(company=company,number='INS-2023.00205')
rep_problema_file=list(map(str,ReportingFile.objects.filter(reporting=rep_problema).values_list('uuid',flat=True)))

In [ ]:
for a in rep_problema.form_data['therapy']:
    for b in a['treatment_images']:
        if b in rep_problema_file:
            pass
        else:
            print(b)
            a['treatment_images'].remove(b)

In [ ]:
rep_problema.form_data['therapy']

In [ ]:
rep_problema.save()

In [ ]:
rep=Reporting.history.filter(uuid="e2563928-ed3e-40d6-bf65-29bd36f53fed")
rep

In [ ]:
rep_edit=[]
user=User.objects.get(username="rlcs")
for a in tqdm(values):
    rdo = MultipleDailyReport.objects.get(number=a['Serial do RDO'])
#     print(rdo.number,rdo.firm.name,a['PARA'])
    try:
        rdo.firm = Firm.objects.get(name=a['PARA'])
        rdo.save()
    except Exception as e:
        print(f'{e} | {rdo.number}')

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=200,user=user)

In [ ]:
reps= Reporting.objects.get(company__name="Entrevias SISOAE", number__icontains='2023.01366')
reps.save()

In [ ]:
company = Company.objects.get(name="Eixo Sp - Pavimento")
company

In [ ]:
company.custom_options

In [ ]:
company.custom_options.keys()

In [ ]:
company.custom_options['reporting']['exporter']['extra_columns'][0].keys()

In [ ]:
key,header=[],[]
for a in company.custom_options['reporting']['exporter']['extra_columns']:
#     print(a['key'],a['header'])
    key.append(a['key'].replace("'",'"'))
    header.append(a['header'])

In [ ]:
print(key)
print(header)

In [ ]:
reps = Reporting.history.filter(company__name="Arteris Fernão Dias - Conserva",history_type='~',history_date__gte='2023-5-11',history_user_id='f4204ee0-2084-4bff-b3b6-ecee2ce40808')
reps.count(),reps[0].__dict__

In [ ]:
for a in reps:
    print(a.number)

In [ ]:
shape = ShapeFile.objects.filter(company__name="Demo Concessionárias")

In [ ]:
reps = Reporting.objects.filter(company__name="Demo Concessionárias",form_data__can_filter__isnull=True)
reps.count()

In [ ]:
userPerm = UserPermission.objects.filter(companies__name="Demo Concessionárias")
for i,a in enumerate(userPerm):
    print(i,a.name)

In [ ]:
userPerm[8].permissions

In [ ]:
lista1=[ "link", "number","roadName","km","endKm","latitude","longitude","occurrenceType","occurrenceKind","length", "width","height","lane","direction","status","createdBy","updatedBy","firm","job","foundAt","updatedAt","executedAt", "dueAt","notes","demandOrigin","artespCode","csp","lot" ]
lista2= ["Link", "Número","Rodovia","km inicial","km final","Latitude","Longitude","Classe","Natureza","Comprimento", "Largura","Espessura","Faixa","Sentido","Status","Criado por","Atualizado por","Equipe","Programação","Encontrado em", "Atualizado em","Executado em","Prazo","Observações","Origem","Cód. ARTESP","CSP", "Lote"]
len(lista1),len(lista2)

In [ ]:
rep = Reporting.objects.get(uuid="8576c680-14c0-4782-92bd-25a3aa91d531")

In [ ]:
rdo = MultipleDailyReport.objects.filter(reportings=rep)
rdo.count(),rdo[0].number

In [ ]:
bol = MeasurementBulletin.objects.get(uuid='dfa8318e-c56d-421d-9805-ea19645b5f19')
bol

In [ ]:
bol.delete()

In [ ]:
company=Company.objects.get(name="Eixo Sp - Pavimento")
UserInCompany.objects.filter(company=company,permissions__name='Usuário Padrão').count()


In [ ]:
panControl=UserPermission.objects.get(name__icontains='Usuário Padrão',companies=company)
panControl.delete()

In [ ]:
UserInCompany.objects.filter(company=company,permissions__name='Coordenador')

In [ ]:
UserInCompany.objects.filter(company=company,permissions=panControl).count()

In [ ]:
jobs = Job.objects.filter(company__name="Entrevias",archived=True,progress__lte=99)
jobs.count()

In [ ]:
jobs[0].__dict__

In [ ]:
jobsH=Job.history.filter(company__name="Entrevias",archived=True,history_type='~',history_date__gte='2023-1-1').order_by('-history_date')

In [ ]:
jobsH.count(),jobsH[0].__dict__

In [ ]:
for a in jobsH:
    print(a.history_user,'|', a.number,'|',a.history_date.strftime('%d-%m-%Y') ,'|',str(a.progress)+'%')

In [ ]:
occ=OccurrenceType.objects.filter(company__name="CART")
occ.count(),occ[0].__dict__

In [ ]:

for a in occ:
    print(a.name)
    for b in a.form_fields['fields']:
        try:
            print('\t',b['displayName'])
        except:
            print('\t',b['display_name'])

In [ ]:
userC = UserInCompany.objects.filter(company__name="Demo Concessionárias", user__username="rlcs")
userC

In [ ]:
user = User.objects.get(username='daniella.gomes')
user

In [ ]:
for a in userC:
    a.uuid=None
    a.user = user
    a.save()

In [ ]:
reps=Reporting.objects.filter(created_at__gte='2023-1-1',company__name='Entrevias').values('occurrence_type__occurrence_kind').annotate(qtd_occ=Count('reporting'))

In [ ]:
for rep in reps:
    print(rep)

In [ ]:
reps = Reporting.objects.filter(number__in=['48638.2023','43846.2023','43734.2023','43610.2023'],company__name="CART")
reps.count()

In [ ]:
for a in reps:
    a.occurrence_type=occ
    a.save()

In [ ]:
occ = OccurrenceType.objects.get(company__name="CART", name__icontains="Motor não funciona")

In [ ]:
occ

In [ ]:
al = ActionLog.objects.filter(company=Company.objects.get(name="Entrevias"),content_type__app_label='service_orders')
al[0].__dict__,al[0].object_id,al.count()

In [ ]:
sos=ServiceOrderActionStatusSpecs.history.filter(company=Company.objects.get(name="Entrevias"),status__uuid="6411edf1-1309-43dd-818f-c9c95c835710").order_by('-history_date')
sos

In [ ]:
for a in sos:
    print(a.order,a.history_user,a.history_date)
sos[0].__dict__

In [ ]:
from django.core import serializers
import json
from io import StringIO
data = serializers.serialize("json", Reporting.objects.filter(company__name="Entrevias",found_at__gte='2023-5-24'))

In [ ]:
json.loads(data)[0]

In [ ]:
UserPermission.objects.get(companies__name__icontains="Entrevias",name="Suporte").__dict__

In [ ]:
UserInCompany.objects.filter(company__name__icontains="Afirma",user__username="rlcs")[0].__dict__

In [ ]:
rep=Reporting.objects.get(number="ENT-PAV-2023.00673")
rep

In [ ]:
rep.__dict__

In [ ]:
rep.form_metadata['area'] = {'manually_specified': False}
rep.form_metadata['height'] = {'manually_specified': False}
rep.form_metadata['tonnage'] = {'manually_specified': False}
rep.form_metadata['length'] = {'manually_specified': False}
rep.save()

In [ ]:
rdo=MultipleDailyReport.history.filter(company__name="CAJ", uuid="710d397b-7037-426a-9a9b-83ccef34dbd4")
rdo[0].history_user

In [ ]:
dro = DailyReportOccurrence.history.get(uuid='fabbd043-809a-4a2a-9995-133945d6ce04')
dro.__dict__,dro.firm.name

In [ ]:
contracts = Contract.objects.filter(company__name="Entrevias")

In [ ]:
perm = UserPermission.objects.get(companies__name="Arteris Planalto Sul - Conserva", name="Coordenador")

In [ ]:
UserInCompany.objects.filter(user__email="cesar.souza@arteris.com.br",company__name="Arteris Planalto Sul - Conserva").update(permissions=perm)

In [ ]:
users=User.objects.filter(companies__name="CART")
users.count()

In [ ]:
users[0].__dict__

In [ ]:
users_perm = UserInCompany.objects.filter(company__name="Eixo SP")

In [ ]:
for a in users_perm:
    try:
        print(a.user,';',a.permissions.name, ';',a.user.last_login.strftime('%d-%m-%Y'))
    except:
        pass

In [ ]:
occ=OccurrenceType.objects.filter(company__name="CART").first()

In [ ]:
len(occ.form_fields['fields'][0]['apiName'])

In [ ]:
con=Contract.objects.get(uuid="6034b6d5-c217-408d-8ce9-320c101d1469")
con

In [ ]:
con.administration_services.count()

In [ ]:
con.performance_services.count()

In [ ]:
fs=FieldSurvey.objects.filter(name='Avaliação do dia 05/10/2023')
fs

In [ ]:
fs.delete()

In [ ]:
MeasurementBulletin.objects.get(uuid='5b9053e9-e4c4-4b84-a821-6c92f8b34c53').delete()

In [ ]:
con2 = Contract.objects.get(uuid="75755d48-c275-446d-8011-568550e370c3")
con2.unit_price_services.count()

In [ ]:
reps=Reporting.objects.filter(company__name="Arteris Fluminense - Conserva", occurrence_type__name="Remoção (Desmembrar - Cerca)")
reps.count()

In [ ]:
occ = OccurrenceType.objects.filter(name__icontains="(Desmembrar - Cerca)")
occ.count()

In [ ]:
for a in occ:
#     a.delete()

In [ ]:
rep = Reporting.history.model.objects.filter(company__name="Arteris Fernão Dias - Conserva",number="AFD-2023.02018",history_type="-")
rep[0].instance.save()

In [ ]:
repf = ReportingFile.history.model.objects.filter(reporting=Reporting.objects.get(company__name="Arteris Fernão Dias - Conserva",number="AFD-2023.02018"))
repf.count()

In [ ]:
for a in repf:
    print(a.__dict__)

In [ ]:
userincompany=UserInCompany.objects.get(user__first_name="Denil")
userincompany

In [ ]:
al = ActionLog.objects.filter(user=userincompany.user,created_at__gte="2023-05-24",action="Create", object_id__isnull=False)

In [ ]:
al[0].__dict__,al.count()

In [ ]:
for a in al:
    print(a.action,a.content_object.__dict__)

In [ ]:
rep=Reporting.objects.get(number="INS-2023.00205")
inv=Reporting.objects.get(number="05.03.29.23.00.09")

In [ ]:
rep,inv

In [ ]:
rep.parent = inv
rep.save()

In [ ]:
occ=OccurrenceType.objects.filter(company__name="Rota do Oeste",)
occ.count()

In [ ]:
occ[0].__dict__

In [ ]:
for a in occ:
    found=False
    for b in a.form_fields['fields']:
        if 'arrayOfObjects' == b['dataType']:
            size=len(b['innerFields'])
            for c in b['innerFields']:
                if 'notes' in c['apiName']:
                    c['dataType']
#     a.save()

In [ ]:
con=Contract.objects.get(uuid="47b07c38-eeb6-41ba-8f5c-d63d7d9c3d5a")
con

In [ ]:
con.delete()

In [ ]:
teste=ReportingFile.objects.filter(uploaded_at__date='2023-6-5',description__icontains='Imagem de Campo', created_by__username='Jeferson.sartori').order_by('-uploaded_at')
for a in teste:
    print(a)

In [ ]:
teste[81].__dict__

In [ ]:
lis = [22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]
lis = list(map(str,lis))

In [ ]:
print(lis)

In [ ]:
rep=Reporting.objects.get(uuid="91c8592f-8255-45ea-a6ce-86d687afa77d")
rep.__dict__,rep.created_by

In [ ]:
rep.created_by = User.objects.get(username="luan.poletti")

In [ ]:
rep.save()

In [ ]:
rep. = User.objects.get(username="luan.poletti")

In [ ]:
rep.save()

In [ ]:
rep.created_by

In [ ]:
al=ActionLog.objects.filter(company__name="CCR - Rio Sp",action="Update",content_type_id=12).order_by('-created_at')
al[0].__dict__

In [ ]:
for a in al:
    print(a.user,a.created_at,a.object_id)

In [ ]:
c=Company.history.filter(name="CCR - Rio Sp")

In [ ]:
c.count()

In [ ]:
c[0].__dict__

In [ ]:
for a in c:
    try:
        print(a.history_date,a.history_user,a.custom_options['daily_report_occurrence'])
    except:
        print(a,a.history_user)

In [ ]:
occ=OccurrenceType.objects.filter(company__name="Arteris Via Paulista")

In [ ]:
occ,occ.__dict__

In [ ]:
columns=[]
for a in occ:
    for b in a.form_fields['fields']:
        try:
            if 'arrayOfObjects' in b['dataType']:
                for c in b['innerFields']:
                    print(c)
        except:
            pass

In [ ]:
for match in tqdm(occ):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    match.save()

In [ ]:
from tqdm.notebook import tqdm
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names

In [ ]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [ ]:
for a in OccurrenceType.objects.filter(company__name="CCR - Rio Sp"):
    for b in a.form_fields['fields']:
        try:
            if 'ufAcess' == b['apiName']:
                b['autofill'] = {"substr": [{"var": "roadName"},-2,2]}
                print(a)
                a.save()
        except:
            pass

In [ ]:
ac = ActionLog.objects.filter(company__name="Eixo SP",object_id=Job.objects.get(uuid="2660c601-a013-4a26-ae4e-3441abfafa60").uuid)
ac

In [ ]:
ac[0].__dict__,ac[0]

In [ ]:
mr = MultipleDailyReport.objects.filter(subcompany__uuid="918ae6bb-adeb-409c-ba74-cf7a97706474")

In [ ]:
user = User.objects.get(uuid="132cfe89-9150-4ebd-a085-dacf09a6faa3")
user

In [ ]:
user.history.all().count(),user.history.all().order_by("-history_date")[0].__dict__
# for a in con.history.all():
#     print(a.history_id,a.history_date,a.spend_schedule)

In [ ]:
uic = UserInCompany.objects.filter(company__name="Entrevias", user=user)
uic[0].permissions.name

In [ ]:
ast = ApprovalStep.objects.get(uuid="2c0f3011-cf65-49f1-ae8c-38e94aaa843e")
ast.__dict__

In [ ]:
rep=Reporting.objects.get(uuid="adab443d-35f9-4c52-99c5-f84a3a50c24c")

In [ ]:
rep.occurrence_type.created_at,rep.occurrence_type.history.all()[23].__dict__

In [ ]:
rep.__dict__

In [ ]:
rep.occurrence_type.history.all()[23].__dict__

In [ ]:
rep.form_data

In [ ]:
rep.save()

In [ ]:
company=Company.objects.get(name='ELO Via Lagos')
company

In [ ]:
for a in UserPermission.objects.filter(companies__name='ELO Via Lagos'):
    up=UserPermission(name=a.name,permissions=a.permissions)
    up.save()
    up.companies.add(company)

In [ ]:
for a in UserInCompany.objects.filter(company=company):
#     u=UserPermission.objects.filter(companies=company,name=a.permissions.name)[1]
    print(a.permissions)

In [ ]:
con = Contract.objects.get(uuid="4b2fb621-6aca-47e2-ad7a-b9d07d60cf42")
con.__dict__

In [ ]:
lista=[1,2,3,4,5]
lista= list(map(str,lista))

In [ ]:
lista

In [ ]:
con=Contract.objects.get(uuid="60f30e3e-63fc-4c33-ab0d-d86ff1421a39")

In [ ]:
con.unit_price_services.all()[0].

In [ ]:
pr=ProcedureResource.objects.filter(uuid__in=["7b9187af-782d-48a0-bf8b-5b3883b50ef9"])
pr.count()

In [ ]:
for a in pr:
    a.approval_status = "WAITING_APPROVAL"
    a.save()

In [ ]:
occ_origin = OccurrenceType.objects.get(company__name='CART',uuid='46146623-2d0f-4dd9-bfb7-f02a38021d85')
occ_destiny=OccurrenceType.objects.get(company__name='CART',uuid='999003b9-1ae7-4790-af45-80c1f44c8a5c')
occ_origin,occ_destiny

In [ ]:
reps=Reporting.objects.filter(company__name="CART", occurrence_type=occ_origin)
reps.count()

In [ ]:
rep_edit=[]
for a in tqdm(reps):
    a.occurrence_type=occ_destiny
    rep_edit.append(a)

In [ ]:
user=User.objects.get(username="rlcs")
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=2000,user=user)

In [ ]:
occ_origin.active=False

In [ ]:
rep=Reporting.objects.filter(number='2022.03859')
rep[1].__dict__

In [ ]:
rep[1].company.name

In [ ]:
occ[0].__dict__

In [ ]:
occ = OccurrenceType.objects.filter(company__name="CART",active=True)

In [ ]:
tem2=False
for a in occ:
    for b in a.form_fields['fields']:
        try:
            if "lacre da amostra" in b['displayName']:

                print(a)
            
                
                
        except:
#             if "Quantitativos executados" in b['display_name'] and "Quantitativos" == b['display_name']:
#                 print(a.name)
            pass

In [ ]:
oi='oi'
if 'i'== oi:
    print('merda')

In [ ]:
def generate_select_export_occ(field,occ):
    result={"if":[]} 
    
    for value in occ:
        result["if"].append({"==":[{"var": "formData.therapy.0."+field["apiName"]}, str(value["uuid"])]})
        result["if"].append(value["name"])
        
    return result

In [ ]:
occ = OccurrenceType.objects.filter(company__name="CAJ").values('name','uuid')
occ.count()

In [ ]:
occcampo=OccurrenceType.objects.filter(company__name="CAJ",name__exact="Avaliação")
field = occcampo[0].form_fields['fields'][5]['innerFields'][0]

In [ ]:
field

In [ ]:
str(occ[0]['uuid'])

In [ ]:
r=generate_select_export_occ(field,occ)

In [ ]:
r

In [ ]:
import pandas as pd
%run 'LPKNB - Funcao Traducao.ipynb'

In [ ]:
company=Company.objects.get(name="Arteris Fluminense")

In [ ]:
occ = OccurrenceType.objects.filter(company=company).values_list('occurrence_kind','name').order_by('occurrence_kind').exclude(occurrence_kind='999',occurrence_kind__isnull=True)
values=[list(a) for a in occ]
for i,a in enumerate(values):
    if traduzirNaturezaValue(a[0]) !='':
        values[i].insert(1,traduzirNaturezaValue(a[0]))

In [ ]:
df=pd.DataFrame(data=values)
df.to_excel('AF2023.xlsx')

In [ ]:
df

In [ ]:
import pandas as pd
%run 'LPKNB - Funcao Traducao.ipynb'

In [ ]:
company=Company.objects.get(name="Arteris Fluminense")

In [ ]:
occ = OccurrenceType.objects.filter(company=company).values_list('occurrence_kind','name','form_fields').order_by('occurrence_kind').exclude(occurrence_kind='999',occurrence_kind__isnull=True)
values=[list(a) for a in occ]
# for i,a in enumerate(values):
#     if traduzirNaturezaValue(a[0]) !='':
#         values[i].insert(1,traduzirNaturezaValue(a[0]))

In [ ]:
df=pd.DataFrame(data=values)
# df.to_excel('AF2023.xlsx')

In [ ]:
df.to_excel('testeCCR.xlsx')

In [ ]:
Reporting.objects.filter(occurrence_type__uuid="a898794d-03ca-411d-bfdf-8a96d5c673fe").count()

In [ ]:
occ=OccurrenceType.objects.get(uuid="3ef5544e-b3fe-4b06-9703-0c6e583605a9")

In [ ]:
occ.delete()

In [ ]:
company=Company.objects.get(name="Arteris Via Paulista - SISOAE")

In [ ]:
rep=Reporting.objects.get(number="05.03.29.06.00.11",company=company)

In [ ]:
rep.number = '05.03.29.06.00.11'

In [ ]:
rep.__dict__

In [ ]:
lista=["leonardo.ramos",
"ayrton.lima",
"breno.hack",
"isadoraschmitz",
"bruna.alves",
"esther.madeira",
"isadora.medeiros",
"julia.santana",
"monique.andreola",
"nicolas.whitechurch",
"paola.hansen"]

In [ ]:
users=User.objects.filter(username__in=lista).order_by('username')
users[1].__dict__

In [ ]:
for a in users:
    a.is_active = False
    a.is_staff = False
    a.is_superuser = False
    a.save()

In [ ]:
for a in users:
    ui = UserInCompany.objects.filter(user__username=a.username)
#     print(ui.company)
    for b in ui:
        b.permissions = UserPermission.objects.filter(companies=b.company,name="Inativo").first()
        b.save()
        

In [ ]:
# permissions = UserPermission.objects.get(uuid='abca0b11-4750-44fd-b411-74386cb51cf1',companies__name="CART")
ui = UserInCompany.objects.filter(user__username=a.username)
ui[1].company

In [ ]:
filename = "Terraplenos_estruturas_contencao.xlsx"
wb = load_workbook(filename)
ws = wb['Terraplenos e Estrutura de C']

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
        
        for i, img in enumerate(obj['img']):
            if i == 0:
                obj['img'][i].name = obj['Descrição Foto Contenção 1_1']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 1_1']
            elif i == 1:
                obj['img'][i].name = obj['Descrição Foto Contenção 1_2']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 1_2']
            elif i == 2:
                obj['img'][i].name = obj['Descrição Foto Contenção 2_1']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 2_1']
            elif i == 3:
                obj['img'][i].name = obj['Descrição Foto Contenção 2_2']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 2_2']
            elif i == 4:
                obj['img'][i].name = obj['Descrição Foto Contenção 3_1']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 3_1']
            elif i == 5:
                obj['img'][i].name = obj['Descrição Foto Contenção 3_2']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 3_2']
            elif i == 6:
                obj['img'][i].name = obj['Descrição Foto Contenção 4_1']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 4_1']
            elif i == 7:
                obj['img'][i].name = obj['Descrição Foto Contenção 4_2']
                obj['img'][i].tipo = obj['Tipo Foto Contenção 4_2']

        
    values.append(obj)

In [ ]:
print(values[0]['img'][2].tipo)

In [ ]:
import openpyxl
from openpyxl_image_loader import SheetImageLoader

# Chamando o image_loader
image_loader = SheetImageLoader(ws)

# Obtendo a imagem (coloque a célula que você precisa em vez de 'A1')
image = image_loader.get('A1')

# Mostrando a imagem
image.show()

In [ ]:
dr=DailyReportContractUsage.objects.get(uuid='7b9187af-782d-48a0-bf8b-5b3883b50ef9')
dr

In [ ]:
dr.delete()

In [ ]:
rep = Reporting.objects.get(uuid='a22ba886-cc13-4ea1-baa9-8146efea9de2')
rep.__dict__

In [ ]:
repFile=ReportingFile.objects.get(uuid='dbb2b49f-da5a-43e3-85cb-7beeceb2b4f9')
repFile

In [ ]:
company=Company.objects.get(name="Eixo SP")

In [ ]:
uc=UserInCompany.objects.filter(company__name='Eixo SP',user__last_login__lte='2022-7-31',permissions=perm_inativo)
uc.count()

In [ ]:
uc[0].user.last_login,uc[0].user.is_active,uc[0].permissions.name,uc.count()

In [ ]:
for a in uc:
    a.permissions = 

In [ ]:
perm_inativo

# Ler cabeçalho e valores da celulas openpyxl

In [ ]:
from openpyxl import load_workbook

wb = load_workbook(filename='Terraplenos_estruturas_contencao.xlsx')
sheet = wb.active
data = []

headers = [cell.value for cell in sheet[1]]

for row in sheet.iter_rows(min_row=2):
    row_dict = {}
    for key, cell in zip(headers, row):
        row_dict[key] = cell.value
#         if 'Foto Contenção 1_1' == key:
#             print(cell.value)
    data.append(row_dict)

In [ ]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize() if value != 'Amarela fluorescente (VII)' else value
    elif sign_field == 'Tipo de Película':
        value=value
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [ ]:
occ=OccurrenceType.objects.get(name__contains="Terraplenos e Estrutura de C",company__name="CCR - Rio Sp")
occ

In [ ]:
from openpyxl import load_workbook
import tempfile

wb = load_workbook(filename='Terraplenos_estruturas_contencao.xlsx')
sheet = wb.active
data = []

headers = [cell.value for cell in sheet[1]]
# img = [a for a in ws._images if a.anchor._from.row == index]
for index, row in enumerate(sheet.iter_rows(min_row=2),1):
    row_dict = {}
    for key, cell in zip(headers, row):
        row_dict[key] = cell.value
    if row_dict['Descrição Foto Contenção 1_1'] is not None:
        row_dict['img'] = [a for a in sheet._images if a.anchor._from.row == index]
        for i,img in enumerate(row_dict['img']):
            if i == 0 and row_dict['Descrição Foto Contenção 1_1'] is not None:
                img.name = row_dict['Descrição Foto Contenção 1_1']
                img.tipo = row_dict['Tipo Foto Contenção 1_1']
            if i == 1 and row_dict['Descrição Foto Contenção 1_2'] is not None:
                img.name = row_dict['Descrição Foto Contenção 1_2']
                img.tipo = row_dict['Tipo Foto Contenção 1_2']
            if i == 2 and row_dict['Descrição Foto Contenção 2_1'] is not None:
                img.name = row_dict['Descrição Foto Contenção 2_1']
                img.tipo = row_dict['Tipo Foto Contenção 2_1']
            if i == 3 and row_dict['Descrição Foto Contenção 2_2'] is not None:
                img.name = row_dict['Descrição Foto Contenção 2_2']
                img.tipo = row_dict['Tipo Foto Contenção 2_2']
            if i == 4 and row_dict['Descrição Foto Contenção 3_1'] is not None:
                img.name = row_dict['Descrição Foto Contenção 3_1']
                img.tipo = row_dict['Tipo Foto Contenção 3_1']
            if i == 5 and row_dict['Descrição Foto Contenção 3_2'] is not None:
                img.name = row_dict['Descrição Foto Contenção 3_2']
                img.tipo = row_dict['Tipo Foto Contenção 3_2']
            if i == 6 and row_dict['Descrição Foto Contenção 4_1'] is not None:
                img.name = row_dict['Descrição Foto Contenção 4_1']
                img.tipo = row_dict['Tipo Foto Contenção 4_1']
            if i == 7 and row_dict['Descrição Foto Contenção 4_2'] is not None:
                img.name = row_dict['Descrição Foto Contenção 4_2']
                img.tipo = row_dict['Tipo Foto Contenção 4_2']
        row_dict['contencao'] = row_dict['img']
    data.append(row_dict)

In [ ]:
data[0]['contencao']

In [ ]:
for a in tqdm(data):
    a['contencao'] = list(filter(None, [{'structurekind': get_sign('Tipo de Estrutura',occ,a['Tipo de Estrutura_1']) if a['Tipo de Estrutura_1'] is not None else None, 'other': a['Outro tipo de estrutura_1'], 'extenstion': a['Extensão Contenção (m)_1'], 'innerHeight': a['Altura Contenção (m)_1'],'anchor':get_sign('Ancoragens da Contenção',occ,a['Ancoragens da Contenção_1']) if a['Ancoragens da Contenção_1'] is not None else None, 'innerQuantity': a['Quantidade (unidade)_1']} if a['Tipo de Estrutura_1'] is not None else None,
                                    {'structurekind': get_sign('Tipo de Estrutura',occ,a['Tipo de Estrutura_2']) if a['Tipo de Estrutura_2'] is not None else None, 'other': a['Outro tipo de estrutura_2'], 'extenstion': a['Extensão Contenção (m)_2'], 'innerHeight': a['Altura Contenção (m)_2'],'anchor':get_sign('Ancoragens da Contenção',occ,a['Ancoragens da Contenção_2']) if a['Ancoragens da Contenção_2'] is not None else None, 'innerQuantity': a['Quantidade (unidade)_2']} if a['Tipo de Estrutura_2'] is not None else None,
                                    {'structurekind': get_sign('Tipo de Estrutura',occ,a['Tipo de Estrutura_3']) if a['Tipo de Estrutura_3'] is not None else None, 'other': a['Outro tipo de estrutura_3'], 'extenstion': a['Extensão Contenção (m)_3'], 'innerHeight': a['Altura Contenção (m)_3'],'anchor':get_sign('Ancoragens da Contenção',occ,a['Ancoragens da Contenção_3']) if a['Ancoragens da Contenção_3'] is not None else None, 'innerQuantity': a['Quantidade (unidade)_3']} if a['Tipo de Estrutura_3'] is not None else None,
                                    {'structurekind': get_sign('Tipo de Estrutura',occ,a['Tipo de Estrutura_4']) if a['Tipo de Estrutura_4'] is not None else None, 'other': a['Outro tipo de estrutura_4'], 'extenstion': a['Extensão Contenção (m)_4'], 'innerHeight': a['Altura Contenção (m)_4'],'anchor':get_sign('Ancoragens da Contenção',occ,a['Ancoragens da Contenção_4']) if a['Ancoragens da Contenção_4'] is not None else None, 'innerQuantity': a['Quantidade (unidade)_4']} if a['Tipo de Estrutura_4'] is not None else None,]))

In [ ]:
rep=Reporting.objects.get(uuid='d4305bdc-cdf7-437f-8843-37ce34804d19')
rep.__dict__

In [ ]:
job=Job.objects.get(uuid="7def981e-fe77-449e-82c5-e7710a4ffe14")
job

In [ ]:
job.delete()

In [ ]:
company=Company.objects.get(name__contains="Kapa")
company

In [ ]:
occ= OccurrenceType.objects.filter(occurrence_kind='1',company=company)
occ

In [ ]:
roads = Road.objects.filter(company__uuid='7c7b39ac-7711-420a-9402-0859df283634',id__in=['1698','1701','1702','1699'])

In [ ]:
roads[0].__dict__

In [ ]:
for a in tqdm(roads):
    new_road=Road(uf=a.uf,direction=a.direction,name=a.name,description=a.description,marks=a.marks,metadata=a.metadata)
    new_road.save()
    new_road.company.add(company)

In [ ]:
company=Company.objects.get(name="Demo Concessionárias")
company

In [ ]:
occs=OccurrenceType.objects.filter(is_oae=True, name="Inspeção Especial")

In [ ]:
occs

In [ ]:
ss=ServiceOrderActionStatusSpecs.objects.filter(
                company=company, order=2
            ).first()

In [ ]:
ss.__dict__

In [ ]:
from helpers.strings import get_obj_from_path
from helpers.apps.reportings import get_lane

In [ ]:
possible_lane_path = (
        "reporting__fields__occurrence_kind__selectoptions__options"
    )
occurrence_types = get_obj_from_path(
        company.custom_options, possible_lane_path
    )

In [ ]:
occurrence_types

In [ ]:
dicionario=[{'name': 'Levantamento', 'value': '1'},
 {'name': 'Inventário', 'value': '2'},
 {'name': 'Pavimento flexível', 'value': '13'}]

In [ ]:

dicionario[0]['teste']=dicionario[0].pop('name')
dicionario

In [ ]:
car = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

car['marca']=car.pop('brand')

In [ ]:
car

In [ ]:
ui=UserInCompany.objects.filter(user__username="erico.lopes@kartado.com.br",permissions__isnull=True)
ui.count()

In [ ]:
job = Job.objects.get(uuid='81eccfc8-5b53-4c9a-8e3d-c39f71d0209c')
job.__dict__

In [ ]:
ui=UserInCompany.objects.filter(user__username='matheus.santos')
ui[0].__dict__,ui[1].permissions.name

In [ ]:
reps= Reporting.objects.filter(company__name="Caminhos da Serra Gaúcha",occurrence_type__name__in=['Cercas',''])
reps.count(),reps[0].__dict__

In [ ]:
rep_edit=[]
for a in tqdm(reps):
    if 'extension' in a.form_data or 'extensao' in a.form_data:
        rep_edit.append(a)

In [ ]:
len(rep_edit)

In [ ]:
for a in tqdm(rep_edit):
    a.save()

# CART alterar equipe rdo

In [ ]:
company=Company.objects.get(name="CART")
company

In [ ]:
rd = MultipleDailyReport.objects.get(number="CART-RDO-2023.19774")
rd

In [ ]:
rd.firm = Firm.objects.get(company=company,name='RMA_T3_07')
rd.save()

In [ ]:
from django.db.models import Q

In [ ]:
users = User.objects.filter(Q(email__icontains='ramosleonardo') | Q(email__icontains='leonardo@kartado.com.br') | Q(email__icontains='leonardo.ramos.1@hotmail.com') | Q(username='leonardo.ramos'))

In [ ]:
users.count(),users[0].__dict__

In [ ]:
leo=[]
for a in users:
    if a.is_active is not True:
        print(a.username)
        leo.append(a.uuid)

In [ ]:
companies=[]
for a in users:
    a.is_active = False
    a.is_staff = False
    a.is_superuser = False
    a.save()
    

In [ ]:
companies

In [ ]:
Reporting.objects.filter(created_by__in=leo).count()

In [ ]:
programacoes = ['PROG-2023.00249',
'PROG-2023.00383',
'PROG-2023.00384',
'PROG-2023.00385',
'PROG-2023.00386',
'PROG-2023.00387',
'PROG-2023.00388',
'PROG-2023.00389',
'PROG-2023.00390',
'PROG-2023.00391',
'PROG-2023.00392',
'PROG-2023.00393',
'PROG-2023.00394',
'PROG-2023.00398',
'PROG-2023.00397',
'PROG-2023.00399',
'PROG-2023.00400',
'PROG-2023.00401',
'PROG-2023.00402',
'PROG-2023.00434',
'PROG-2023.00403',
'PROG-2023.00404',
'PROG-2023.00435',
'PROG-2023.00405',
'PROG-2023.00406',
'PROG-2023.00407']
inspecao = ['05.03.01.36',
'05.03.01.48',
'05.03.01.37',
'05.03.29.01.00.06',
'05.03.29.01.00.07',
'05.03.29.01.00.17',
'05.03.29.01.00.20',
'05.03.29.01.00.23',
'05.03.29.01.00.26',
'05.03.29.01.00.27',
'05.03.29.01.00.29',
'05.03.29.01.00.36',
'05.03.29.01.00.41',
'05.03.29.01.00.42',
'05.03.29.04.00.03',
'05.03.29.04.00.04',
'05.03.29.04.00.06',
'05.03.29.04.00.05',
'05.03.03.78',
'05.03.04.93',
'05.03.04.73',
'05.03.29.09.00.03',
'05.03.29.09.00.04',
'05.03.29.10.00.03',
'05.03.29.13.00.01',
'05.03.29.23.00.09'
]

In [ ]:
for a,b in zip(programacoes,inspecao):
    
    try:
        job = Job.objects.get(company__name="Arteris Via Paulista - SISOAE", number=a)
        job.parent_inventory = Reporting.objects.get(company__name="Arteris Via Paulista - SISOAE", number=b)
        job.save()
    except:
        print(a,b)

In [ ]:
job = Job.objects.get(company__name="Arteris Via Paulista - SISOAE", number='PROG-2023.00387')
inventory = Reporting.objects.get(company__name="Arteris Via Paulista - SISOAE", number='05.03.29.01.00.17')
job.parent_inventory = inventory
job.save()

In [ ]:
from helpers.strings import get_obj_from_path
possible_lane_path = (
        "reporting__fields__occurrence_kind__selectoptions__options"
    )
occurrence_types = get_obj_from_path(
        Company.objects.get(name="Via Brasil").custom_options, possible_lane_path
    )
occurrence_types

In [ ]:
from tqdm.notebook import tqdm
reps=Reporting.objects.filter(company__name='Eixo SP',
                              occurrence_type__name="Sinalização vertical", 
                              found_at__gte='2020-1-1',
                              found_at__lte='2022-5-31',
                              lot__in=['2','3'],
                              occurrence_type__occurrence_kind=2)
reps.count()

In [ ]:
for b in range(reps.count()):
    for a in tqdm(reps[:900]):
        a.delete()

In [ ]:
con = Contract.objects.filter(roads__company__name="Arteris Fernão Dias - Conserva").first()
# get_item_total_price(con,con.performance_services.all())
con.get_item_total_price()

In [ ]:
con.performance_services.all()[0].weight

In [ ]:
con.total_price

In [ ]:
conH = Contract.history.filter(uuid="06d69ed6-0b7c-4d44-8f9c-bc8f13be43a8").order_by('-history_date')

In [ ]:
conH.__dict__

In [ ]:
for a in conH:
    print(a.history_user,a.history_date)

In [ ]:
a=UserPermission.objects.filter(companies__name="ATIVO-TREINAMENTO ENERGIA",name="GERENCIAL")
a,a[0].__dict__

In [ ]:
ui=UserInCompany.objects.get(user=User.objects.get(username="rlcs"),
                 company=Company.objects.get(name="ATIVO-TREINAMENTO ENERGIA"),
                permissions=a[0])
ui

In [ ]:
ui.permissions.name

In [ ]:
ui.delete().strip(' .,;')

In [ ]:
st='luan .,;;'
st = st.strip(' .,;')
print(st)

In [ ]:
UserInCompany._meta.get_fields()

In [ ]:
company=Company.objects.get(name="Caminhos da Serra Gaúcha")
company

In [ ]:
reps=Reporting.objects.filter(occurrence_type__occurrence_kind='2', road__name='BRS-470')
reps.count()

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(reps):
    a.delete()

In [ ]:
import pandas as pd

# Primeiro DataFrame
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})

# Segundo DataFrame
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7']})

# Terceiro DataFrame
df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11']})

# Concatenando os DataFrames
result = pd.concat([df1, df2,df3], axis=0)

result

In [ ]:
ar=AccessRequest.objects.filter(company__name="ATIVO-TREINAMENTO ENERGIA").order_by('-created_at')
ar[0].__dict__

In [ ]:
ar[0].delete()

In [ ]:
ui = UserInCompany.objects.filter(permissions=UserPermission.objects.get(companies__name="Arteris Intervias", name="Visualizador"))
for a in ui:
    print(a.user.full_name,' - ',a.user.username)

# Correção CCR apiName, displayName e selectOptions

In [ ]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
import json

In [ ]:
def load_excel(filename,sheetPosision=0):
    wb = load_workbook(filename=filename+'.xlsx')
    sheet = wb[wb.sheetnames[sheetPosision]]
    data = []

    headers = [cell.value for cell in sheet[1]]

    for row in sheet.iter_rows(min_row=2):
        row_dict = {}
        for key, cell in zip(headers, row):
            row_dict[key] = cell.value
    #         if 'Foto Contenção 1_1' == key:
    #             print(cell.value)
        data.append(row_dict)
    return data

In [ ]:
dados=[]
for a in tqdm(load_excel('CCR Padronizacao',1)):
    occ = OccurrenceType.objects.get(company__name="CCR - Rio Sp",name=a['occ_type'],occurrence_kind=a['r'])
    for o in occ.form_fields['fields']: 
        if a['apiName'] == o['apiName']:
            o['apiName'] = a['apiName (Padrão)']
    occ.save()
#     dados.append(occ)
            

In [ ]:
dados2=[]
for a in tqdm(load_excel('CCR Padronizacao',2)):
    occ = OccurrenceType.objects.get(company__name="CCR - Rio Sp",name=a['occ_type'],occurrence_kind=a['r'])
    for o in occ.form_fields['fields']: 
        if a['apiName'] == o['apiName'] and a['displayName'] == o['displayName']:
            o['apiName'] = a['apiName (Padrão)']
            o['displayName'] = a['displayName (Padrão)']
            if a['dataType'] == 'select':
                selectO = str(a['selectOptions (Padrão)'])
                o['selectOptions'] = json.loads(selectO.replace("'", '"'))
    occ.save()
#     dados2.append(occ)

In [ ]:
rep = Reporting.objects.filter(company__name="Garcia Monteiro", form_data__material_provider='2')

In [ ]:
rep[280]

In [ ]:
reps=Reporting.objects.filter(company__name="Eixo Sp - Pavimento", number__in=["EIXO-TER-2023.00541","EIXO-TER-2023.00537"])

In [ ]:
reps.count()

In [ ]:
reps=Reporting.objects.filter(road__name__icontains='SP-345', company__name="Arteris Via Paulista - SISOAE").order_by('-found_at')

In [ ]:
reps[0].__dict__

In [ ]:
reps=Reporting.objects.filter(road__name__icontains='453', company__name="Caminhos da Serra Gaúcha", occurrence_type__name='Sinalização Vertical')

In [ ]:
reps.count()

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(reps):
    a.delete()

In [ ]:
occ = OccurrenceType.objects.filter(company__name="Caminhos da Serra Gaúcha", occurrence_kind='2')

In [ ]:
for a in occ:
    spec=OccurrenceTypeSpecs.objects.get(occurrence_type=a)
    print(spec.color)

In [ ]:
lista=iter(['#355E1F', '#339936', '#2C846A', '#235E6A', '#1E4559', '#3D77B8', '#3D77B8', '#CA7661', '#BA953E', '#AED175', '#ADE4AF', '#672261', '#A23660'])

In [ ]:
for a in occ:
    a.color = next(lista)
#     a.save()
    print(a.color)

In [ ]:
mb = MeasurementBulletin.objects.filter(uuid__in=['f2129e5e-054f-4741-bcfa-14a92d75129b','b6cd6105-21aa-429b-898a-640326b8e3ef'])

In [ ]:
import os
import json
import re

folder_path = 'CCR_Problem'
new_options=[]
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        with open(os.path.join(folder_path, filename), 'r') as file:
            data={}
            key=re.findall(r'\d+', filename.replace('.json',''))
            data[key[0]] = json.load(file)
            new_options.append(data)

In [ ]:
for a in new_options:
    kind=str(list(a.keys())[0])
    occs = OccurrenceType.objects.filter(occurrence_kind=kind,company__name="CCR - Rio Sp")
    for b in occs:
        for c in b.form_fields['fields']:
            if c['apiName'] == 'problem':
                c['selectOptions']['options'] = a[kind]
        b.save()

In [ ]:
numbers=['ALS-2023.20074',
'ALS-2023.22563',
'ALS-2023.18715',
'ALS-2023.22597']

In [ ]:
reps=Reporting.objects.filter(number__in=numbers)

In [ ]:
reps.count(),reps[0]

In [ ]:
pr=ProcedureResource.objects.filter(reporting=reps[3]
                                   )

In [ ]:
pr

In [ ]:
con = Contract.objects.get(uuid="e17c6a19-b59e-4fe8-9dba-b46face83d40")
con.delete()

In [ ]:
rdo=MultipleDailyReport.objects.filter(number__in=['CART-RDO-2023.13936'])
firm=Firm.objects.get(name='EPO_T3_01')
rdo,firm

In [ ]:
for a in rdo:
    a.firm=firm
    a.save()

In [ ]:
rdo=MultipleDailyReport.objects.filter(number__in=['CART-RDO-2023.13983'])

In [ ]:
rdo.delete()

In [ ]:
company=Company.objects.get(name='Construtora Castilho')
company

In [ ]:
roads = Road.objects.filter(company__name="Caminhos da Serra Gaúcha", name='RSC-453')
roads.count()

In [ ]:
# roads[0].company.add(Company.objects.get(name='Demo Concessionárias'))

In [ ]:
for a in tqdm(roads):
    #cria nova road sem o lot_logic
    r=Road(name=a.name,direction=a.direction,marks=a.marks,uf=a.uf,metadata=a.metadata)
    r.save()
    r.company.add(company)

In [ ]:
Entity._meta.get_fields()

In [ ]:
from datetime import datetime, timedelta

In [ ]:
reps=Reporting.objects.filter(occurrence_type__occurrence_kind='2',
                              company__name='Caminhos da Serra Gaúcha',
                              occurrence_type__name__icontains='Sinalização Vertical',
                              form_data__retro__isnull=False)

In [ ]:
reps[0].form_data['retro'][0]

In [ ]:
for b in reps:
    print(b.number)
    for a in b.form_data['retro']:
        print(a['type_of_film'])

In [ ]:
for occ in OccurrenceType.objects.filter(company__name="Entrevias"):
    for a in occ.form_fields['fields']:
        if 'fresa' in a['displayName']:
            print(occ.name)
        if 'select' in a['dataType']:
            for b in a['selectOptions']['options']:
                if 'fresa' in b['name']:
                    print(occ.name)

In [ ]:
for a in Reporting.objects.filter(occurrence_type__occurrence_kind='2',
                                  occurrence_type__name__icontains='Sinalização Aérea'):
    if a.point[0] == -51.247291 and a.point[1] == -29.189454:
        print('Serial:',a.number,' | Classe:',a.occurrence_type.name.split(' - ')[0],' | ',a.road_name, 'Coordenadas:', a.point[1],a.point[0],' | Atualizado em:',a.updated_at.strftime('%d-%m-%Y | %H:%M'))

In [ ]:
for a in Reporting.objects.filter(occurrence_type__occurrence_kind='2', road_name__icontains='122',
                                  occurrence_type__name__icontains='Sinalização Horizontal'):
    if (a.point[0] == -51.340856 and a.point[1] == -29.309806) or (a.point[0] == -51.252303 and a.point[1] == -29.192978) or (a.point[0] == -51.181926 and a.point[1] == -28.999584) or (a.point[0] == -51.105770 and a.point[1] == -28.662179):
        print('Serial:',a.number,' | Classe:',a.occurrence_type.name.split(' - ')[0],' | ',a.road_name, 'Coordenadas:', a.point[1],a.point[0],' | Atualizado em:',a.updated_at.strftime('%d-%m-%Y | %H:%M'))

In [ ]:
for a in Reporting.objects.filter(occurrence_type__occurrence_kind='2', road_name__icontains='453',
                                  occurrence_type__name__icontains='Sinalização Vertical'):
    if (a.point[0] == -51.4767 and a.point[1] == -29.2000) or (a.point[0] == -51.3368 and a.point[1] == -29.2216) or (a.point[0] == -51.485492 and a.point[1] == -29.207537):
        print('Serial:',a.number,' | Classe:',a.occurrence_type.name.split(' - ')[0],' | ',a.road_name, 'Coordenadas:', a.point[1],a.point[0],' | Atualizado em:',a.updated_at.strftime('%d-%m-%Y | %H:%M'))

In [ ]:
occ = OccurrenceType.objects.all()

In [ ]:
occ.count(),occ[0].form_fields['fields']

In [ ]:
from tqdm.notebook import tqdm
dados=[{}]

for a in tqdm(occ):
    if 'fields' in a.form_fields:
        for b in a.form_fields['fields']:
            try:
                if 'apiName' in b or 'displayName' in b:
                    dados.append({'apiName':b['apiName'],'displayName':b['displayName']})
                else:
                    dados.append({'apiName':b['api_name'],'displayName':b['display_name']})
            except:
                pass

In [ ]:
import pandas as pd
df2=pd.DataFrame(data=dados).to_excel('RelPedro.xlsx')

In [ ]:
lista = [len(a['form_fields__fields']) if a['form_fields__fields'] is not None else 0 for a in occ]

In [ ]:
lista

In [ ]:
from collections import Counter

count = dict(Counter(lista))

print(count)

In [ ]:
import pandas as pd
df2=pd.DataFrame(data=count,index=['a']).to_excel('RelProduto.xlsx')

In [ ]:
df2.tail()

In [ ]:
ui = UserInCompany.objects.filter(user__username='Wagner.França')

In [ ]:
ui[0].permissions

In [ ]:
rdo = MultipleDailyReport.objects.get(number='RDO AFD-RDO-2023.01408')
rdo

In [ ]:
con=Contract.history.filter(history_type='-',subcompany__company=company)

In [ ]:
con

In [ ]:
company=Company.objects.get(name="CART")

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
rdos=MultipleDailyReport.objects.filter(company=company,number__in=['CART-RDO-2023.18902'])
rdos.count(),rdos[0].firm

In [ ]:
firm=Firm.objects.get(name='RMC_T3_03',company=company)

In [ ]:
for a in rdos:
    a.firm = firm
    a.save()

In [ ]:
job=Job.history.filter(uuid='772256e1-e2e4-4388-8888-15b0dd009d4a').order_by('-history_date')

In [ ]:
job[1].__dict__,job[1].history_user

In [ ]:
for a in job:
    print(a.reporting_count)

In [ ]:
user = User.objects.get(username="allana.wendler")
user

In [ ]:
user_firms = list((user.user_firms.all()).union(
                    user.user_firms_manager.all()))
            
user_firms

In [ ]:
ReportingFile.objects.filter(reporting__firm__in=user_firms).count()

In [ ]:
con= Contract.objects.get(uuid='60f30e3e-63fc-4c33-ab0d-d86ff1421a39')
con.resources.history.all()

In [ ]:
cons=ContractService.objects.get(uuid="e0f21114-42cd-4d46-bb36-008ae80fc5d2")
cons

In [ ]:
cons._meta.get_fields()

In [ ]:
from django.db.models import Q
up=UserPermission.objects.filter(Q(permissions__has_key='MultipleDailyReport')|Q(permissions__has_key='multipleDailyReport')|Q(permissions__has_key='multiple_daily_report')).order_by('companies__name')
up.count()

In [ ]:
up[0].name,'MultipleDailyReport' in up[0].permissions

In [ ]:
dados=[{}]
for a in up:
    dados.append({'company':a['companies__name'],'permissão':a['name']})

In [ ]:
import pandas as pd
df = pd.DataFrame(data=dados)

In [ ]:
df.to_excel('permissoes.xlsx')

In [ ]:
lista=["ENGENHEIRO(A)",
"Sala técnica",
"Coordenador",
"Engenheiro",
"Gerente",
"HOMOLOGATOR",
"Administrador",
"GESTOR",
"Gerente/Engenheiro",
"Planejamento e controle",
"Homologador",
"Administrador do sistema",
"Escritório",
"Coord/Supervisor",
"Sala técnica AMP",
"Sala técnica AJR",
"Usuário de controle",
"Encarregado com Configurações",
"Suporte"]

In [ ]:
for a in up:
    if a.name in lista:
        if 'MultipleDailyReport' in a.permissions:
            a.permissions['MultipleDailyReport']['can_create_and_edit_all_firms'] = True
            a.permissions['MultipleDailyReport']['can_view_all_firms'] = True
            a.save()
        if 'multiple_daily_report' in a.permissions:
            a.permissions['multiple_daily_report']['can_create_and_edit_all_firms'] = True
            a.permissions['multiple_daily_report']['can_view_all_firms'] = True
            a.save()
        if 'multipleDailyReport' in a.permissions:
            a.permissions['multipleDailyReport']['can_create_and_edit_all_firms'] = True
            a.permissions['multipleDailyReport']['can_view_all_firms'] = True
            a.save()

In [ ]:
ContractItemUnitPrice.objects.filter(entity__name='Arteris Fluminense')

In [ ]:
con.resources.all().count()

In [ ]:
ContractItemUnitPrice._meta.get_fields()

In [ ]:
ContractItemUnitPrice.objects.filter(sort_string='11111').delete()

In [ ]:
con= Contract.objects.get(uuid='46d3622a-80c4-4d4d-91c6-70da479b1d11')

In [ ]:
con.field_survey

In [ ]:
fs=FieldSurvey.objects.filter(contract=con,name__in=['Avaliação do dia 05/10/2023','A03 - SETEMBRO/2023'])

In [ ]:
fs.count(),fs.delete()

In [ ]:
fs.delete()

In [ ]:
up=UserInCompany.objects.filter(company__name="Pavicolet (Amapá)").exclude(user__email__icontains='kartado')
for a in up:
    print(a.user.full_name,'|',a.permissions.name)

In [ ]:
company=Company.objects.get(name='Demo Energia')

In [ ]:
st=SearchTagOccurrenceType(company=company,
                           occurrence_type=OccurrenceType.objects.get(company__name='Demo Energia',
                                                                       name='Visita técnica a parte interessada'))
st.save()

In [ ]:
st.search_tags.add(SearchTag.objects.get(name__icontains='Visita técnica a parte interessada',company=company))

In [ ]:
st.__dict__

In [ ]:
uc=UserInCompany.objects.filter(Q(user__email__icontains='kartado')| Q(user__email__icontains='hermes'),company__name="Pavicolet (Amapá)")

In [ ]:
for a in uc:
    print(a.permissions.name)
    a.permissions=UserPermission.objects.get(companies__name="Pavicolet (Amapá)",name='Suporte')
    a.save()
    print(a.permissions.name)

In [ ]:
up=UserPermission.objects.filter(companies__name="Pavicolet (Amapá)",name='Coordenador').exclude(name='Inativo')
up[0].__dict__

In [ ]:
for a in up:
    a.uuid=None
    a.name='Suporte'
    a.save()

In [ ]:
a

In [ ]:
from openpyxl import load_workbook
filename2 = "Lista de Materiais Conserva RioSP.xlsx"
wb = load_workbook(filename2)
ws = wb[wb.sheetnames[0]]
# wb.sheetnames[0]

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
# Unidade
company=Company.objects.get(name='Demo Concessionárias')
company

In [ ]:
# Contrato
contract = Contract.objects.filter(subcompany__company=company,name='treinamento dia 22/03')
contract

In [ ]:
# Seção do contrato
cs=ContractService.objects.annotate(qtd_unit_price=Count('contract_item_unit_prices')).filter(firms__subcompany__company=company,qtd_unit_price__gte=1)
cs.count()

In [ ]:
# Usuário
user=User.objects.get(username='rlcs')

In [ ]:
# Entidade
entity=Entity.objects.filter(company=company).first()
entity

In [ ]:
# Quantidade de itens
amount=10000

In [ ]:
for a in values:
    r=Resource(company=company,
               name=a['Descrição do Material'],
               unit=a['UN'],
               total_amount=amount)
    r.save()

In [ ]:
# Recurso(pode ser do RDO ou Contrato)
r=Resource(company=company,name=values[0]['Descrição do Material'],unit=values[0]['UN'],total_amount=amount)
r.save()

In [ ]:
# Recurso do Contrato
sor = ServiceOrderResource(contract=contract[0],
                           resource=r,
                           entity=entity,
                           amount=amount,
                           created_by=user)
sor.save()

In [ ]:
sor.uuid

In [ ]:
# atribui o Recurso do Contrato ao Item de Preço Unitário
cup = ContractItemUnitPrice(entity=entity,resource=sor)
cup.save()

In [ ]:
cup.uuid

In [ ]:
# Adiciona a seção do contrato desejada
cs[0].contract_item_unit_prices.add(cup)

In [ ]:
cs[0].__dict__

In [ ]:
resource=Resource.objects.filter(company__name='Demo Concessionárias',name='Teste 04')

In [ ]:
resource[0].__dict__

In [ ]:
resource1=Resource.objects.filter(company__name='Demo Concessionárias',name='CBUQ',unit='unidade')
resource1[0].__dict__

In [ ]:
from django.conf import settings

# Obtenha a lista de aplicativos instalados
installed_apps = settings.INSTALLED_APPS

# Crie a lista de app_labels
app_labelss = [app.split('.')[-1] for app in installed_apps]
app_labelss

In [ ]:
import pandas as pd
import openpyxl as op
from tqdm.notebook import tqdm

In [ ]:
company = Company.objects.get(name__exact="Garcia Monteiro")
filename = company.name.replace(' ','_')+'_teste'
pd.DataFrame().to_excel(filename+'.xlsx')
wb=op.load_workbook(filename+'.xlsx')
# wb['Sheet1'].title = "Disclaimer"
company,filename

In [ ]:
occs = OccurrenceType.objects.filter(company=company)
occs.count()

In [ ]:
index={}
sheetname =""
for a in tqdm(occs):
    index[a.name] = {}
    sheetname = a.name.translate(str.maketrans('', '', './,!?'))[:15]
    for b in a.form_fields['fields']:
        for key,value in b.items():
            index[a.name][key]=value
        
    df2 = pd.DataFrame(data=index[a.name].values(),index=index[a.name].keys())
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df2.to_excel(writer,sheet_name=sheetname,na_rep="-")

In [ ]:
uc_origin = UserInCompany.objects.filter(user__username="danieli.soares").order_by('company__name')
print(uc_origin[0].company.name)

In [ ]:
for a in uc_origin:
    print(a.permissions.name,a.company.name)

In [ ]:
company=Company.objects.get(name='Arteris Régis Bittencourt')
user = User.objects.get(username='rlcs')
company,user

In [ ]:
up=UserPermission.objects.filter(companies=company)
for i,a in enumerate(up):
    print(a.name,i)

In [ ]:
UserInCompany.objects.get(user=user,company=company).permissions.name

In [ ]:
# ui=UserInCompany.objects.filter(user=user,company=company).update(permissions=up[4])

In [ ]:
ui=UserInCompany.objects.filter(company=company,permissions__name='Inativo')

In [ ]:
for a in ui:
    print(a.permissions.name,a.company.name,a.permissions)

In [ ]:
up[2].companies.all()

In [ ]:
company=Company.objects.get(name='CART')

In [ ]:
for a in OccurrenceType.objects.filter(company=company):
    for b in a.form_fields['groups']:
        try:
            if 'Quantitativos de campo' == b['displayName']:
                print(a.uuid,a.name)
        except:
            pass

In [ ]:
MeasurementBulletin.objects.filter(firm__subcompany__company__name="Sinaliza Via")

In [ ]:
for a in Reporting.objects.filter(occurrence_type__isnull=True,company__name="Rota do Oeste - Obras"):
    print(a.number)

In [ ]:
company=Company.objects.get(name="Arteris Fernão Dias")
company

In [ ]:
for a in UserPermission.objects.filter(companies__name='Arteris Litoral Sul').exclude(name='Inativo'):
    a.uuid=None,
#     a.companies.set(company)
    a.save()
    a.companies.add(company)

In [ ]:
for a in UserInCompany.objects.filter(company=company):
#     if a.permissions is not None:
#         a.permissions = UserPermission.objects.get(name=a.permissions.name,companies=company)
#         a.save()
#     else:
#         a.delete()
    print(a.permissions,a.user.username)

In [ ]:
up=UserPermission.objects.filter(name='ENGENHEIRO(A)')

In [ ]:
up[0].companies.add(company)

In [ ]:
up[0]

In [ ]:
UserInCompany.objects.filter(company=company,user__username='rlcs')

In [ ]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs
from helpers.histories import bulk_update_with_history
from django.contrib.gis.geos import LineString, Point
from tqdm.notebook import tqdm

In [ ]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [ ]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [ ]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
    print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
    print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
            print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
        print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
                print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [ ]:
company=Company.objects.get(name="Caminhos da Serra Gaúcha")

In [ ]:
reps=Reporting.objects.filter(road_name="ERS-122", company=company)

In [ ]:
reps.count(),reps[0].__dict__

In [ ]:
# point, road = get_road_coordinates(reps[0].road_name, reps[0].km, reps[0].direction , company)
point, road = get_road_coordinates(reps[0].road_name, reps[0].km, reps[0].direction , company)
point,road,point[0],point[1]

In [ ]:
rep_edit=[]
for a in tqdm(reps):
    point, road = get_road_coordinates(a.road_name, 
                                       a.km, 
                                       a.direction, 
                                       company)
    if road.name == 'ERS-122':
        a.road=road
        a.point=point
        rep_edit.append(a)
    else:
        print(a.number)

In [ ]:
user=User.objects.get(username="rlcs")

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=2000,user=user)

In [ ]:
for a in UserInCompany.objects.filter(user__username='carla.devides',company__name__startswith='Intervias'):
    try:
        a.permissions=UserPermission.objects.get(companies=a.company,name="Inativo")
        a.save()
    except:
        print(UserPermission.objects.get(companies=a.company,name=a.permissions.name))

In [ ]:
ar=AccessRequest.objects.filter(uuid='07040b37-26b2-475a-8aae-a70efd4feaa4')
ar[0].permissions

In [ ]:
reps=Reporting.objects.filter(occurrence_type__name="Marcos Quilométricos (temporário)",company__name="Eixo SP")
reps.count()

In [ ]:
from tqdm.notebook import tqdm
for i in range(2):
    for a in tqdm(reps[:800]):
        a.delete()

In [ ]:
occ2 = OccurrenceType.objects.filter(company__name="Modera")

In [ ]:
occCoisas=[]
for o in occ2:
    for a in o.form_fields['fields']:
        if a.get("apiName"):
            occCoisas.append({"apiName":a["apiName"],"displayName":a["displayName"],'dataType':a["dataType"]})
        else:
            occCoisas.append({"apiName":a["apiName"],"displayName":a["displayName"],'dataType':a["dataType"]})
#         if a.get('dataType') == 'arrayOfObjects':
#             occCoisas['arrayOfObjects'] = []
#             for b in a['innerFields']:
#                 occCoisas['arrayOfObjects'].append({"apiName":a["apiName"],"displayName":a["displayName"],'dataType':a["dataType"]})
#         if a.get('data_type') == 'arrayOfObjects':
#             occCoisas['arrayOfObjects'] = []
#             for b in a['inner_fields']:
#                 occCoisas['arrayOfObjects'].append({"apiName":a["apiName"],"displayName":a["displayName"],'dataType':a["dataType"]})

In [ ]:
occCoisas

In [ ]:
import json
with open('ModeraFields.txt', 'w') as f:
    json.dump(occCoisas, f)

In [ ]:
mb = MeasurementBulletin.objects.get(uuid="4c35bc03-28ed-4d89-ba9b-fe123448605d")
mb

In [ ]:
mb.__dict__

In [ ]:
pr=ProcedureResource.objects.filter(measurement_bulletin__contract__uuid="e492a29f-852c-4cf9-b461-2a01a6f272bb")

In [ ]:
pr[0].__dict__

In [ ]:
for a in pr:
    a.approval_date = a.creation_date
    a.save()

In [ ]:
pr[0]

In [ ]:
uc = UserInCompany.objects.filter(user__username="rlcs",company__name__startswith='Neovia')
uc.count()

In [ ]:
for a in uc:
    try:
        a.uuid = None
        a.user = User.objects.get(username="pedro.cardoso")
        a.save()
    except:
        print(a.company.name)

# TRANSFERENCIA DE FOTOS PAVICOLET

In [ ]:
pavicolet-CON-2023.15057 >		 pavicolet-CON-2023.14930	
pavicolet-CON-2023.13742 >		 pavicolet-CON-2023.14824	
pavicolet-CON-2023.15074 >		 pavicolet-CON-2023.14825	
pavicolet-CON-2023.15087 >		 pavicolet-CON-2023.14830 

In [ ]:
origem=['pavicolet-CON-2023.15057','pavicolet-CON-2023.13742','pavicolet-CON-2023.15074','avicolet-CON-2023.15087'
]
destino=['pavicolet-CON-2023.14930',
'pavicolet-CON-2023.14824',
'pavicolet-CON-2023.14825',
'pavicolet-CON-2023.14830'
]
len(origem),len(destino)

In [ ]:
rep=Reporting.objects.get(number='pavicolet-CON-2023.14767')

In [ ]:
rf=ReportingFile.objects.filter(reporting__number='pavicolet-CON-2023.14681').update(reporting_id=rep.uuid)

In [ ]:
for orig,dest in zip(origem,destino):
    d=Reporting.objects.get(number=dest)
    ReportingFile.objects.filter(reporting__number=orig).update(reporting_id=d)

In [ ]:
Reporting.objects.get(uuid='cfedb230-b47e-4aad-8acd-6623ce63a492').company

In [ ]:
rep=Reporting.objects.filter()

In [ ]:
for a in rep:
    print(a)

In [ ]:
firm=Firm.objects.get(name="REMAN - AMP - MULTI DRENAGEM - 01")

In [ ]:
reps=Reporting.objects.filter(company__name="Entrevias", occurrence_type__occurrence_kind='18')

In [ ]:
reps.count()

In [ ]:
reps[39705].__dict__

In [ ]:
user=User.objects.get(username='rlcs')

In [ ]:
rep_edit=[]
for a in tqdm(reps):
    if a.form_data.get('demand_origin'):
        a.form_data['demand_origin'].append('2')
    else:
        a.form_data['demand_origin'] = ['2']
    rep_edit.append(a)

In [ ]:
rep_edit[1].__dict__

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=900,user=user)

In [ ]:
reps=Reporting.objects.filter(company__name="Arteris Via Paulista - Faixa de Domínio", occurrence_type__name='Ocupações')

In [ ]:
reps.count()

In [ ]:
for a in reps:
    a.form_metadata['total_acumulado'] = {'manually_specified': False}
    a.save()

In [ ]:
import json
from tqdm.notebook import tqdm
rep_edit=[]
for a in tqdm(reps):
    a.keywords = a.keywords.replace("{'reduce': [{'var': 'formData.annuityArr'}, {'+': [{'var': 'current.annuityValuePaidStep'}, {'var': 'accumulator'}]}, 0]}","")
    rep_edit.append(a)

In [ ]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=900,user=user)

In [ ]:
for a in tqdm(reps[5:10]):
    print(a)

In [ ]:
reps=Reporting.objects.filter(company__name='Entrevias',firm__name='Remanescente Sinalização',created_at__lte='2023-9-17',occurrence_type__occurrence_kind=2)

In [ ]:
reps.count()

In [ ]:
from tqdm.notebook import tqdm
for a in range(10):
    for b in tqdm(reps[:800]):
        b.delete()

In [ ]:
lista=['ROO-RDO-2023.00001',
'ROO-RDO-2023.00007',
'ROO-RDO-2023.00010',
'ROO-RDO-2023.00012',
'ROO-RDO-2023.00015',
'ROO-RDO-2023.00033',
'ROO-RDO-2023.00108',
'ROO-RDO-2023.00116',
'ROO-RDO-2023.00125',
'ROO-RDO-2023.00172',
'ROO-RDO-2023.00269']

In [ ]:
for a in MultipleDailyReport.objects.filter(company__name="Rota do Oeste - Obras", number__in=lista):
    a.delete()

In [ ]:
rdo=MultipleDailyReport.objects.get(number='AFD-RDO-2023.02202')

In [ ]:
jobs=Job.objects.filter(company__name='Arteris Via Paulista',reporting_count__gte=1).annotate(qtd_apontamentos = Count('reportings')).exclude(archived=True)

In [ ]:
jobs[0].__dict__

In [ ]:
for a in jobs:
    if a.reporting_count != a.qtd_apontamentos:
        print(a.number)

In [ ]:
MultipleDailyReport.objects.all().values('firm').annotate(qtd_por_firm=Count('uuid'))

In [ ]:
!cat .env

In [ ]:
ProcedureResource._meta.get_fields()

In [ ]:
ProcedureResource.objects.filter(firm__uuid__in=[*list(UserInFirm.objects.filter(user__username='Alessandro.bonalume').values_list('firm',flat=True))])

In [ ]:
list(UserInFirm.objects.filter(user__username='Alessandro.bonalume').values_list('firm',flat=True))

In [ ]:
# feito: f4841ea6-f7f0-4ff0-9e50-b2854e071b7e; 67b065b9-aac3-4bff-b49a-e4e20fe465dc; d0397913-fe99-47f9-84e8-70dcf75071c1;
# 613f892e-26fc-491e-9dad-cc24953fbb97; 79dc048f-7b17-437e-b738-a0d5e38a996c; 0c63c7c2-40bb-4c11-a392-eb1a7f032944
# 03b2e043-ac62-4fee-8fcc-1b8983484a72
ProcedureResource.objects.filter(uuid='03b2e043-ac62-4fee-8fcc-1b8983484a72').delete()

In [ ]:
Reporting.objects.filter(company__name="Construtora Castilho", occurrence_type__isnull=True)

In [ ]:
lista={"link", "number", "roadName", "km", "endKm", "latitude", "longitude", "occurrenceType", "length", "width", "height", "lane", "track", "branch", "kmReference", "direction", "occurrenceKind", "status", "createdBy", "updatedBy", "firm", "subcompany", "job", "createdAt", "foundAt", "updatedAt", "executedAt", "notes", "dueAt", "price", "equipamentoVeiculoInner1", "horimetroOdometro1", "placaPrefixo1", "operadorMotorista1", "empresa1", "tipo1", "quantidadeOleo1", "diesel1", "acumuladoBomba1", "saldoAnterior1", "entrada1", "notaFiscal1", "saida1", "saldoAtual1", "ponto1", "quantidadeGraxa1", "notes1", "equipamentoVeiculoInner2", "horimetroOdometro2", "placaPrefixo2", "operadorMotorista2", "empresa2", "tipo2", "quantidadeOleo2", "diesel2", "acumuladoBomba2", "saldoAnterior2", "entrada2", "notaFiscal2", "saida2", "saldoAtual2", "ponto2", "quantidadeGraxa2", "notes2", "equipamentoVeiculoInner3", "horimetroOdometro3", "placaPrefixo3", "operadorMotorista3", "empresa3", "tipo3", "quantidadeOleo3", "diesel3", "acumuladoBomba3", "saldoAnterior3", "entrada3", "notaFiscal3", "saida3", "saldoAtual3", "ponto3", "quantidadeGraxa3", "notes3", "equipamentoVeiculoInner4", "horimetroOdometro4", "placaPrefixo4", "operadorMotorista4", "empresa4", "tipo4", "quantidadeOleo4", "diesel4", "acumuladoBomba4", "saldoAnterior4", "entrada4", "notaFiscal4", "saida4", "saldoAtual4", "ponto4", "quantidadeGraxa4", "notes4", "equipamentoVeiculoInner5", "horimetroOdometro5", "placaPrefixo5", "operadorMotorista5", "empresa5", "tipo5", "quantidadeOleo5", "diesel5", "acumuladoBomba5", "saldoAnterior5", "entrada5", "notaFiscal5", "saida5", "saldoAtual5", "ponto5", "quantidadeGraxa5", "notes5", "equipamentoVeiculoInner6", "horimetroOdometro6", "placaPrefixo6", "operadorMotorista6", "empresa6", "tipo6", "quantidadeOleo6", "diesel6", "acumuladoBomba6", "saldoAnterior6", "entrada6", "notaFiscal6", "saida6", "saldoAtual6", "ponto6", "quantidadeGraxa6", "notes6", "equipamentoVeiculoInner7", "horimetroOdometro7", "placaPrefixo7", "operadorMotorista7", "empresa7", "tipo7", "quantidadeOleo7", "diesel7", "acumuladoBomba7", "saldoAnterior7", "entrada7", "notaFiscal7", "saida7", "saldoAtual7", "ponto7", "quantidadeGraxa7", "notes7", "equipamentoVeiculoInner8", "horimetroOdometro8", "placaPrefixo8", "operadorMotorista8", "empresa8", "tipo8", "quantidadeOleo8", "diesel8", "acumuladoBomba8", "saldoAnterior8", "entrada8", "notaFiscal8", "saida8", "saldoAtual8", "ponto8", "quantidadeGraxa8", "notes8", "equipamentoVeiculoInner9", "horimetroOdometro9", "placaPrefixo9", "operadorMotorista9", "empresa9", "tipo9", "quantidadeOleo9", "diesel9", "acumuladoBomba9", "saldoAnterior9", "entrada9", "notaFiscal9", "saida9", "saldoAtual9", "ponto9", "quantidadeGraxa9", "notes9", "equipamentoVeiculoInner10", "horimetroOdometro10", "placaPrefixo10", "operadorMotorista10", "empresa10", "tipo10", "quantidadeOleo10", "diesel10", "acumuladoBomba10", "saldoAnterior10", "entrada10", "notaFiscal10", "saida10", "saldoAtual10", "ponto10", "quantidadeGraxa10", "notes10", "equipamentoVeiculoInner11", "horimetroOdometro11", "placaPrefixo11", "operadorMotorista11", "empresa11", "tipo11", "quantidadeOleo11", "diesel11", "acumuladoBomba11", "saldoAnterior11", "entrada11", "notaFiscal11", "saida11", "saldoAtual11", "ponto11", "quantidadeGraxa11", "notes11", "equipamentoVeiculoInner12", "horimetroOdometro12", "placaPrefixo12", "operadorMotorista12", "empresa12", "tipo12", "quantidadeOleo12", "diesel12", "acumuladoBomba12", "saldoAnterior12", "entrada12", "notaFiscal12", "saida12", "saldoAtual12", "ponto12", "quantidadeGraxa12", "notes12", "equipamentoVeiculoInner13", "horimetroOdometro13", "placaPrefixo13", "operadorMotorista13", "empresa13", "tipo13", "quantidadeOleo13", "diesel13", "acumuladoBomba13", "saldoAnterior13", "entrada13", "notaFiscal13", "saida13", "saldoAtual13", "ponto13", "quantidadeGraxa13", "notes13", "equipamentoVeiculoInner14", "horimetroOdometro14", "placaPrefixo14", "operadorMotorista14", "empresa14", "tipo14", "quantidadeOleo14", "diesel14", "acumuladoBomba14", "saldoAnterior14", "entrada14", "notaFiscal14", "saida14", "saldoAtual14", "ponto14", "quantidadeGraxa14", "notes14", "equipamentoVeiculoInner15", "horimetroOdometro15", "placaPrefixo15", "operadorMotorista15", "empresa15", "tipo15", "quantidadeOleo15", "diesel15", "acumuladoBomba15", "saldoAnterior15", "entrada15", "notaFiscal15", "saida15", "saldoAtual15", "ponto15", "quantidadeGraxa15", "notes15", "comprimentoProgramado", "notesProgramado", "volume", "larguraProgramada", "espessuraProgramada", "volumeProgramado", "area", "areaProgramada", "distancia", "dmt", "distanciaProgramada", "dmtProgramada", "amount", "unidadesProgramadas", "massaTon", "volumeMapa", "tipoDeBordo1", "tipoDeLinha1", "corLinha1", "lengthInner1", "widthInner1", "areaInner1", "tipoDeBordo2", "tipoDeLinha2", "corLinha2", "lengthInner2", "widthInner2", "areaInner2", "tipoDeBordo3", "tipoDeLinha3", "corLinha3", "lengthInner3", "widthInner3", "areaInner3", "tipoDeBordo4", "tipoDeLinha4", "corLinha4", "lengthInner4", "widthInner4", "areaInner4", "tipoDeBordo5", "tipoDeLinha5", "corLinha5", "lengthInner5", "widthInner5", "areaInner5", "tipoDeBordo6", "tipoDeLinha6", "corLinha6", "lengthInner6", "widthInner6", "areaInner6", "tipoDeBordo7", "tipoDeLinha7", "corLinha7", "lengthInner7", "widthInner7", "areaInner7", "tipoDeBordo8", "tipoDeLinha8", "corLinha8", "lengthInner8", "widthInner8", "areaInner8", "tipoDeBordo9", "tipoDeLinha9", "corLinha9", "lengthInner9", "widthInner9", "areaInner9", "tipoDeBordo10", "tipoDeLinha10", "corLinha10", "lengthInner10", "widthInner10", "areaInner10", "tipoDeBordo11", "tipoDeLinha11", "corLinha11", "lengthInner11", "widthInner11", "areaInner11", "tipoDeBordo12", "tipoDeLinha12", "corLinha12", "lengthInner12", "widthInner12", "areaInner12", "tipoDeBordo13", "tipoDeLinha13", "corLinha13", "lengthInner13", "widthInner13", "areaInner13", "tipoDeBordo14", "tipoDeLinha14", "corLinha14", "lengthInner14", "widthInner14", "areaInner14", "tipoDeBordo15", "tipoDeLinha15", "corLinha15", "lengthInner15", "widthInner15", "areaInner15", "massa", "massaProgramada", "prefixo1", "placa1", "kmHorimetroInicial1", "kmHorimetroFinal1", "situacao1", "prefixo2", "placa2", "kmHorimetroInicial2", "kmHorimetroFinal2", "situacao2", "prefixo3", "placa3", "kmHorimetroInicial3", "kmHorimetroFinal3", "situacao3", "prefixo4", "placa4", "kmHorimetroInicial4", "kmHorimetroFinal4", "situacao4", "prefixo5", "placa5", "kmHorimetroInicial5", "kmHorimetroFinal5", "situacao5", "prefixo6", "placa6", "kmHorimetroInicial6", "kmHorimetroFinal6", "situacao6", "prefixo7", "placa7", "kmHorimetroInicial7", "kmHorimetroFinal7", "situacao7", "prefixo8", "placa8", "kmHorimetroInicial8", "kmHorimetroFinal8", "situacao8", "prefixo9", "placa9", "kmHorimetroInicial9", "kmHorimetroFinal9", "situacao9", "prefixo10", "placa10", "kmHorimetroInicial10", "kmHorimetroFinal10", "situacao10", "prefixo11", "placa11", "kmHorimetroInicial11", "kmHorimetroFinal11", "situacao11", "prefixo12", "placa12", "kmHorimetroInicial12", "kmHorimetroFinal12", "situacao12", "prefixo13", "placa13", "kmHorimetroInicial13", "kmHorimetroFinal13", "situacao13", "prefixo14", "placa14", "kmHorimetroInicial14", "kmHorimetroFinal14", "situacao14", "prefixo15", "placa15", "kmHorimetroInicial15", "kmHorimetroFinal15", "situacao15", "placa", "prefixo", "hodometroInicial", "localOrigem", "servicoExecutado", "escavadeiraPrefixo", "material", "horaCarga", "localDestino", "servicoDestino", "hodometroFinal", "estacaInicial", "estacaFinal", "horaDescarga", "serial"}

In [ ]:
occs=[]
for a in Company.objects.get(name="Construtora Castilho").custom_options['reportingSpreadsheet']['extraColumns']:
        if a['key'] in lista:
            continue
        else:
            occs.append(a)

In [ ]:
occs

In [ ]:
MobileSync.objects.filter(created_by__username='luan.poletti',kind='USER_FULL_SYNC_DATA').order_by('-created_at')

In [ ]:
occ=[]
erro=[]
for a in Reporting.objects.filter(company__name="Eixo Sp - Pavimento",executed_at__gte='2023-7-16',executed_at__lte='2023-7-31',road_name='SP-225'):
    try:
        if a.occurrence_type.name not in occ:
            occ.append(a.occurrence_type.name)
            print(a.occurrence_type.name)
    except:
        erro.append(a)
        print(a.number)

In [ ]:
for b in OccurrenceType.objects.filter(company__name="Eixo Sp - Pavimento"):
    for c in b.form_fields['fields']:
        if c['dataType'] == 'arrayOfObjects' and c['apiName'] == 'alertaServiceEspessura':
            
            print(b.form_fields['id'],b.name,b.occurrence_kind,b.uuid)

In [ ]:
r=Reporting.history.filter(number='EIXO-TER-2023.00972')

In [ ]:
r[0].__dict__

In [ ]:
UserInCompany.objects.filter(company__name="Entrevias",permissions__uuid='b99fbd77-8556-4274-a9c0-4b40a2159c08')

In [ ]:
up = UserPermission.objects.filter(
    companies__name__startswith='Arteris'
).exclude(
    companies__name__icontains='paulista'
).exclude(
    companies__name__icontains='intervias'
).exclude(
    companies__name__icontains='lab'
)

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(up):
    if a.permissions.get('quality_assay'):
        a.permissions['quality_assay'] ={"can_edit": False, 
                                           "can_view": False, 
                                           "queryset": "all", 
                                           "can_create": False, 
                                           "can_delete": False}
    if a.permissions.get('quality_sample'):
        a.permissions['quality_sample'] ={"can_edit": False, 
                                           "can_view": False, 
                                           "queryset": "all", 
                                           "can_create": False, 
                                           "can_delete": False}
    if a.permissions.get('quality_project'):
        a.permissions['quality_project'] ={"can_edit": False, 
                                           "can_view": False, 
                                           "queryset": "all", 
                                           "can_create": False, 
                                           "can_delete": False}
    if a.permissions.get('qualityAssay'):
        a.permissions['qualityAssay'] ={"can_edit": False, 
                                           "can_view": False, 
                                           "queryset": "all", 
                                           "can_create": False, 
                                           "can_delete": False}
    if a.permissions.get('qualitySample'):
        a.permissions['qualitySample'] ={"can_edit": False, 
                                           "can_view": False, 
                                           "queryset": "all", 
                                           "can_create": False, 
                                           "can_delete": False}
    if a.permissions.get('qualityProject'):
        a.permissions['qualityProject'] ={"can_edit": False, 
                                           "can_view": False, 
                                           "queryset": "all", 
                                           "can_create": False, 
                                           "can_delete": False}
    a.save()

In [ ]:
up[0].permissions

In [ ]:
import json
company=Company.objects.get(name='Entrevias SISOAE')
print(company.name)
print('Permissão | Qtd_Verdadeiros | Qtd_All')
for a in UserPermission.objects.filter(companies=company):
    
    print(a.name,'|',json.dumps(a.permissions).count('true'),'|',json.dumps(a.permissions).count('all'))

In [ ]:
reps = Reporting.objects.filter(company__name='Construtora Castilho')

In [ ]:
for a in reps:
    if not a.occurrence_type:
        print(a.number)

In [ ]:
ap = ApprovalStep.objects.get(uuid='387e5cce-4aa1-4360-aec8-ee34abd46a15')

In [ ]:
ap.__dict__

In [ ]:
ap2 = ApprovalStep.objects.get(uuid='3983b63f-ccee-4ae6-89ac-8cffe414bbd2')

In [ ]:
ap2.__dict__

In [ ]:
ap2._meta.get_fields()

In [ ]:
ap2.responsible_users.all()

In [ ]:
criado=ApprovalStep.objects.get(approval_flow__company__name='Construtora Castilho', name='Criado')
aprovacao_1=ApprovalStep.objects.get(approval_flow__company__name='Construtora Castilho', name='Aguardando Aprovação 1ª camada')
# aprovacao_2=ApprovalStep.objects.get(approval_flow__company__name='Construtora Castilho', name='Aguardando Aprovação 2ª camada')
aprovado=ApprovalStep.objects.get(approval_flow__company__name='Construtora Castilho',
                                  approval_flow__target_model__icontains='reporting',
                                  name='Aprovado')

In [ ]:
aprovacao_1.name='Aguardando Aprovação'
aprovacao_1.save()

In [ ]:
at_2=ApprovalTransition.objects.get(destination__approval_flow__company__name='Construtora Castilho',destination=aprovacao_2)

In [ ]:
at_2.__dict__

In [ ]:
at_2.name="Aprovar"
at_2.condition= {'===': [{'var': 'request.action'}, 'Aprovar']}
at_2.save()

In [ ]:
reps=Reporting.objects.filter(company__name='Construtora Castilho',approval_step=aprovacao_2)

In [ ]:
criado.responsible_firms.add(*Firm.objects.filter(company__name='Construtora Castilho',name='Suporte Kartado'))

In [ ]:
ReportingFile.objects.get(uuid='b6ce7dc4-87dd-48f5-b399-b329cd9b90c4')

In [ ]:
up=UserPermission.objects.get(uuid='32076d08-901e-4005-a296-8a3e907fa5de').history.all().order_by('-history_date')

In [ ]:
for a in up:
    print(a.history_user,' > ',a.history_date,a.permissions.get('occurrence_record',"Vazio"))

In [ ]:
up[2].permissions['occurrence_record'],up[0].__dict__

In [ ]:
UserInFirm.objects.filter(user__username="luan.poletti",firm__company__name="Arteris Litoral Sul - Conserva").delete()

In [ ]:
user = User.objects.get(username="cassio.conci")
user_firms = list((user.user_firms.all()).union(
                    user.user_firms_manager.all()))
len(user_firms)

In [ ]:
for a in UserInFirm.objects.filter(user=user,firm__company__name="Arteris Litoral Sul - Conserva"):
    a.uuid=None
    a.user=User.objects.get(username="luan.poletti")
#     a.save()

In [ ]:
ufirm = UserInFirm.objects.filter(user=user,firm__company__name="Arteris Litoral Sul - Conserva")

In [ ]:
ufirm.uuid=None
ufirm.user=User.objects.get(username="luan.poletti")
# ufirm.save()

In [ ]:
UserInFirm.objects.filter(user__username="luan.poletti",firm__company__name="Arteris Litoral Sul - Conserva").count()

In [ ]:
ufirm

In [ ]:
# company= Company.objects.get(name='Rota do Oeste')
company= Company.objects.get(uuid='fea2b0fe-868f-4bb8-826a-337f03c69305')

In [ ]:
a=company.custom_options
a

In [ ]:
import json
b = json.dumps(a).encode('ascii').decode()
print(b)

In [ ]:
OccurrenceType.objects.get(uuid='ffddc9a8-f0c9-4628-8a61-8ea3b64af904').form_fields

In [ ]:
MultipleDailyReport.objects.filter(uuid='ac7d6266-f0fe-48cb-b336-62918955a976').delete()

# Inativar em massa

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(UserInCompany.objects.filter(company__name="Neovia Vacaria")):
    a.permissions=UserPermission.objects.filter(companies__name='Neovia Vacaria',name="Inativo").first()
    a.save()
#     ui=a.history.all().order_by('-history_date')[1]
#     ui.instance.save()

# Apontamentos por permissão

In [ ]:
company=Company.objects.get(name="Arteris Régis Bittencourt - Conserva")

In [ ]:
user_qtd_criado={}
for a in Reporting.history.model.objects.filter(company=company, history_type__in=["+"]).values("history_user"):
    if str(a['history_user']) in user_qtd_criado:
        
        user_qtd_criado[str(a['history_user'])] +=1
    else:
        user_qtd_criado[str(a['history_user'])]=0

In [ ]:
# user_qtd_criado

In [ ]:
perm_qtd_criado={}
for b in user_qtd_criado.items():
#     print(b)
    if b[0] != None:
        try:
            perm = UserInCompany.objects.filter(user__uuid=b[0]).order_by('permissions__name').first()
            perm_name = perm.permissions.name
            
            if perm_name in perm_qtd_criado:
                perm_qtd_criado[perm_name]=b[1] + perm_qtd_criado[perm_name]
            else:
                perm_qtd_criado[perm_name] = b[1]
        except:
            pass

In [ ]:
# perm_qtd_criado

In [ ]:
user_qtd_alter={}
for a in Reporting.history.model.objects.filter(company=company, history_type__in=["~"]).values("history_user"):
    if str(a['history_user']) in user_qtd_alter:
        
        user_qtd_alter[str(a['history_user'])] +=1
    else:
        user_qtd_alter[str(a['history_user'])]=0

In [ ]:
# user_qtd_alter

In [ ]:
perm_qtd_alter={}
for b in user_qtd_alter.items():
#     print(b)
    if b[0] != None:
        try:
            perm = UserInCompany.objects.filter(user__uuid=b[0]).order_by('permissions__name').first()
            perm_name = perm.permissions.name
            
            if perm_name in perm_qtd_alter:
                perm_qtd_alter[perm_name]=b[1] + perm_qtd_alter[perm_name]
            else:
                perm_qtd_alter[perm_name] = b[1]
        except:
            pass

In [ ]:
perm_qtd_criado,perm_qtd_alter

In [ ]:
from tqdm.notebook import tqdm
company = Company.objects.get(name="Arteris Régis Bittencourt - Conserva")
usuarios_que_criaram_ou_alteraram = Reporting.history.model.objects.filter(company=company, history_type__in=["+", "~"]).values_list("history_user", flat=True).annotate(qtd_apt = Count('uuid')).distinct()
# usuarios_que_criaram = Reporting.history.model.objects.filter(company=company, history_type__in=["+"]).values_list("history_user", flat=True).annotate(qtd_apt = Count('uuid')).distinct()
# usuarios_que_alteraram = Reporting.history.model.objects.filter(company=company, history_type__in=["~"]).values_list("history_user", flat=True).annotate(qtd_apt = Count('uuid')).distinct()
permissions = []
for user in tqdm(list(set(usuarios_que_criaram_ou_alteraram))):
    try:
         user_obj = User.objects.get(uuid=user)
    except Exception as e:
        print(e)
        continue
    perm = user_obj.companies_membership.get(company=company).permissions
    if perm:
        permissions.append(perm.name)
print(list(set(permissions)))

In [ ]:
usuarios_que_criaram_ou_alteraram

In [ ]:
UserInCompany._meta.get_fields()

In [ ]:
reps[0].created_by.__dict__

In [ ]:
reps[0].__dict__

In [ ]:
company=Company.objects.get(name="VF Gomes - Vila Bela")
permission=UserPermission.objects.get(companies=company,name='Inativo')
company,permission

In [ ]:
UserInCompany.objects.filter(company=company).update(permissions=permission)

In [ ]:
for a in UserInCompany.objects.filter(company__name="Neovia Vacaria"):
    print(a.user.username,a.permissions.name)

In [ ]:
MultipleDailyReport.objects.filter(number='KapaMun-RDO-2023.00004').delete()

In [ ]:
new_step = ApprovalStep.objects.get(
    approval_flow__company=Company.objects.get(name="Rota do Oeste - Obras"), 
    name="Aguardando Aprovação 2ª camada",
    approval_flow__target_model__icontains="reporting"
)

In [ ]:
new_step.responsible_firms.add(*Firm.objects.filter(company__name="Rota do Oeste - Obras", 
                                                    name__in=['ADP - Lote 2A - Funcional',
                                                                     'ADP - Lote 2A - Reciclagem',
                                                                    'ADP - Lote 2B',
                                                                    'ADP - Lote 3A',
                                                                    'ADP - Lote 3B']))

In [ ]:
lista=["A2– Colaboradores portando crachá padrão Arteris?",
"A2.1– O Encarregado está na frente de serviço?",
"B2 - PDST preenchido e assinado pelos colaboradores e líder?",
"C1– PTC preenchida e assinada (quando aplicável)?",
"Atividade informada ao CCO (OC.)?",
"D1 - Sinalização de obras e serviços está de acordo com o manual de Sinalização Arteris?",
"D1.1– Robô Bandeira está em boas condições (quando aplicável)?",
"D1.2– Moto Bandeira sinalizado e visível (quando aplicável)?",
"D2 - Área de operação de máquinas, movimentação de cargas ou alto risco delimitada ou isolada (quando aplicável)?",
"D3 - Colaborador designado e identificado para vigilância da área de operações de equipamentos 'SPOTTER'?",
"D3.1– Colaborador designado para “ANJO DA GUARDA”?",
"D9 - Inflamáveis estocados em local adequados/sinalizados?",
"D10 - Local específico delimitado/sinalizado para uso de aparelho celular?",
"D11 - Ponto de travessia segura implantado (quando necessário)?",
"D15 - Condições adequadas de banheiros, área de vivência e água potável?",
"D15.1– Local de Trabalho está organizado?",
"D15.2– Resíduos gerados pela atividade foram recolhidos?",
"D17 - Veículo de proteção posicionado adequadamente (quando aplicável)?",
"E1 - Colete/Uniforme em boas condições de uso?",
"E2 - Uniformes possuem tecido de alta visibilidade conservado e faixas refletivas em boas condições?",
"E3 - Todos os colaboradores estão utilizando os EPI’s?",
"E4 - Os EPI's encontram-se em boas condições de uso?",
"F1 - Pneus e rodas em boas condições?",
"F2 - Luzes (freio, faróis, giroflex) funcionando?",
"F3 - Alarme sonoro funcionando?",
"F4 - Câmera de Ré funcionando?",
"F6 - Cintos de Segurança em boas condições?",
"F11 - Documentação do Veículo válida?",
"F13 - Checklist do veículo realizado?",
"F13.1– Veículo devidamente identificado?",
"F14 - Habilitação para transporte coletivo (quando aplicável)?",
"F14.1– Motorista capacitado e habilitado?",
"G1 - Operador capacitado e habilitado?",
"G3 - Checklist do equipamento / máquina realizado?",
"G7 - Equipamentos Manuais em condições adequadas?"]

In [ ]:
dicionario = []
for i,a in enumerate(lista,start=1):
    dicionario.append({'id':i,
                    'displayName':a,
                    'dataType':'boolean',
                      'apiName':'securityWork'+str(i)})

In [ ]:
{
					"key": "isTro",
					"logic": {
						"var": "formData.isTro"
					},
					"header": "TRO?"
				}

In [ ]:
exporter=[]
for a in OccurrenceType.objects.get(company__name="Arteris Litoral Sul - Conserva", name='Segurança do Trabalho').form_fields['measurementColumns']:
    exporter.append({"key": a['key'],
                    "header": a['header'],
                    "logic":{"var": "formData."+a['key']}})

In [ ]:
exporter

In [ ]:
company=Company.objects.get(name="Arteris Litoral Sul - Conserva")

In [ ]:
c=company.history.all().order_by('-history_date')
ce=c[5]

In [ ]:
ce.history_user.username,ce.history_date,ce.custom_options

In [ ]:
# c[5].instance.save()

In [ ]:
import json
for a in Reporting.objects.filter(company=company):
    if a.occurrence_type == None:
        print(a)

In [ ]:
Reporting.objects.filter(company=company,occurrence_type__isnull=True)

In [ ]:
for a in OccurrenceType.objects.filter(company=company):
    if a.form_fields.get('fields'):
        for b in a.form_fields['fields']:
            for x,c in b.items():
                if 'security' in x or 'security' in c:
                    print(a.name)

In [ ]:
oc=OccurrenceType.objects.get(company=company,uuid='8f9a1e51-07fa-4707-9074-097e558f7c0f')

In [ ]:
teste=oc.history.all().order_by('-history_date')

In [ ]:
teste[2].instance.save()

In [ ]:
user = User.objects.get(username="marcio.iran")
user_firms = list((user.user_firms.all()).union(
                    user.user_firms_manager.all()))
len(user_firms)

In [ ]:
UserInFirm.objects.filter(user=user)[0].history.all().order_by('-history_date')

In [ ]:
job=Job.objects.get(number='INT-job-2023.4812')

In [ ]:
job_history=job.history.all().order_by('-history_date')

In [ ]:
j=job_history[0]

In [ ]:
j.archived

In [ ]:
j.history_user,j.history_date

In [ ]:
"""
INT-job-2023.4870 > 05/10/23
INT-job-2023.4835 > 29/09/23
INT-job-2023.4808 > 29/09/23
INT-job-2023.4838 > 29/09/23
INT-job-2023.4812 > 29/09/23
"""


In [ ]:
uf=UserInFirm.history.filter(user__username='marcio.iran')

In [ ]:
for a in uf:
    print(a)

In [ ]:
MultipleDailyReport.objects.get(uuid='158c62dc-fc78-4dc1-a46a-95ec103a6f27').delete()

# Tela branca no login do usuário erro de solicitação de acesso/permissão

In [1]:
for a in UserInCompany.objects.filter(user__username='arthur.barros'):
    try:
        UserPermission.objects.get(companies=a.company,name=a.permissions.name)
        print('tudo certo pra',a.company.name,' Permissão: ',a.permissions.name)
    except Exception as e:
        print(f'essa permissão {a.permissions.name} não existe pra essa unidade {a.company.name} | Erro: {e}')

tudo certo pra Construtora Castilho (Grupo Pátria)  Permissão:  Suporte
tudo certo pra Modera - Maya  Permissão:  Suporte
tudo certo pra Eixo SP  Permissão:  Suporte
tudo certo pra Modera - Consórcio Gerencia OAE SP  Permissão:  Suporte
tudo certo pra Kartado Modelo (Avançado)  Permissão:  Suporte
tudo certo pra Demonstração Ferrovias  Permissão:  Suporte
tudo certo pra Eixo SP - SISOAE  Permissão:  Suporte
tudo certo pra Construtora Castilho (Ferrovias)  Permissão:  Suporte
tudo certo pra Demonstração Saneamento  Permissão:  Suporte
tudo certo pra Construtora Castilho (Ramilândia)  Permissão:  Suporte
tudo certo pra Demonstração Concessionárias  Permissão:  Suporte
tudo certo pra Eixo Sp - Pavimento  Permissão:  Suporte
tudo certo pra Modera - Consolare  Permissão:  Suporte
tudo certo pra Modera - Cortel  Permissão:  Suporte
tudo certo pra LCM - Criciúma  Permissão:  Suporte
tudo certo pra LCM - Gaspar  Permissão:  Suporte
tudo certo pra Eco Noroeste SISOBRAS  Permissão:  Suporte
tudo

In [ ]:
for a in UserPermission.objects.filter(companies__name='VF Gomes - BR158'):
    print(a.name)

In [ ]:
for a in AccessRequest.objects.filter(user__username='heverton.juliano').order_by('-created_at'):
    print(a.user.full_name,a.company.name,a.permissions.name,a.created_by)

In [ ]:
AccessRequest.objects.filter(user__username='talita.arantes')[0].__dict__

In [ ]:
UserPermission.objects.filter(companies__name='Neovia Vacaria',name='Inativo')

In [ ]:
# UserInCompany.objects.filter(user=User.objects.get(username='LETICIA'),
#              company=Company.objects.get(name='Neovia Vacaria')).update(
#              permissions=UserPermission.objects.filter(companies__name='Neovia Vacaria',name='Inativo').first())

# UserInCompany.objects.filter(
#              company=Company.objects.get(name='Neovia Vacaria')).update(
#              permissions=UserPermission.objects.get(companies__name='Neovia Vacaria',uuid='abca0b11-4750-44fd-b411-74386cb51cf1'))

# Relatório Km inicial e final/ coordenadas

In [ ]:
values=[]
for a in Road.objects.filter(company__name="Eixo Sp - Pavimento"):
    inicial=a.marks[list(a.marks)[0]]
    final=a.marks[list(a.marks)[-1]]
    obj={}
    obj={'rodovia':a.name,
        'km inicial':inicial['km'],
        'km final':final['km'],
        'longitude_inicial': inicial['point']['coordinates'][0],
        'latitude_inicial': inicial['point']['coordinates'][1],
        'longitude_final': final['point']['coordinates'][0],
        'latitude_final': final['point']['coordinates'][1]}
    values.append(obj)

In [ ]:
values[0]

In [ ]:
a

# todas as coordenadas

In [ ]:
values=[]
for a in Road.objects.filter(company__name__icontains="Modera"):
    marks=a.marks
    obj={}
    obj={'rodovia':a.name,
        'marcos':[{'km':marks[data]['km'],
                   'longitude': marks[data]['point']['coordinates'][0],
                   'latitude': marks[data]['point']['coordinates'][1]} for data in marks]}
    values.append(obj)

In [ ]:
values

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
filename = 'Rodovias_Modera'
pd.DataFrame().to_excel(filename+'.xlsx')

In [ ]:
for dataU in tqdm(values):
    index={}
    sheetname = dataU['rodovia'].replace('/','-')
    df2 = pd.DataFrame(data=dataU['marcos'])
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df2.to_excel(writer,sheet_name=sheetname,na_rep="-")

# arquivo corrompido

In [ ]:
reporting_file=ReportingFile.objects.get(uuid='d7d05a98-bd23-4fe8-9dc4-c3840bb84af9')
reporting_file.__dict__

In [ ]:
reporting_file.upload.url

In [ ]:
ReportingFile.objects.filter(reporting__in=Reporting.objects.filter(created_by__username='ingrid.santos',found_at__gte='2023-2-1',found_at__lte='2023-2-28'))

In [ ]:
for a in ReportingFile.objects.filter(reporting__in=Reporting.objects.filter(created_by__username='barbara.ferreira')):
    try:
        a.upload.url
    except:
        print(a.reporting)

In [ ]:
for a in ReportingFile.objects.filter(uuid='d7d05a98-bd23-4fe8-9dc4-c3840bb84af9'):
    try:
        a.upload.url
    except:
        print(a.reporting)                                  

In [ ]:
con=Contract.objects.get(subcompany__company__name="Pavicolet (Amapá)",
                            name='Manutenção (Conservação/Recuperação) BR-156/AP - km 659,44 à km 769,80 - Lote 3')
con

In [ ]:
cu=ContractItemUnitPrice.objects.get(
    resource__contract=con,
    resource__resource__name='Desmatamento, destocamento, limpeza de área e estocagem do material de limpeza com árvores de diâmetro até 0,15 m')

In [ ]:
for a in cu.resource.history.all().order_by('-history_date'):
    print(a.history_date)

In [ ]:
lista=['Fresagem Estrutural',
'Recuperação de Base',
'Reconstrução',
'Microfresagem e Microrrevestimento',
'Remendo Corretivo',
'Remendo Emergencial',
'Tapa Buraco',
'Dreno Pavimento  Longitudinal',
'Dreno Pavimento Transversal']
len(lista)

In [ ]:
company=Company.objects.get(name="CART")
company

In [ ]:
for a in OccurrenceType.objects.filter(company=company,name='Reposição/Substituição (baixa refletância/furto/avariada) (solo/aérea)'):
    a.uuid=None
    a.name='Reposição/Substituição (baixa refletância/furto/avariada) (solo/aérea) Inventário'
    a.occurrence_kind='2'
    a.save()
    spec=OccurrenceTypeSpecs(occurrence_type=a, company=company)
    spec.save()
    

In [ ]:
occ=OccurrenceType.objects.filter(company=company).order_by('-created_at')
occ[1]

In [ ]:
for b in lista:
    spec=OccurrenceTypeSpecs(occurrence_type=OccurrenceType.objects.get(name=b,occurrence_kind='9'), company=company)
#     spec.save()
    print(OccurrenceType.objects.get(name=b,occurrence_kind='9'))

In [ ]:
for a in Company.objects.all().order_by('name'):
    print(a.name," | ",a.company_group)

In [ ]:
!cat .env

In [ ]:
import datetime as dt
from tqdm.notebook import tqdm

In [ ]:
company=Company.objects.filter(name='Demo Concessionárias').first()
description="Teste Performance"
user=User.objects.get(username='rlcs')
firm=Firm.objects.get(company=company,name='Teste Performance')
data=dt.datetime.today()
company

In [ ]:
duz=Reporting.objects.filter(company=company,job_id__isnull=True).order_by('-number').distinct()[:200]
quin=Reporting.objects.filter(company=company,job_id__isnull=True).order_by('-number').distinct()[:500]
mil=Reporting.objects.filter(company=company,job_id__isnull=True).order_by('-number').distinct()[:1000]
milq=Reporting.objects.filter(company=company,job_id__isnull=True).order_by('-number').distinct()[:1500]

In [ ]:
job=Job(description=description,
       firm=firm,
       created_by=user,
        company=company,
       start_date=data,
        title=description,
       worker=user)
job.save()
job.reportings.add(*duz)
job.watcher_users.add(*User.objects.filter(username__in=['luan.poletti','renato.dandaluzia']))

In [ ]:
duz.count(),quin.count(),mil.count(),milq.count()

In [ ]:
job.reportings.add(*duz)

In [ ]:
for i,a in enumerate(range(15)):
    duz=Reporting.objects.filter(company=company,job_id__isnull=True).order_by('-number').distinct()[:200]
    job=Job(description=description+str(i),
       firm=firm,
       created_by=user,
        company=company,
       start_date=data,
        title=description+str(i),
       worker=user)
    job.save()
    job.reportings.add(*duz)
    job.watcher_users.add(*User.objects.filter(username__in=['luan.poletti','renato.dandaluzia']))

In [ ]:
for i,a in enumerate(Job.objects.filter(firm=firm)):
    a.save()

In [ ]:
rf_vacaria=ReportingFile.objects.filter(reporting__company__name="Neovia Vacaria")[:4]

In [ ]:
rf=[]
for i,a in tqdm(enumerate(Reporting.objects.filter(job__firm=firm,company=company))):
    for vaca in rf_vacaria:
        rf.append(ReportingFile(created_by=user,
                reporting=a,
                description='teste performance'+str(i),
                km=a.km,
                point=a.point,
                upload=vaca.upload))


In [ ]:
ReportingFile.objects.bulk_create(rf)

In [ ]:
Reporting.objects.filter(job__firm=firm,company=company).count()

In [ ]:
rf_vacaria[0]._meta.get_fields(),rf_vacaria[0].upload,rf_vacaria.count()

In [ ]:
job2=Job.objects.get(uuid='2c514499-a0a4-4f06-8a73-bd7f59eabf36')
job2.reporting_count=146
job2.save()

In [ ]:
for i,a in enumerate(job2.reportings.all()):
    print(i,a)

In [ ]:
job2.save()

In [ ]:
job._meta.get_fields()

In [ ]:
Reporting.objects.filter(company=company,job_id__isnull=True).count()

In [ ]:
for a in Company.objects.all():
    if a.custom_options.get('reporting'):
        print(a.custom_options.get('reporting').get('listColumns'))

In [ ]:
DailyReportConfig._meta.get_fields()

In [ ]:
company=Company.objects.get(name='Construtora Castilho')

In [ ]:
for a in company.history.all():
    for b,c in a.metadata.items():
        if b == 'field_to_automatically_link_reportings_to_rdo':
            print(b,c)

# Relatório Antigo Kapa

In [ ]:
import re

def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [ ]:
def traduz_occ_apiName(field_raw,occ):
    field=convert_string(field_raw)
    for a in occ.form_fields['fields']:
        if a['apiName'] == field:
#             print(a['apiName'],a['displayName'],field)
            return a['displayName'].strip()
        elif a['dataType'] == 'arrayOfObjects':
            for b in a['innerFields']:
                if b['apiName'] == field:
#                     print(b['apiName'],field)
                    return b['displayName'].strip()
        elif a['dataType'] == 'select':
            for c in a['selectOptions']['options']:
                if c['value'] == field:
#                     print(b['apiName'],field)
                    return c['name'].strip()
    
    return field

In [ ]:
reps=[]
for a in Reporting.objects.filter(company__name='Kapa Município'):
    rep={'Serial':a.number,
                'Rodovia':a.road_name,
                'Km':a.km,
                'Km Final':a.end_km,
                'Lote':a.lot,
                "Sentido":a.direction,
                'Faixa':a.lane,
                'Classe':a.occurrence_type.name,
                'Criado Em':a.created_at.strftime('%Y-%m-%dT03:00:00.000Z') if a.created_at is not None else None,
                'Executado Em':a.executed_at.strftime('%Y-%m-%dT03:00:00.000Z') if a.executed_at is not None else None
                }
    
    
    
    
    
    for form_data_field in a.form_data:
#         print(a.form_data[form_data_field])
        for fields in a.occurrence_type.history.all().order_by('-history_date')[1].form_fields['fields']: 
            if fields['apiName'] == form_data_field and fields['dataType'] == 'select':
                for v in fields['selectOptions']['options']:
                    if v['value'] == a.form_data[form_data_field]:
                        rep.update({fields['displayName']: v['value']})
            if fields['dataType'] == 'arrayOfObjects':
                for inner in fields['innerFields']:
                    if inner['apiName'] == form_data_field:
                        rep.update({fields['displayName']: a.form_data[form_data_field]})
            if fields['apiName'] == form_data_field:
                rep.update({fields['displayName']: a.form_data[form_data_field]})
        
        
        
        
#         for name, valor in a.form_data.items():
#             if isinstance(valor,list):
#                 for i,val in enumerate(valor):
#                     for k,v in enumerate(val.items(),start=1):
#     #                     print(v[0])
#                         display=traduz_occ_apiName(v[0],a.occurrence_type)
#     #                     print(display)
#                         rep.update({display+'_'+str(i): v[1]})
#             if valor == 'select':
#             else:
#                 display2 = traduz_occ_apiName(name,a.occurrence_type)
#                 rep.update({display2: valor})

    reps.append(rep)
    

In [ ]:
reps[0]

In [ ]:
company=Company.objects.get(name='CART')

In [ ]:
for a in OccurrenceType.objects.filter(company=company,occurrence_kind='13'):
    print(f"'{a.uuid}'")

In [ ]:
Job._meta.get_fields()

# Relatório Programações

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
jobs=Job.objects.filter(archived=False,start_date__gte='2023-6-1',reporting_count__gt=0).values('company__name','reporting_count','uuid').order_by('company__name')

In [ ]:
jobs_total=Job.objects.filter(archived=False,start_date__gte='2023-6-1').values('company__name').annotate(qtd_prog=Count('uuid')).order_by('company__name')

In [ ]:
jobs.count()

In [ ]:
jobs[7]

In [ ]:
df = pd.DataFrame(data=jobs,columns=['company__name','reporting_count','uuid'])

In [ ]:
df

In [ ]:
jobs_total[0]

In [ ]:
df2 = pd.DataFrame(data=jobs_total,columns=['company__name','qtd_prog'])

In [ ]:
df2

In [ ]:
# df.set_index("Unidade", inplace=True)

In [ ]:
# df.to_excel('jobs.xlsx')

In [ ]:
# with pd.ExcelWriter("Jobs.xlsx") as writer:
#     df.to_excel(writer, sheet_name="Qtd por programação")  
#     df2.to_excel(writer, sheet_name="Qtd programação por unidade")

In [ ]:
m = pd.merge(df, df2, how = 'left', on = 'company__name')

In [ ]:
m

In [ ]:
intervalos=[a for a in range(0,1001,10)]
intervalos[-1]

In [ ]:
m['group'] = pd.cut(m['reporting_count'], intervalos)

# Use groupby() para agrupar os dados pela nova coluna
grouped = m.groupby('group')

# Imprima os grupos e suas contagens
last=[]
for name, group in grouped:
    last.append([name,len(group)])
    print(f'{name}: {len(group)}')

In [ ]:
df3 = pd.DataFrame(data=last,columns=['Grupo de apontamentos','Quantidade de programações por grupo'])

In [ ]:
# df3=df3.drop(0)

In [ ]:
df3

In [ ]:
# Crie o histograma
plt.hist(df3['Quantidade de programações por grupo'], bins=intervalos, density=True)

# Adicione a linha de distribuição
mu, std = stats.norm.fit(df3['Quantidade de programações por grupo'])
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

# Defina os limites do eixo y como [0, 1]
plt.ylim([0, 0.1])

# Adicione rótulos aos eixos x e y
plt.xlabel('Reporting Count')
plt.ylabel('Density')

# Exiba o gráfico
plt.show()

In [ ]:
# Crie o gráfico de barras
ax = df3.plot.bar(x='Grupo de apontamentos', y='Quantidade de programações por grupo', rot=90)

# Adicione rótulos aos eixos x e y
ax.set_xlabel('Grupo de apontamentos')
ax.set_ylabel('Quantidade de programações por grupo')

fig = ax.get_figure()
fig.set_figheight(10)
fig.set_figwidth(20)
# ax.set_yscale('log')
ax.set_ylim(top=1000)

# Exiba o gráfico
plt.show()

In [ ]:
with pd.ExcelWriter("Jobs.xlsx") as writer:
    df.to_excel(writer, sheet_name="Qtd por programação")  
    df2.to_excel(writer, sheet_name="Qtd programação por unidade")
    df3.to_excel(writer, sheet_name="Grupos Programações")
    m.to_excel(writer, sheet_name="Merge")

In [ ]:
# Crie a figura com um tamanho maior
plt.figure(figsize=(10,8))

# Crie uma série Pandas com os dados
s = pd.Series(m['reporting_count'])

# Crie a função de distribuição acumulada empírica
ecdf = np.cumsum(np.sort(s)) / float(s.sum())

# Crie o gráfico de distribuição acumulada
plt.step(np.sort(s), ecdf)

# Defina os valores e rótulos dos marcadores do eixo x
plt.xticks(range(0, 1001, 50))

# Adicione grades ao gráfico
plt.grid(which='both')

# Defina os limites do eixo y como [0, 1]
plt.ylim([0, 1])



# Adicione rótulos aos eixos x e y
plt.xlabel('Reporting Count')
plt.ylabel('Cumulative Distribution')

# Exiba o gráfico
plt.show()

In [ ]:
# Crie uma série Pandas com os dados
s = pd.Series(m['reporting_count'])

# Crie a função de distribuição acumulada empírica
ecdf = s.value_counts(normalize=True).sort_index().cumsum()

# Crie o gráfico de distribuição acumulada interativo
fig = px.line(x=ecdf.index, y=ecdf.values)

# Defina o título do gráfico e dos eixos x e y
fig.update_layout(title='Distribuição Acumulada', xaxis_title='Qtd Apontamentos', yaxis_title='Distribuição Acumulativa')

# Defina o intervalo entre os marcadores do eixo x como 10
fig.update_xaxes(dtick=10)

# Transforme o eixo y em porcentagem
fig.update_yaxes(tickformat='.0%')

fig.add_trace(
    go.Scatter(
        mode='markers+text',
        x=[50,100,200],
        y=[0.93,0.96,0.99],
        text=['50-93%','100-96%','200-99%'],
        textposition="bottom center",
        marker=dict(
            color='red',
            size=5
        ),
        showlegend=False
    )
)

# Exiba o gráfico
fig.show()

In [ ]:
# Replace 'fig' with your Plotly figure object
pio.write_html(fig, file='chart.html', auto_open=True)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for a in OccurrenceType.objects.filter(company__name='Construtora Castilho (Ferrovias)'):
    a.form_fields['measurementColumns'] = []
#     a.save()

In [ ]:
MultipleDailyReport.objects.filter(number='CART-RDO-2023.16148').delete()

In [ ]:
for a in OccurrenceType.objects.filter(company__name='Rota do Oeste - Obras'):
    dados=[]
    for b in a.form_fields['fields']:
        if b['dataType'] == 'arrayOfObjects':
            for c in b['innerFields']:
                if c['apiName'] == 'unidades':
                    print(a.name)
#                     c['apiName'] = 'unidades'
#                     a.save()

In [ ]:
for a in Firm.objects.filter(company__name='Rota do Oeste - Obras'):
    print(a.name)

In [ ]:
lista=['Totem não funciona',
'Iluminação interna queimada',
'Iluminação externa queimada',
'Tomada externa com defeito',
'Tomada interna com defeito',]
len(lista)

In [ ]:
company=Company.objects.get(name='Eixo SP')
occ=OccurrenceType.objects.get(company__name='Eixo SP',name='Manutenção de junta de dilatação')
occ,company

In [ ]:
occ.form_fields['displayName']

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(lista):
    new_occ=occ
    new_occ.uuid=None
    new_occ.name=a.strip()
    new_occ.occurrence_kind='23'
    new_occ.form_fields['name'] = a.strip()
    new_occ.form_fields['displayName'] = a.strip()
    new_occ.save()
    
    spec=OccurrenceTypeSpecs(occurrence_type=new_occ, company=company)
    spec.save()

In [ ]:
!pip list

In [ ]:
mdr=MultipleDailyReport.objects.get(uuid='b24ba9f3-ebe9-4303-aa00-3ada3cbe46ee')
rep=Reporting.objects.get(uuid='0068d45e-511c-4082-8622-c0e1922d8f96')

In [ ]:
mdr._meta.get_fields()

In [ ]:
mdr.reportings.all()

In [ ]:
Reporting._meta.get_fields()

In [ ]:
rep._meta.get_fields()

In [ ]:
Reporting.objects.filter(occurrence_type__occurrence_kind='2',
                         company__name='Eixo SP',
                         created_at__gte='2023-10-09',
                         reporting_multiple_daily_reports__gte=1).count()

In [ ]:
for a in Reporting.objects.filter(occurrence_type__occurrence_kind='2',
                                  company__name='Eixo SP',
                                  created_at__gte='2023-01-01',
                                  reporting_multiple_daily_reports__gte=1).order_by('created_at'):
    if a.reporting_multiple_daily_reports.all().exists():
        for b in a.reporting_multiple_daily_reports.all():
            
            print(a.number,'|',b.number,'|',b.created_at.strftime('%d-%m-%Y'),'|',b.firm.name)

In [ ]:
mdr=MultipleDailyReport.objects.get(uuid='bbf805e7-5ca5-4d74-8b77-fc25a43d10a2')

In [ ]:
header_info = mdr.header_info

In [ ]:
mdr.firm.subcompany.name

In [ ]:
for a in MultipleDailyReport.objects.filter(company__name="Construtora Castilho"):
    if a.firm.subcompany.name == 'Obra Paraguaçu Paulista':
        a.header_info = header_info
#         a.save()

In [ ]:
for a in mdr.history.all().order_by('-created_at'):
    print(a.history_user,a.__dict__)

In [ ]:
sc=SubCompany.objects.get(uuid='f137fccf-6654-4ac8-9a27-4fd3f156eac5')

In [ ]:
for a in sc.history.all().order_by('-history_date'):
    print(a.cnpj)

In [ ]:
sc.__dict__

In [ ]:
UserInCompany.objects.filter(user__username='MATEUS.LIMA')

In [ ]:
for a in Reporting.objects.filter(number__in=['111368.2023','111211.2023','111210.2023']):
    print(a.form_data)

In [ ]:
occs=OccurrenceType.objects.filter(occurrence_kind__in=['31','32','33'],company__name='Rota do Oeste - Obras')

In [ ]:
occs[0].__dict__

In [ ]:
{"order": 3,
      "apiName": "ensaio",
      "dataType": "select",
      "displayName": "Ensaio",
      "selectOptions": {"options": [{"name": "Determinação do teor de umidade e grau de compactação 'in situ' - Reciclagem",
         "value": "1"},
        {"name": "Taxa de aplicação da emulsão asfáltica (TSD)", "value": "2"},
        {"name": "Taxa de aplicação da emulsão asfáltica (Capa selante)",
         "value": "3"},
        {"name": "Taxa de aplicação dos agregados (Capa selante)",
         "value": "4"},
        {"name": "Taxa de aplicação dos agregados (TSD)", "value": "5"},
        {"name": "Taxa de aplicação da emulsão asfáltica (Pintura de Ligação)",
         "value": "6"},
        {"name": "Extração de corpos de prova e grau de compactação (CAUQ)",
         "value": "7"},
        {"name": "Determinação da taxa de distribuição do cimento - Reciclagem",
         "value": "8"},
        {"name": "Determinação de finura - Reciclagem", "value": "9"},
        {"name": "Certificado do fabricante/distribuidor - Reciclagem",
         "value": "10"},
        {"name": "Abrasão Los Angeles", "value": "11"},
        {"name": "Índice de forma", "value": "12"},
        {"name": "Partículas lamelares", "value": "13"},
        {"name": "Durabilidade", "value": "14"},
        {"name": "Equivalente de areia", "value": "15"},
        {"name": "Análise granulométrica da mistura reciclada", "value": "16"},
        {"name": "Ensaio de compactação (proctor)", "value": "17"},
        {"name": "Determinação do teor de umidade da mistura reciclada",
         "value": "18"},
        {"name": "Ensaio de resistência à compressão axial aos 7 dias de cura",
         "value": "19"},
        {"name": "Ensaio de resistência à compressão axial aos 7 dias de cura",
         "value": "20"},
        {"name": "Ensaio de resistência à compressão axial aos 28 dias de cura",
         "value": "21"},
        {"name": "Ensaio de resistência à tração por compressão diametral aos 7 dias de cura",
         "value": "22"},
        {"name": "Ensaio de resistência à tração por compressão diametral aos 28 dias de cura",
         "value": "23"}],
       "optionsFilter": {"filter": [{"var": "options"},
         {"if": [{"==": [{"var": {"cat": ["data.formData.servico",
                {"var": "index.0"},
                ".identificacaoDeControle"]}},
             1]},
           {"<=": [{"var": "value"}, 8]},
           {">": [{"var": "value"}, 8]}]}]}}},

In [ ]:
for a in occs:
    for fields in a.form_fields['fields']:
        if fields['apiName'] == 'servico' and fields['dataType'] == 'arrayOfObjects':
            for inner in fields['innerFields']:
                if inner['apiName'] == 'ensaio':
                    inner['selectOptions']['options']=[{"name": "Determinação do teor de umidade e grau de compactação 'in situ' - Reciclagem",
         "value": "1"},
        {"name": "Taxa de aplicação da emulsão asfáltica (TSD)", "value": "2"},
        {"name": "Taxa de aplicação da emulsão asfáltica (Capa selante)",
         "value": "3"},
        {"name": "Taxa de aplicação dos agregados (Capa selante)",
         "value": "4"},
        {"name": "Taxa de aplicação dos agregados (TSD)", "value": "5"},
        {"name": "Taxa de aplicação da emulsão asfáltica (Pintura de Ligação)",
         "value": "6"},
        {"name": "Extração de corpos de prova e grau de compactação (CAUQ)",
         "value": "7"},
        {"name": "Determinação da taxa de distribuição do cimento - Reciclagem",
         "value": "8"},
        {"name": "Determinação de finura - Reciclagem", "value": "9"},
        {"name": "Certificado do fabricante/distribuidor - Reciclagem",
         "value": "10"},
        {"name": "Abrasão Los Angeles", "value": "11"},
        {"name": "Índice de forma", "value": "12"},
        {"name": "Partículas lamelares", "value": "13"},
        {"name": "Durabilidade", "value": "14"},
        {"name": "Equivalente de areia", "value": "15"},
        {"name": "Análise granulométrica da mistura reciclada", "value": "16"},
        {"name": "Ensaio de compactação (proctor)", "value": "17"},
        {"name": "Determinação do teor de umidade da mistura reciclada",
         "value": "18"},
        {"name": "Ensaio de resistência à compressão axial aos 7 dias de cura",
         "value": "19"},
        {"name": "Ensaio de resistência à compressão axial aos 7 dias de cura",
         "value": "20"},
        {"name": "Ensaio de resistência à compressão axial aos 28 dias de cura",
         "value": "21"},
        {"name": "Ensaio de resistência à tração por compressão diametral aos 7 dias de cura",
         "value": "22"},
        {"name": "Ensaio de resistência à tração por compressão diametral aos 28 dias de cura",
         "value": "23"}]
    a.save()

In [ ]:
name= "Taxa de aplic. da emulsão asfáltica(TSD)"
len(name)

In [ ]:
for a in occs[:1]:
    for fields in a.form_fields['fields']:
        if fields['apiName'] == 'servico' and fields['dataType'] == 'arrayOfObjects':
            for i,inner in enumerate(fields['innerFields']):
                if inner['apiName'] == 'ensaioss':
                    del fields['innerFields'][i]
                    
    print(a)
    a.save()

In [ ]:
a.__dict__

In [ ]:
import json
a.form_fields=json.loads(json.dumps(a.form_fields).replace('ensaio','ensaios').replace('Ensaio','Ensaios').replace(": 'id'",": 'ids'"))

In [ ]:
a.form_fields

In [ ]:
for a in occs[:1]:
    for fields in a.form_fields['fields']:
        print(fields['apiName'])
        if fields['apiName'] == 'servico' and fields['dataType'] == 'arrayOfObjects':
            for inner in fields['innerFields']:
                print('\t',inner['apiName'])

In [ ]:
reps=Reporting.objects.filter(occurrence_type__name='1. Procedência do material')

In [ ]:
for a in reps:
    a.form_data['codigo_rastreio'] = a.form_data['codigo_rastreio'].strip()
    a.save()

In [ ]:
occ=OccurrenceType.objects.get(name="Emulsões Asfálticas",occurrence_kind__in=['31','32','33'],company__name='Rota do Oeste - Obras')

In [ ]:
history=occ.history.filter(history_date__lte='2023-11-1').order_by('-history_date')

In [ ]:
history[0].instance.save()

In [ ]:
outro=1
print(f'Rodovia|Seq|km')
for a in Road.objects.filter(company__name='Construtora Castilho (Ferrovias)'):
    for i,mark in a.marks.items():
        if (int(i) - outro) >1: 
            print(a.name,'|',i,'|',mark['km'])
        outro=int(i)

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(OccurrenceType.objects.filter(occurrence_kind__in=[18,19,20,21,22,23],company__name='Eixo SP')):
    for field in a.form_fields['fields']:
        if field.get('displayName') in ['Código de edital']:
            del field['autofill']
    a.save()

In [ ]:
company=Company.objects.get(name="Via Brasil")
ident = ServiceOrderActionStatus.objects.get(companies=company,name="Identificado")
em_validacao = ServiceOrderActionStatus.objects.get(companies=company,name="Em Validação")

In [ ]:
Reporting.objects.filter(status=ident,company=company).exclude(occurrence_type__occurrence_kind=2).update(status=em_validacao)

In [ ]:
aps=ApprovalStep.objects.get(name='Aguardando Aprovação',approval_flow__company=company,approval_flow__target_model='reportings.Reporting')

In [ ]:
ApprovalTransition.objects.filter(destination=aps,name='Solicitar Análise')[0].__dict__

In [ ]:
Reporting.objects.filter(approval_step=aps).update(editable=True)

In [ ]:
company=Company.objects.get(name='Arteris Fernão Dias - Conserva')
firm = Firm.objects.get(name='Multifuncional - Lote 01',company=company)
user=User.objects.get(username='rodolfo.rodrigues')

In [ ]:
for a in Reporting.history.filter(firm=firm,company=company,km__gte=494,km__lte=496,history_type='-'):
    print('km:',a.km,'deletado em:',a.history_date.strftime('%d-%m-%Y'),'serial:',a.number,'criador:',a.created_by.full_name)

In [ ]:
Reporting.objects.filter(company__name="Caminhos da Serra Gaúcha",occurrence_type__occurrence_kind=2).count()

In [ ]:
rep_edit=[]
for a in tqdm(Reporting.objects.filter(company__name="Caminhos da Serra Gaúcha",occurrence_type__occurrence_kind=2)):
    a.form_data['monitoring_id'] = 'Cadastro Inicial'
    rep_edit.append(a)

In [ ]:
user=User.objects.get(username='rlcs')
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=2500,user=user)

In [ ]:
job=Job.objects.get(number__icontains='pavicolet-job-2023.33',company__name='Pavicolet (Amapá)')

In [ ]:
for a in job.history.all().order_by('-history_date'):
#     for b in Reporting.objects.filter(company=a.company,job__number=a):
#         print(b)
    print(a.history_date.strftime('%d-%m-%Y %HH:%MM'),a.reporting_count,a.history_user.full_name)

In [ ]:
for a in ActionLog.objects.filter(company__name='Pavicolet (Amapá)',object_id=job.uuid):
    print(a)

In [ ]:
Job.objects.filter(created_by__username='karla.zorzam',company__name='Modera - Consórcio ME2').delete()

In [ ]:
company_origin= Company.objects.get(name='Sinalizavia (Municípios)')
company = Company.objects.get(name='Sinaliza Via (Rodovias)')
company_origin,company

In [ ]:
ApprovalFlow.objects.filter(company=company_origin)

In [ ]:
for a in ApprovalFlow.objects.filter(company=company_origin):
    a.uuid = None
    a.company=company
    a.save()

In [ ]:
for a in ApprovalStep.objects.filter(approval_flow__company=company_origin):
    a.uuid=None
    a.approval_flow = ApprovalFlow.objects.get(name=a.approval_flow.name,company=company)
    a.save()

In [ ]:
for a in ApprovalTransition.objects.filter(origin__approval_flow__company=company_origin):
    a.uuid=None
    a.origin = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.origin.name,
                                        approval_flow__target_model=a.origin.approval_flow.target_model)
    a.destination = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.destination.name,
                                        approval_flow__target_model=a.destination.approval_flow.target_model)
    a.save()
    print(a.origin,a.destination)

In [ ]:
job=Job.history.filter(number='Modera-job-2023.5').order_by('-history_date')

In [ ]:
job[0].__dict__

In [ ]:
for a in job:
    print(a.history_date,a.created_by,a.worker)
    for b in a.firm.history.all():
        print(b.__dict__)
#     print(a.firm.users.all())

In [ ]:
import json
 
original_list = [
    "ST01", "ST02", "ST03", "ST04", "ST05", "ST06", "ST07", "ST08", "ST09", "ST10", 
    "ST100", "ST11", "ST101", "ST102", "ST13", "ST103", "ST104", "ST15", "ST105", 
    "ST105A", "ST105B", "ST106", "ST17", "ST107", "ST108", "ST19", "ST109", "ST21", 
    "ST110", "ST23", "ST111", "ST112", "ST24", "ST113", "ST114", "ST25", "ST115", 
    "ST116", "ST28", "ST117", "ST118", "ST30", "ST118A", "ST118B", "ST118C", "ST119", 
    "ST12", "ST120", "ST121", "ST122", "ST123", "ST124", "ST125", "ST126", "ST127", 
    "ST128", "ST129", "ST31", "ST130", "ST131", "ST132", "ST133", "ST134", "ST135", 
    "ST136", "ST137", "ST138", "ST139", "ST14", "ST140", "ST141", "ST142", "ST143", 
    "ST144", "ST145", "ST146", "ST147", "ST147A", "ST147B", "ST148", "ST149", "ST32", 
    "ST150", "ST151", "ST152", "ST153", "ST154", "ST155", "ST156", "ST157", "ST158", 
    "ST159", "ST16", "ST160", "ST161", "ST162", "ST163", "ST164", "ST165", "ST166", 
    "ST167", "ST168", "ST169", "ST34", "ST170", "ST171", "ST172", "ST173", "ST174", 
    "ST175", "ST176", "ST177", "ST178", "ST179", "ST18", "ST180", "ST181", "ST183", 
    "ST185", "ST186", "ST187", "ST188", "ST189", "ST35", "ST190", "ST191", "ST192", 
    "ST193", "ST194", "ST195", "ST197", "ST198", "ST199", "ST20", "ST200", "ST201", 
    "ST203", "ST204", "ST205", "ST206", "ST207", "ST208", "ST209", "ST36", "ST210", 
    "ST211", "ST212", "ST214", "ST215", "ST216", "ST217", "ST218", "ST219", "ST22", 
    "ST220", "ST221", "ST222", "ST223", "ST224", "ST225", "ST226", "ST227", "ST228", 
    "ST229", "ST37", "ST230", "ST231", "ST232", "ST233", "ST235", "ST237", "ST238", 
    "ST239", "ST39", "ST240", "ST241", "ST243", "ST245", "ST247", "ST248", "ST249", 
    "ST41", "ST250", "ST251", "ST252", "ST253", "ST254", "ST255", "ST257", "ST259", 
    "ST26", "ST261", "ST263", "ST265", "ST267", "ST269", "ST27", "ST271", "ST273", 
    "ST275", "ST277", "ST279", "ST43", "ST281", "ST283", "ST285", "ST287", "ST289", 
    "ST29", "ST291", "ST293", "ST295", "ST296", "ST298", "ST44", "ST300", "ST302", 
    "ST304", "ST306", "ST308", "ST46", "ST310", "ST312", "ST314", "ST316", "ST317", 
    "ST319", "ST31A", "ST48", "ST321", "ST322", "ST323", "ST325", "ST327", "ST329", 
    "ST33", "ST331", "ST332", "ST334", "ST335", "ST337", "ST339", "ST50", "ST341", 
    "ST343", "ST345", "ST347", "ST348", "ST53", "ST350", "ST352", "ST354", "ST356", 
    "ST358", "ST359", "ST55", "ST361", "ST363", "ST365", "ST367", "ST369", "ST57", 
    "ST371", "ST373", "ST375", "ST377", "ST378", "ST38", "ST380", "ST382", "ST384", 
    "ST386", "ST388", "ST59", "ST390", "ST392", "ST394", "ST396", "ST398", "ST40", 
    "ST400", "ST402", "ST404", "ST406", "ST408", "ST60", "ST410", "ST411", "ST413", 
    "ST415", "ST417", "ST419", "ST42", "ST421", "ST423", "ST425", "ST427", "ST429", 
    "ST61", "ST431", "ST433", "ST435", "ST437", "ST439", "ST63", "ST441", "ST443", 
    "ST445", "ST447", "ST449", "ST45", "ST451", "ST453", "ST455", "ST457", "ST458", 
    "ST459", "ST65", "ST461", "ST463", "ST464", "ST467", "ST469", "ST47", "ST471", 
    "ST473", "ST475", "ST477", "ST479", "ST67", "ST481", "ST484", "ST486", "ST488", 
    "ST49", "ST490", "ST492", "ST494", "ST496", "ST498", "ST69", "ST500", "ST502", 
    "ST504", "ST506", "ST508", "ST51", "ST510", "ST512", "ST514", "ST516", "ST517", 
    "ST518", "ST52", "ST520", "ST522", "ST524", "ST525", "ST526", "ST527", "ST529", 
    "ST71", "ST53A", "ST53B", "ST54", "ST73", "ST55A", "ST55B", "ST56", "ST75", "ST57A", 
    "ST57B", "ST58", "ST77", "ST59A", "ST59B", "ST79", "ST81", "ST62", "ST83", "ST64", 
    "ST85", "ST66", "ST66B", "ST86", "ST68", "ST88", "ST70", "ST90", "ST72", "ST92", 
    "ST74", "ST94", "ST76", "ST96", "ST78", "ST98", "ST80", "ST82", "ST84", "ST87", 
    "ST89", "ST91", "ST93", "ST95", "ST97", "ST99"
]
 
formatted_list = [{"name": name, "value": str(i + 1)} for i, name in enumerate(original_list)]
 
# Converter para aspas duplas
formatted_json = json.dumps(formatted_list, indent=4)
 
print(formatted_json)

In [ ]:
import openpyxl as op

In [ ]:
filename2 = "APONTAMENTOS DE SELA TRINCA - MUDAR ORIGEM PARA INTERNA (1).xlsx"
wb = op.load_workbook(filename2)
ws = wb['Apontamentos']

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
erros=[]
for a in values:
    try:
        rep=Reporting.objects.get(number=a['Serial'])
        rep.form_data['demand_origin']=['1']
        rep.save()
    except Exception as e:
        erros.append(a)
        print(a['Serial'],e)

In [ ]:
occs=OccurrenceType.objects.filter(occurrence_kind__in=[5,6],company__name='Eixo Sp - Pavimento')

In [ ]:
apis=[]
for a in occs[:2]:
    for campo in a.form_fields['fields']:
        temp={}
        temp[a.name]=[]
        temp[a.name].append(campo['apiName'])
        if campo['dataType'] == "arrayOfObjects":
            t={}
            t['inner']=[inner['apiName'] for inner in campo['innerFields']]
        temp[a.name].append(t)
    apis.append(temp)
        

In [ ]:
apis

In [ ]:
company=Company.objects.get(name='Eixo Sp - Pavimento')

In [ ]:
reps=Reporting.objects.filter(company=company,occurrence_type__occurrence_kind__in=[5,6])
reps.count()

In [ ]:
rep_edit=[]
for a in reps:
    if 'service' in a.form_data.keys():
        rep_edit.append({a.number:[st.get('servicetype') for st in a.form_data['service']]})
#         for st in a.form_data['service']:
#             rep_edit.append()
#             print(st.get('servicetype'))

In [ ]:
rep_edit

In [ ]:
lista_nova={
    "1": "39",
    "2": "40",
    "3": "40",
    "4": "40",
    "5": "40",
    "6": "42",
    "7": "40",
    "8": "40",
    "9": "44",
    "10": "46",
    "11": "45",
    "12": "49",
    "13": "50",
    "14": "51",
    "15": "54",
    "16": "52",
    "17": "55",
    "18": "57",
    "19": "58",
    "20": "59",
    "21": "62",
    "22": "63",
    "23": "74",
    "24": "65",
    "25": "66",
    "26": "67",
    "27": "68",
    "28": "69",
    "29": "70",
    "31": "64",
    "32": "71",
    "33": "72",
    "35": "73",
    "36": "60",
    "37": "40"
}

In [ ]:
outra=[]
for r in rep_edit:
    for k,v in r.items():
        try:
            rep=Reporting.objects.get(number=k,company=company,created_at__lte='2023-10-10')
            for i,a in enumerate(rep.form_data['service']):
                if a.get('servicetype'):
                    a['servicetype'] = lista_nova[a.get('servicetype')] if lista_nova.get(a.get('servicetype')) else a['servicetype']
            rep.save()
            outra.append(rep)
        except:
            print(rep.form_data['service'])

In [ ]:
from helpers.histories import bulk_update_with_history
user=User.objects.get(username='rlcs')
apontamentos = bulk_update_with_history(outra,Reporting, use_django_bulk=True,batch_size=1000,user=user)

In [ ]:
MultipleDailyReport.objects.get(number='ROO-RDO-2023.00712')

In [ ]:
dados=[]
for a in OccurrenceType.objects.filter(company__name="Eixo Sp - Pavimento"):
    temp={}
    temp[a.name]=[]
    for field in a.form_fields['fields']:
        if field['dataType'] == 'arrayOfObjects':
            field_arr={}
            field_arr[field['apiName']]=[]
            for inner in field['innerFields']:
                field_arr[field['apiName']].append(inner['apiName'])
            temp[a.name].append({field['apiName']:field_arr[field['apiName']]})
        else:
            temp[a.name].append(field['apiName'])
    dados.append(temp)

In [ ]:
dados

In [ ]:
reps=Reporting.history.filter(company__name='LCM Construção e Comércio',history_type='-')

In [ ]:
dados=[]
for a in reps:
    dados.append({"Deletado por":a.history_user.full_name,"Data Exclusão":a.history_date.strftime('%d/%m/%Y'),"Apontamento":a.number})

In [ ]:
pd.DataFrame(data=dados).to_excel('LCMExclusoes.xlsx')

In [ ]:
company=Company.objects.get(name="Arteris Fernão Dias - Conserva")

In [ ]:
cs=ContractService.objects.get(uuid='156b44a7-2127-4d6c-a733-fa4b6ed674b0')

In [ ]:
for a in cs.contract_item_administration.all():
    h=a.history.

In [ ]:
cs.contract_item_administration.all()[0].history.first()

In [ ]:
DailyReportContractUsage.objects.get(uuid='85aaef28-3329-479a-9680-799d4f80e2e1').__dict__

In [ ]:
for a in worker:
    h=a.history.order_by('-history_date').first()
    print(h.history_user, h.history_date)

In [ ]:
cia=DailyReportContractUsage.objects.filter(worker=worker)

In [ ]:
cia

In [ ]:
DailyReportContractUsage._meta.get_fields()

In [ ]:
FieldSurvey.objects.first().__dict__

In [ ]:
for a in OccurrenceType.objects.filter(company__name='CART'):
    for field in a.form_fields['fields']:
        if field.get('displayName') and 'campo' in field.get('displayName').lower():
            print(a)

In [ ]:
OccurrenceType.objects.filter(company__name='CART')[0].form_fields['fields']

# CART Alteração em massa comprimento, largura e area executada

In [ ]:
from openpyxl import load_workbook
filename = "00 - Producao Compilado - Kartado.xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[1]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
from tqdm.notebook import tqdm
rep_edit=[]
company=Company.objects.get(name='CART')
for a in tqdm(values):
    
    if str(a['Serial'])[-4:] == '2021':
        number = int(str(a['Serial'])[:-4])
        number = '{:05}'. format(number) + '.2021'    
    elif str(a['Serial'])[-4:] == '2022':
        number = int(str(a['Serial'])[:-4])
        number = '{:05}'. format(number) + '.2022'    
    elif str(a['Serial'])[-4:] == '2023':
        number = int(str(a['Serial'])[:-4])
        number = '{:05}'. format(number) + '.2023'
    elif str(a['Serial'])[-4:] == '2024':
        number = int(str(a['Serial'])[:-4])
        number = '{:05}'. format(number) + '.2024'
    else:
        number = a['Serial']
    
    rep=Reporting.objects.get(number=number,company=company)
#     rep.executed_at = a['Executado em']
    if a['Comprimento Executado (m)'] is not None:
        rep.form_data['executed_length'] = a['Comprimento Executado (m)']
    if a['Largura Executada (m)'] is not None:
        rep.form_data['executed_width'] = a['Largura Executada (m)']
    if a['Área Executada (m²)'] is not None:
        rep.form_data['executed_area'] = a['Área Executada (m²)']
#     if a['Peso Executado'] is not None:
#         rep.form_data['executed_weight'] = a['Peso Executado']
    rep_edit.append(rep)

In [ ]:
user=User.objects.get(username="rlcs")
user

In [ ]:
from helpers.histories import bulk_update_with_history
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=800,user=user)

In [ ]:
company=Company.objects.get(name='Arteris Via Paulista - SISOBRAS/SISQUALI')
up=UserPermission.objects.filter(name='Suporte',companies=company)
company,up

In [ ]:
for a in up:
    a.companies.remove(company)
    uperm = a
    uperm.uuid=None
    uperm.companies.set([company])
    uperm.save()
    UserInCompany.objects.filter(company=company,permissions__name=a.name).update(permissions=uperm)

In [ ]:
rep=Reporting.objects.get(uuid='e1817019-5ed7-41dc-99b5-270d8b31aa99')

In [ ]:
rep.__dict__

In [ ]:
reps=Reporting.objects.filter(number__in=['111211.2023','111210.2023','111368.2023'])

In [ ]:
for a in reps:
    a.occurrence_type = OccurrenceType.objects.get(name='Reposição/Substituição (baixa refletância/furto/avariada) (solo/aérea)',
                                                   company__name='CART')
    a.save()

In [ ]:
company=Company.objects.get(name='Eixo Sp - Pavimento')

In [ ]:
reps=Reporting.objects.filter(occurrence_type__occurrence_kind__in=[5,6,8],company=company
                             ).exclude(form_data__has_key='service')

In [ ]:
reps.count()

In [ ]:
reps=Reporting.objects.filter(uuid='bc957403-87af-4473-8ff0-779e3df894ab')

In [ ]:
for a in reps:
    a.form_data['service'] = []
    a.save()

In [ ]:
team=Firm.objects.get(name='RMC_T3_01')

In [ ]:
team

In [ ]:
mdr=MultipleDailyReport.objects.get(number='CART-RDO-2023.18902')

In [ ]:
mdr.firm=team
mdr.save()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
occs = OccurrenceType.objects.all().values('name')

In [ ]:
occs

In [ ]:
for a in occs:
    a['tamanho_string'] = len(a['name'])

In [ ]:
occs

In [ ]:
df = pd.DataFrame(data=occs)
df

In [ ]:
intervalos=[a for a in range(0,200,10)]
intervalos

In [ ]:
df['group'] = pd.cut(df['tamanho_string'], intervalos)

# Use groupby() para agrupar os dados pela nova coluna
grouped = df.groupby('group')

# Imprima os grupos e suas contagens
last=[]
for name, group in grouped:
    last.append([name,len(group)])
    print(f'{name}: {len(group)}')

In [ ]:
df2 = pd.DataFrame(data=last,columns=['Range de Caracteres','Quantidade Classes por grupo de caracteres'])

In [ ]:
df2

In [ ]:
df2.to_excel('Quantidade Classes por grupo de caracteres.xlsx')

In [ ]:
# Crie uma série Pandas com os dados
s = pd.Series(df2['Quantidade Classes por grupo de caracteres'])

# Crie a função de distribuição acumulada empírica
ecdf = s.value_counts(normalize=True).sort_index().cumsum()

# Crie o gráfico de distribuição acumulada interativo
fig = px.line(x=ecdf.index, y=ecdf.values)

# Defina o título do gráfico e dos eixos x e y
fig.update_layout(title='Distribuição Acumulada', xaxis_title='Quantidade por grupo de caracteres', yaxis_title='Distribuição Acumulativa')

# Defina o intervalo entre os marcadores do eixo x como 10
fig.update_xaxes(dtick=500)

# Transforme o eixo y em porcentagem
fig.update_yaxes(tickformat='.0%')

# fig.add_trace(
#     go.Scatter(
#         mode='markers+text',
#         x=[50,100,200],
#         y=[0.93,0.96,0.99],
#         text=['50-93%','100-96%','200-99%'],
#         textposition="bottom center",
#         marker=dict(
#             color='red',
#             size=5
#         ),
#         showlegend=False
#     )
# )

# Exiba o gráfico
fig.show()

In [ ]:
mdr=MultipleDailyReport.objects.get(uuid='09d4c37f-66f1-4e0a-88aa-482b21d1d471')

In [ ]:
mdr.multiple_daily_report_occurrences.all()[0].__dict__

In [ ]:
from helpers.strings import get_obj_from_path
from helpers.apps.reportings import get_lane

In [ ]:
company=Company.objects.get(name="CART")

In [ ]:
from openpyxl import load_workbook
filename = "CORRECAO RDO (1).xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[0]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
possible_lane_path = (
        "daily_report_occurrence__fields__description__selectoptions__options"
    )
occurrences = get_obj_from_path(
        company.custom_options, possible_lane_path
    )

In [ ]:
occurrences

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(values):
    rdo=MultipleDailyReport.objects.get(number=a['Serial do RDO'])
    firm=rdo.firm
    description = next(o['value'] for o in occurrences if o['name'] == a['Descrição'])
    origin = 'EXECUTOR' if a['Origem'] == 'Executora' else 'INSPECTION'
    starts_at = a['Horário de Início']
    ends_at = a['Horário de Fim']
    impact_duration = a['Impacto']
    dro = DailyReportOccurrence(firm=firm,description=description,origin=origin,
                               starts_at=starts_at,ends_at=ends_at,impact_duration=impact_duration)
    dro.save()
    rdo.multiple_daily_report_occurrences.add(dro)

In [ ]:
dro

In [ ]:
rdo

In [ ]:
Firm.objects.filter(company__name='Demo Concessionárias',subcompany__name='M&A LTDA')

In [ ]:
occs=OccurrenceType.objects.filter(company__name__startswith='Arteris')

In [ ]:
occs.count()

In [ ]:
occs[0].company.all()[0].name

In [ ]:
for a in occs:
    if a.form_fields['fields'] is not None or a.form_fields['fields'] is not {}:
        for field in a.form_fields['fields']:
            try:
                if 'city' in field.get('apiName'):
                    print(field['dataType'],a.name,next(b.name for b in a.company.all()))
            except:
                pass

In [ ]:
MeasurementBulletin._meta.get_fields()

In [ ]:
mb=MeasurementBulletin.objects.filter(number__in=["AFD-BM-2023.12", "AFD-BM-2023.14", "AFD-BM-2023.13"],
                                     contract__subcompany__company=Company.objects.get(name="Arteris Fernão Dias - Conserva"))

In [ ]:
mb

In [ ]:
mb[0].__dict__

In [ ]:
up=UserPermission.objects.filter(companies__name='CART')

In [ ]:
for a in up:
    print(a.name,a.permissions['MultipleDailyReport'])

In [ ]:
for a in UserInCompany.objects.filter(company__name__icontains='Kapa'):
    print(a.permissions.name,a.user.username)

In [ ]:
company=Company.objects.get(name="CART")

In [ ]:
from openpyxl import load_workbook
filename = "Unidades ativas Kartado Geral (1).xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[2]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
values[0]['Unidades:'].split(' | ')[0].strip()

In [ ]:
ativas=[]
for a in values:
    ativas.append(Company.objects.get(name=a['Unidades:'].split('|  [')[0].strip()).uuid)
#     Company.objects.get(name=a['Unidades:'].split('|  [')[0].strip())

In [ ]:
Company.objects.filter(uuid__in=ativas).update(active=True)

In [ ]:
Company.objects.all()[0].active

In [ ]:
Company.objects.all().exclude(uuid__in=ativas).update(active=False)

In [ ]:
up=UserPermission.objects.filter(companies__name='CART')

In [ ]:
up.count()

In [ ]:
company=Company.objects.get(name='Sinalizavia (Municípios)')

In [ ]:
for a in company.history.all().order_by('-history_date'):
    print(a.history_date,a.history_user)

In [ ]:
company.history.all().order_by('-history_date')[2].history_user,company.history.all().order_by('-history_date')[2].history_date

In [ ]:
rep=Reporting.objects.get(number='SV-SinV-2023.00010')

In [ ]:
rep.history.all().order_by('-history_date')[0].history_user

In [ ]:
Reporting.history.model.objects.filter(
    history_date__date="2023-07-24", 
    history_type="-", 
    form_data__artesp_code="618034"
)

In [ ]:
occs=OccurrenceType.objects.filter(company__name='CART')

In [ ]:
for a in occs:
    for field in a.form_fields['fields']:
        try:
            if 'altura' in field.get('displayName').lower() and 'executad' in field.get('displayName').lower():
                print(a.name)
        except:
            if 'altura' in field.get('display_name').lower() and 'executad' in field.get('display_name').lower():
                print(a.name)

In [ ]:
for a in occs:
    for field in a.form_fields['fields']:
        try:
            if 'executedHeight' in field.get('apiName'):
                print(a.name)
        except:
            pass

In [ ]:
Reporting.objects.filter(form_data__has_key='executed_height',company__name='CART')

In [ ]:
MultipleDailyReport.objects.get(number='INT-RDO-2023.00134').firm

In [ ]:
con = Contract.objects.get(uuid='f82a9b41-7a23-4f84-8423-42eff51be42f')

In [ ]:
con

In [ ]:
firms_iasin = Firm.objects.filter(subcompany__name='IASIN - SINALIZAÇÕES')
firms_iasin

In [ ]:
rdo=MultipleDailyReport.objects.filter(contract=con,reportings__firm__in=firms_iasin)

In [ ]:
rdo

In [ ]:
mb=MeasurementBulletin.objects.get(number='SV-BM-2023.9')

In [ ]:
mb.delete()

# Alterar Status Itens do contrato

# ProcedureResource Preço Unitário do Contrato

In [ ]:
pr=ProcedureResource.objects.get(reporting=Reporting.objects.get(number='SV-SinH-2023.00085'))

In [ ]:
pr.approval_status = "WAITING_APPROVAL"
pr.save()

In [ ]:
con=Contract.objects.get(uuid='21ba83ee-5610-49fa-9394-1a23011b14a9')

In [ ]:
# ContractItemAdministration.objects.filter(contract=con,name='SERVENTE')approval_status='APPROVED_APPROVAL',
# sor=ServiceOrderResource.objects.filter(contract=con,resource__name='SERVENTE')

# Item de Administração do Contrato

In [ ]:
rdo=MultipleDailyReport.objects.filter(contract=con,date__gte='2023-10-11')

In [ ]:
worker=DailyReportWorker.objects.filter(multiple_daily_reports__in=rdo,approval_status='APPROVED_APPROVAL')
equipment = DailyReportEquipment.objects.filter(multiple_daily_reports__in=rdo,approval_status='APPROVED_APPROVAL')
veiculo = DailyReportVehicle.objects.filter(multiple_daily_reports__in=rdo,approval_status='APPROVED_APPROVAL')

In [ ]:
worker.count(),equipment.count(),veiculo.count(),worker[0].__dict__

In [ ]:
for b in [worker,equipment,veiculo]:
    for a in b:
        a.approval_status = 'WAITING_APPROVAL'
        a.save()
#     if a.worker.approval_status == 'APPROVED_APPROVAL'
#     a.worker.

In [ ]:
company=Company.objects.get(name='Arteris Intervias')

In [ ]:
rep=Reporting.objects.get(number='INT-SIN-Aer-2023.07107')

In [ ]:
pr=ProcedureResource.objects.filter(service_order_resource__contract=con,
                                    reporting=rep,
                                    approval_status='APPROVED_APPROVAL',
                                    resource__name='2.7 - Fornecimento de placas de regulamentação, advertência e dispositivos de segurança confeccionadas em chapa de aço galvanizado, com película tipo III ou tipo X, exceto os elementos em preto, que deverão ser não refletivos com película tipo IV. - Ø 1,20 m (tipo III)')
pr

In [ ]:
pr_teste = pr[0]

In [ ]:
pr_teste.approval_status = 'WAITING_APPROVAL'
pr_teste.save()

In [ ]:
ServiceOrderResource._meta.fields

In [ ]:
up=UserPermission.objects.filter(companies__name='Sinaliza Via (Rodovias)')

In [ ]:
for a in up:
    a.permissions['Reporting']['can_approve'] = True
    a.save()
#     print(a.permissions['Reporting'])

In [ ]:
uis=UserInCompany.objects.filter(company__name='ELO SPVias').exclude(user__email__icontains='kartado').values('user__uuid')

In [ ]:
uis.count(),uis

In [ ]:
User.objects.filter(uuid__in=uis)

In [ ]:
from django.core import serializers

data = serializers.serialize("json", User.objects.filter(uuid__in=uis))

In [ ]:
data

In [ ]:
data=[]
for a in uis:
    user=User()
    for field in a.user._meta.fields:
        setattr(user, field.name, getattr(a.user, field.name))
    
    data.append(user)

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(data, f)

# Exemplo de copia de atributos

In [ ]:
user_source = User.objects.get(user=user)


user_dest = User()

for field in user_source._meta.fields:
    if field.uuid:
        continue  # Skip the primary key field
    setattr(user_dest, field.name, getattr(user_source, field.name))

In [ ]:
uis[0].user._meta.fields

In [ ]:
QueuedEmail.objects.filter(title='Redefinição da sua senha Kartado',company__name='CCR - SPVias')

In [ ]:
status=ServiceOrderActionStatus.objects.get(companies__name='Arteris Via Paulista',name="Executado")

In [ ]:
status

In [ ]:
rep=Reporting.objects.get(number='VPT-SIN-Ver-2023.14695')

In [ ]:
rep.status

In [ ]:
rep.approval_step

In [ ]:
from openpyxl import load_workbook
filename = "elospviasclasses.xlsx"
wb = load_workbook(filename)
ws = wb['Naturezas e Classes Preencher']
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
company=Company.objects.get(name='ELO SPVias')

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(values):
#     try:
    occ=OccurrenceType.objects.filter(name=a['Classes'].strip(),company__name='ELO EcoNoroeste')
    if occ.exists():
        occ = occ.first()
        
        specs=OccurrenceTypeSpecs.objects.filter(occurrence_type=occ,company=company)
        if not specs.exists():
            occ_specs=OccurrenceTypeSpecs(occurrence_type=occ,company=company)
            occ_specs.save()
    else:
        print(specs,occ)
#     except:
#         print(occ)

In [ ]:
nat=[{
							"name": "Pavimento Flexível",
							"value": "5"
						},
						{
							"name": "Pavimento Rígido",
							"value": "6"
						},
						{
							"name": "Revestimento Vegetal",
							"value": "7"
						},
						{
							"name": "Limpeza",
							"value": "8"
						},
						{
							"name": "Drenagem de Plataforma",
							"value": "9"
						},
						{
							"name": "Drenagem Fora de Plataforma",
							"value": "10"
						},
						{
							"name": "Caixas de Captação",
							"value": "11"
						},
						{
							"name": "Bueiros, Galerias e Drenos",
							"value": "12"
						},
						{
							"name": "Sinalização Vertical - Placas",
							"value": "13"
						},
						{
							"name": "Balizadores e Delineadores",
							"value": "14"
						},
						{
							"name": "Dispositivo de Contenção Viária",
							"value": "15"
						}]

In [ ]:
nat

In [ ]:
for a in tqdm(values):
    num_nat = next(num['value'] for num in nat if a['Naturezas'].strip() == num['name'])
    occ=OccurrenceType.objects.filter(name=a['Classes'].strip(),company=company)
    occ_origem = OccurrenceType.objects.get(name=a['Classes'].strip(),company__name='ELO EcoNoroeste')
    if occ.exists():
        occ_dest = OccurrenceType(name=f"{a['Classes'].strip()} ({a['Naturezas']})",
                                 occurrence_kind=num_nat, 
                                  form_fields=occ_origem.form_fields)
        occ_dest.save()
        specs = OccurrenceTypeSpecs(occurrence_type=occ_dest,company=company)
        specs.save()
    else:
        occ_dest = OccurrenceType(name=f"{a['Classes'].strip()}",
                                 occurrence_kind=num_nat, 
                                  form_fields=occ_origem.form_fields)
        occ_dest.save()
        specs = OccurrenceTypeSpecs(occurrence_type=occ_dest,company=company)
        specs.save()

In [ ]:
occ_dest.delete()

In [ ]:
for a in values:
    if a['Classes'] == 'Corte e poda de árvores e arbustos':
        a['Classes'] = 'Corte e Poda de Árvores e Arbustos'

In [ ]:
values

In [ ]:
occ=OccurrenceType.objects.get(name='Fresagem',company__name='Eixo Sp - Pavimento')

In [ ]:
occ.form_fields['fields']

In [ ]:
for a in occ.history.all():
    for b in a.form_fields['fields']:
        if b['apiName'] == 'service':
            for inner in b['innerFields']:
                if inner['apiName'] == 'length':
                    print(a.history_date,inner['requiredIf'])

In [ ]:
rep=Reporting.objects.get(number='EIXO-TER-2023.02940').history.all()

In [ ]:
for a in rep:
    print(a.history_date,a.form_data['service'])

In [ ]:
bi=list({"uuid", "link", "number", "roadName", "km", "endKm", "latitude", "longitude", "occurrenceType", "length", "width", "height", "lane", "track", "branch", "kmReference", "direction", "occurrenceKind", "status", "createdBy", "updatedBy", "firm", "subcompany", "job", "createdAt", "foundAt", "updatedAt", "executedAt", "notes", "dueAt", "price", "construction", "idInventory", "contratualCode", "csp", "artespCode", "receivedDate", "area", "volume", "tonnage", "building", "amount", "demandOrigin", "services", "signName", "signType", "raOld", "raNew", "lcaLength", "lcaWidth", "lcaArea", "zpaLength", "zpaWidth", "zpaArea", "notTree", "manualAmount", "manualLength", "manualWidth", "manualArea", "mecanizadaAmount", "mecanizadaLength", "mecanizadaWidth", "mecanizadaArea", "mourao", "arame", "weight", "canchasDeAplicacao", "cbuqType", "truckArrival2", "finalTemperature", "arrivalTemperature", "compressionTemperature", "ticket", "notifiedId", "notifiedNumber", "lot"})

In [ ]:
api = ["uuid", "link","number","roadName","subCompany","km","endKm","lot","latitude","longitude","occurrenceType",
      "occurrenceKind","lane","direction","status","createdBy","lastHistoryUser","firm","job","foundAt","updatedAt",
      "executedAt","dueAt"]
names=["uuid", "link","Número","Rodovia","Empresa","km inicial","km final","Lote","Latitude","Longitude","Classe","Natureza","Faixa",
      "Sentido","Status","Criado por","Atualizado por","Equipe","Programação","Criado em","Atualizado em","Executado em","Prazo"]
traducao={a:b for a,b in zip(api,names)}

In [ ]:
traducao

In [ ]:
company=Company.objects.get(name='Entrevias')

In [ ]:
company.custom_options['reportingSpreadsheet']

In [ ]:
for i,a in enumerate(bi):
    if a not in [key for key in traducao.keys()]:
        for k in company.custom_options['reportingSpreadsheet']['extraColumns']:
            if a == k['key']:
                traducao[k['key']] = k['header']

In [ ]:
traducao

In [ ]:
for a in UserPermission.objects.filter(name__icontains='artesp'):
    try:
        print(a.name,a.companies.first().name)
        for k,v in a.permissions.items():
            if v.get('can_view') == True:
                print(k,v.get('can_view'),v.get('queryset'),v.get('can_view_sisoaes'))
    except:
        pass

In [ ]:
from django.db.models import Q

In [ ]:
for a in UserPermission.objects.filter(Q(name__icontains='artesp') | Q(name__icontains='antt')):
    print(a.name,[b.name for b in a.companies.all()])

In [ ]:
import pandas as pd
import openpyxl as op
from django.db.models import Q

In [ ]:
filename = 'ArterisQuantidades'
pd.DataFrame().to_excel(filename+'.xlsx')
wb=op.load_workbook(filename+'.xlsx')

In [ ]:
for company in Company.objects.filter(Q(name__startswith='Arteris') | Q(name__icontains='conserva')):
    reps=Reporting.objects.filter(company=company
                             ).values('occurrence_type__name'
                                     ).annotate(qtd_por_classe=Count('uuid')
                                               ).order_by('occurrence_type__name')
    if reps.exists():
        data={a['occurrence_type__name']:a['qtd_por_classe'] for a in reps}
        df2 = pd.DataFrame(data=data,index=['Qtd Apontamentos por Classe'])
        df2 = df2.transpose()
        with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=company.name,na_rep="-")

In [ ]:
rep=Reporting.history.filter(history_type='-',
                             history_date__gt='2023-12-10',
                             firm=Firm.objects.get(name='teste notificacao',company__name='Demo Concessionárias'),
                             company__name='Demo Concessionárias')

In [ ]:
rep[0].instance.save()

In [ ]:
rep=Reporting.objects.get(number='DEMO-SIN-Ver-2023.00013')

In [ ]:
historico = rep.history.all()

In [ ]:
historico,historico.count()

In [ ]:
for a in historico:
    print(a.history_type)

In [ ]:
historico[1].history_user = User.objects.get(username='joao.nitzke')
historico[1].save()

In [ ]:
historico[0].history_relation.__dict__

In [ ]:
historico[0].__dict__

In [ ]:
repfile=ReportingFile.objects.get(uuid='b0a12b5b-5528-4673-8a7c-342d69e55919')

In [ ]:
repfile.__dict__

In [ ]:
ReportingFile.objects.first().upload

In [ ]:
mdr=MultipleDailyReport.objects.get(number='ROO-RDO-2023.00763')

In [ ]:
mdr._meta.fields

In [ ]:
for a in Reportin:
    try:
        a.upload.size
    except:
        print(a.uuid)

In [ ]:
rep=Reporting.objects.get(number='INT-SIN-Aer-2023.06753')

In [ ]:
for a in rep.history.all():
    print(a.history_date.strftime('%d/%m/%Y %HH:%MM'),a.firm.name,a.history_user.full_name)

In [ ]:
rep=Reporting.objects.get(uuid='9ffba2f2-4517-421e-943d-308a39a29c91')

In [ ]:
hist=rep.history.all()

In [ ]:
hist[0].occurrence_type

In [ ]:
rep.occurrence_type

In [ ]:
al=ActionLog.objects.filter(company__name='Neovia Cto Biguaçu')

In [ ]:
for a in ActionLog.objects.filter(company__name='Neovia Cto Biguaçu'):
    print(a)

In [ ]:
MobileSync.objects.filter(created_by__username='edgar.silva')

In [ ]:
MobileSync._meta.fields

In [ ]:
AccessRequest.objects.get(uuid='08de5d30-4df8-4de7-b590-8bdf01ad5f37').created_by

In [ ]:
Reporting.objects.filter(occurrence_type__name="Erro de marco quilométrico", company__name__icontains='Modera')

In [ ]:
rep=Reporting.objects.get(uuid='9ffba2f2-4517-421e-943d-308a39a29c91')

In [ ]:
rep.__dict__.keys()

In [ ]:
duplicates = Road.objects.values('name').annotate(
    count=Count('name')).filter(count__gt=1,
    company__name='Modera - Consórcio ME2')
duplicate_objects = Road.objects.filter(
    name__in=[item['name'] for item in duplicates]
).order_by('name')

In [ ]:
duplicate_objects

In [ ]:
for a in duplicate_objects:
    ini = list(a.marks)[0]
    fim = list(a.marks)[-1]
    print(f'id:{a.id}, name:{a.name}, km inicial:{a.marks[ini]}, km final:{a.marks[fim]},index:{a.all_marks_have_indexes}')

In [ ]:
Reporting.objects.filter(company__name='Modera - Consórcio ME2',road_id=4025)

In [ ]:
duplicate_objects[1].__dict__

In [ ]:
for a in OccurrenceType.objects.filter(company__name='Rota do Oeste - Obras'):
    for b in a.form_fields['fields']:
        if 'massa' in b.get('displayName').lower():
            print(a.name)

In [ ]:
def get_summary_forum(wk_id):
    url = f"https://reports.api.clockify.me/v1/workspaces/{wk_id}/reports/summary"
    headers = {'X-Api-Key': API_key}
    data = {
        "dateRangeStart": "2020-05-10T00:00:00",
        "dateRangeEnd": "2022-07-10T23:59:59",
        "summaryFilter": {
            "groups": ["PROJECT"],
            "sortColumn": "GROUP"
            },
        "exportType": "JSON"
        }

    return requests.post(url, headers=headers, json=data).json()

In [ ]:
get_summary_forum('6388be0aa581104ca1897b5c')

In [ ]:
"ReportingAppConfig": {"can_edit": true, "can_view": true, "queryset": "all", "can_create": true, "can_delete": true}

In [ ]:
up=UserPermission.objects.filter(companies__active=True,name__in=['Suporte',
                                                                  'Coordenador',
                                                                  'HOMOLOGATOR',
                                                                  'SUPORTE',
                                                                  "Administrador",
                                                                  'Sala técnica']).only('permissions')

In [ ]:
up[0].__dict__

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(up):
    a.permissions['ReportingAppConfig'] = {"can_edit": True, 
                                           "can_view": True, 
                                           "queryset": "all", 
                                           "can_create": True, 
                                           "can_delete": False}
    a.save()

In [ ]:
from json_logic import jsonLogic

In [ ]:
jsonLogic(logica,{'formData.service._index.length':10,'formData.service._index.width':2,'formData.service._index.servicetype':"15"})

In [ ]:
for a in Firm.objects.filter(company__name='Via Brasil',name='Aprovação de RDOS'):
    for b in a.users.all():
        uc=UserInCompany.objects.get(company__name='Via Brasil',user=b)
        if uc.permissions.name == 'Usuário padrão':
            uc.permissions = UserPermission.objects.get(name='Usuário padrão RDO',companies__name='Via Brasil')
            uc.save()
        print(uc.user.username,uc.permissions)

In [ ]:
a.users.all()[0]._meta.fields

In [ ]:
rep=Reporting.objects.get(number='ENT-INS-2023.03101')
rep.__dict__

In [ ]:
for maini,a in enumerate(rep.form_data['therapy']):
    for i,treat in enumerate(a['treatment_images']):
        try:
            rep_file=ReportingFile.objects.get(uuid=treat)
        except:
            print(treat)
#             a['treatment_images']
            print(rep.form_data['therapy'][maini]['treatment_images'][i])
#             del rep.form_data['therapy'][maini]['treatment_images'][i]
#             rep.save()
#     rep.form_data['therapy']

In [ ]:
company=Company.objects.get(name='Arteris Via Paulista')

In [ ]:
company

In [ ]:
reps_insp=Reporting.objects.filter(parent__isnull=True,company=company,occurrence_type__occurrence_kind='501')

In [ ]:
reps_insp.count()

In [ ]:
reps_insp[0],

In [ ]:
reps_recup=Reporting.objects.filter(parent__isnull=False,company=company).exclude(occurrence_type__occurrence_kind='501')

In [ ]:
reps_recup.count(),reps_recup[0].parent

In [ ]:
Reporting.objects.filter(job__isnull=False,company=company,occurrence_type__occurrence_kind='2').count()

In [ ]:
ReportingFile.objects.filter(uuid__in=['ba7e9a21-1830-436d-9d98-b2d8ff14b63a'])

In [ ]:
for a in Reporting.objects.filter(company__name='Modera - Consórcio ME2',occurrence_type__name="Local"):
    a.form_data['regCeo'] = a.form_data['diretoria regional']
    del a.form_data['diretoria regional']
    a.save()

# Alterar companyGroups

In [ ]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm

In [ ]:
filename = "Unidades ativas Kartado Geral (2).xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[2]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
for a in values:
    a['Unidades:'] = a['Unidades:'].split('|  [')[0].strip()
    a['Grupo anterior'] = a['Grupo anterior'].strip()
    a['Grupo correto:'] = a['Grupo correto:'].strip()

In [ ]:
values[0]

In [ ]:
user=User.objects.get(username='rlcs')

In [ ]:
for a in tqdm(values):
    try:
        
        company=Company.objects.get(name__exact=a['Unidades:'])
        company_group = CompanyGroup.objects.get(name=a['Grupo anterior'])
        company_group_new = CompanyGroup.objects.filter(name=a['Grupo correto:'])
        if company_group.name == a['Grupo correto:']:
            continue
        elif company_group_new.exists():
            company.company_group = company_group_new.first()
            company.save()
            us=list(UserInCompany.objects.filter(company=company).values_list('user__uuid',flat=True))
            User.objects.filter(uuid__in=us).update(company_group=company_group_new.first())
        elif not company_group_new.exists():
            cg=CompanyGroup(name=a['Grupo correto:'],key_user=user,mobile_app='road')
            cg.save()
            company.company_group = cg
            us=list(UserInCompany.objects.filter(company=company).values_list('user__uuid',flat=True))
            User.objects.filter(uuid__in=us).update(company_group=cg)
        company.save()
            
    except Exception as e:
        print(a['Unidades:'],e)

In [ ]:
CompanyGroup._meta.fields

In [ ]:
UserInCompany._meta.fields

In [ ]:
Company.objects.filter(company_group__name='Pavicolet')

In [ ]:
UserInCompany.objects.filter(user__company_group__name='Pavicolet')

In [ ]:
for a in OccurrenceType.objects.filter(company__name='Rota do Oeste'):
    try:
        a.form_fields['measurementColumns'].insert(0,{
          "key": "dueAt",
          "width": 15,
          "header": "Prazo",
          "logic": {"var": "dueAt"},
          "isDate":True
    })
        a.save()
    except Exception as e:
        print(a.name,e)

In [ ]:
a.form_fields['measurementColumns']

In [ ]:
roads = Road.objects.filter(company__name="Minasul")

In [ ]:
for a in roads:
#     print(a.name)
    historic = a.history.filter(name='Obra Biguaçu')
    if historic.exists():
        print(a.name,a.id,a.lot_logic)

In [ ]:
rep=Reporting.objects.get(number='minasul-Remo-2023.00011')

In [ ]:
for a in rep.history.all():
    print(a.road,a.road_name,a.lot,a.history_user,a.km)

In [ ]:
rep.road_name = 'BR-277'
rep.save()

In [ ]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
from helpers.signals import DisableSignals

In [ ]:
filename = "cart_seila.xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[0]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
company = Company.objects.get(name="CART")
service = ServiceOrderActionStatus.objects.get(companies=company,name="Cancelado")
user=User.objects.get(username='rlcs')
service

In [ ]:
rep_edit=[]
for a in tqdm(values):
    rep = Reporting.objects.get(company=company,number=a['SERIAL'])
    rep.status = service
    rep_edit.append(rep)
#     with DisableSignals():
#         rep.save()

In [ ]:
from helpers.histories import bulk_update_with_history

apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=800,user=user)

In [ ]:
lista=["EIXO-RDO-2024.00137",
"EIXO-RDO-2023.39478",
"EIXO-RDO-2023.39363",
"EIXO-RDO-2023.39230",
"EIXO-RDO-2023.39068",
"EIXO-RDO-2023.38954",
"EIXO-RDO-2023.38779",
"EIXO-RDO-2023.38664",
"EIXO-RDO-2023.38503",
"EIXO-RDO-2023.38370",
"EIXO-RDO-2023.38043",
"EIXO-RDO-2023.37867",
"EIXO-RDO-2023.37773",
"EIXO-RDO-2023.37605",
"EIXO-RDO-2023.37396",
"EIXO-RDO-2023.37295",
"EIXO-RDO-2023.37158",
"EIXO-RDO-2023.37064",
"EIXO-RDO-2023.36934",
"EIXO-RDO-2023.36724",
"EIXO-RDO-2023.36566",
"EIXO-RDO-2023.36399",
"EIXO-RDO-2023.36332",
"EIXO-RDO-2023.36052",
"EIXO-RDO-2023.36050",
"EIXO-RDO-2023.35597",
"EIXO-RDO-2023.35545",]
len(lista)

In [ ]:
company=Company.objects.get(name="Eixo SP")
company

In [ ]:
rdos = MultipleDailyReport.objects.filter(number__in=lista,company=company)
firm = Firm.objects.get(name='SPD-DRENAGEM PROF',company=company)
rdos.count(),firm

In [ ]:
for a in tqdm(rdos):
    rdo_antigo = MultipleDailyReport.objects.filter(company=company,firm=firm,date=a.date)
    if rdo_antigo.exists():
        rdo = rdo_antigo.first()
        print(rdo.number,rdo.date)
    else:
        a.firm = firm
        a.save()

In [ ]:
rdo = MultipleDailyReport.objects.get(number='RDO-DEF-2023.00096')
rdo

In [ ]:
Reporting.objects.get(number='ROO-DEF-2023.00096')

In [ ]:
for a in Reporting.objects.filter(company__name='Rota do Oeste - Obras',occurrence_type__isnull=True):
    print(a.number)

In [ ]:
lista=['ROO-DRE-2023.00002',
'ROO-E-2023.00001',
'ROO-DRE-2023.00003',
'ROO-DRES-2023.00003',
'ROO-RE-2023.00001',
'ROO-DIS-2023.00001',
'ROO-DRES-2023.00002']
len(lista)

In [ ]:
Reporting.objects.filter(company__name='Rota do Oeste - Obras',number__in=lista).delete()

In [ ]:
from tqdm.notebook import tqdm
counter=0
for a in tqdm(ReportingFile.objects.filter(reporting__company__name='Neovia Araranguá')):
    try:
        a.upload.url
        counter+=1
    except:
        print(a.reporting)
counter

In [2]:
wb.sheetnames

['Planilha1']

In [1]:
from openpyxl import load_workbook
filename = "NPS.xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[0]]
ws

<Worksheet "Planilha1">

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
# email e empresa
for a in values:
    try:
        user = User.objects.get(uuid=a['user.userId'])
        empresa = UserInFirm.objects.filter(user=user).values('firm__subcompany__name')
        if empresa.exists():
            empresa=empresa.first()
            a['email']=user.email
            a['Empresa Terceira'] = empresa['firm__subcompany__name']
    except Exception as e:
        print(e)

In [ ]:
import pandas as pd
df = pd.DataFrame(data=values)
df

In [ ]:
df.to_excel('NPS Email Terceiros.xlsx')

# Desmembrar Permissões

In [ ]:
from copy import deepcopy
from tqdm.notebook import tqdm

In [ ]:
company=Company.objects.get(name='ELO Ecovias dos Imigrantes')
perm_ecovias = UserPermission.objects.filter(companies__name__startswith='ELO').exclude(companies=company)
perm_ecovias.count(),company

In [ ]:
# criar uma nova permissão, 
# depois setar a empresa na permissão
# atualizar os usuários com essa nova permissão
# remover a unidade desmembrada
a_inserir=[]
for a in tqdm(perm_ecovias):
    perm_nova = deepcopy(a)
    perm_nova.uuid=None
    perm_nova.save()
#     perm_nova.companies.set(company)
    uic=UserInCompany.objects.filter(company=company,permissions__name=a.name).update(permissions=perm_nova)
    a.companies.remove(company)
    a_inserir.append(perm_nova)

In [ ]:
for c in tqdm(a_inserir):
    c.companies.set(company)

In [ ]:
for a in UserInCompany.objects.filter(company=company):
    print(a.permissions)

In [10]:
from json_logic import jsonLogic

# jsonLogic(json_final,{'km':681,'roadName':'BR-381/MG'})

In [26]:
reps=Reporting.objects.filter(company__name='Eixo SP',road__isnull=True)
reps.count()

421

In [27]:
road = Road.objects.get(company__name='Eixo SP',name=reps[0].road_name)

In [28]:
road

<Road: 504: SPA-360/294>

In [29]:
jsonLogic(road.lot_logic,{'km':0.01,'roadName':'SPA-360/294'}) == None

True

In [31]:
reps[0].point.y

0.0

In [40]:
data=[]
for a in reps:
    road = Road.objects.filter(company__name='Eixo SP',name=a.road_name).first()
    if road == None:
        data.append({'Serial':a.number,"Rodovia":a.road_name,"Km inicial":a.km,"Km Final":a.end_km,
                     "X":a.point.x if a.point is not None else 'None',"Y":a.point.y if a.point is not None else 'None',
                     "Is Road None":"Sim"})
    elif jsonLogic(road.lot_logic,{'km':a.km,'roadName':a.road_name}) == None:
        data.append({'Serial':a.number,"Rodovia":a.road_name,"Km inicial":a.km,"Km Final":a.end_km,
                     "X":a.point.x if a.point is not None else 'None',"Y":a.point.y if a.point is not None else 'None',
                     "Is Road None":"Não"})

In [41]:
data

[{'Serial': 'EIXO-default-2022.19125',
  'Rodovia': 'SPA-585/294',
  'Km inicial': 0.29,
  'Km Final': 0.289,
  'X': 0.0,
  'Y': 0.0,
  'Is Road None': 'Não'},
 {'Serial': 'EIXO-LV-2020.13390',
  'Rodovia': 'SP-225',
  'Km inicial': 920.0,
  'Km Final': 106.0,
  'X': 0.0,
  'Y': 0.0,
  'Is Road None': 'Não'},
 {'Serial': 'EIXO-DRE-2020.00990',
  'Rodovia': 'SPA-228/225',
  'Km inicial': 1440.0,
  'Km Final': 1440.0,
  'X': 0.0,
  'Y': 0.0,
  'Is Road None': 'Não'},
 {'Serial': 'EIXO-LV-2021.06033',
  'Rodovia': 'SPA-005/197',
  'Km inicial': 7.12,
  'Km Final': 7.12,
  'X': 0.0,
  'Y': 0.0,
  'Is Road None': 'Não'},
 {'Serial': 'EIXO-LV-2021.06034',
  'Rodovia': 'SPA-005/197',
  'Km inicial': 9.17,
  'Km Final': 9.17,
  'X': 0.0,
  'Y': 0.0,
  'Is Road None': 'Não'},
 {'Serial': 'EIXO-LV-2021.06035',
  'Rodovia': 'SPA-005/197',
  'Km inicial': 9.17,
  'Km Final': 9.17,
  'X': 0.0,
  'Y': 0.0,
  'Is Road None': 'Não'},
 {'Serial': 'EIXO-LV-2021.06036',
  'Rodovia': 'SPA-005/197',
  'Km 

In [42]:
import pandas as pd

In [43]:
df = pd.DataFrame(data=data)

In [44]:
df.to_excel('erros_EixoSP.xlsx')

In [46]:
from helpers.signals import DisableSignals

In [51]:
for i,a in enumerate(Road.objects.filter(company__name="Demo Concessionárias")):
    if 'modera' in a.name.lower():
        print(a.name)
        a.name = a.name.replace('Modera - ',str(str(i)+' '))
        print(a.name)
        with DisableSignals():
            a.save()

Modera - SP 015
50 SP 015
Modera - SP 019
51 SP 019
Modera - SP 021
52 SP 021
Modera - SP 023
53 SP 023
Modera - SP 029
54 SP 029
Modera - SP 031
55 SP 031
Modera - SP 036
56 SP 036
Modera - SP 039
57 SP 039
Modera - SP 042
58 SP 042
Modera - SP 043
59 SP 043
Modera - SP 046
60 SP 046
Modera - SP 055
61 SP 055
Modera - SP 056
62 SP 056
Modera - SP 057
63 SP 057
Modera - SP 061
64 SP 061
Modera - SP 062
65 SP 062
Modera - SP 063
66 SP 063
Modera - SP 064
67 SP 064
Modera - SP 065
68 SP 065
Modera - SP 066
69 SP 066
Modera - SP 068
70 SP 068
Modera - SP 070
71 SP 070
Modera - SP 073
72 SP 073
Modera - SP 075
73 SP 075
Modera - SP 077
74 SP 077
Modera - SP 079
75 SP 079
Modera - SP 081
76 SP 081
Modera - SP 083
77 SP 083
Modera - SP 088
78 SP 088
Modera - SP 091
79 SP 091
Modera - SP 092
80 SP 092
Modera - SP 095
81 SP 095
Modera - SP 097
82 SP 097
Modera - SP 098
83 SP 098
Modera - SP 099
84 SP 099
Modera - SP 101
85 SP 101
Modera - SP 102
86 SP 102
Modera - SP 105
87 SP 105
Modera - SP 

Modera - SPA 173/250
329 SPA 173/250
Modera - SPA 180/250
330 SPA 180/250
Modera - SPA 294/250
331 SPA 294/250
Modera - SPA 062/251
332 SPA 062/251
Modera - SPA 065/251
333 SPA 065/251
Modera - SPA 014/253
334 SPA 014/253
Modera - SPA 129/253
335 SPA 129/253
Modera - SPA 193/253
336 SPA 193/253
Modera - SPA 194/253
337 SPA 194/253
Modera - SPA 197/253
338 SPA 197/253
Modera - SPA 051/255
339 SPA 051/255
Modera - SPA 074/255
340 SPA 074/255
Modera - SPA 077/255
341 SPA 077/255
Modera - SPA 085/255
342 SPA 085/255
Modera - SPA 106/255
343 SPA 106/255
Modera - SPA 112/255
344 SPA 112/255
Modera - SPA 114/255
345 SPA 114/255
Modera - SPA 115/255
346 SPA 115/255
Modera - SPA 133/255
347 SPA 133/255
Modera - SPA 138/255
348 SPA 138/255
Modera - SPA 179/255
349 SPA 179/255
Modera - SPA 196/255
350 SPA 196/255
Modera - SPA 198/255
351 SPA 198/255
Modera - SPA 214/255
352 SPA 214/255
Modera - SPA 258/255
353 SPA 258/255
Modera - SPA 338/255
354 SPA 338/255
Modera - SPA 282/258
355 SPA 282/258
M

Modera - SPA 407/334
602 SPA 407/334
Modera - SPA 420/334
603 SPA 420/334
Modera - SPA 455/334
604 SPA 455/334
Modera - SPA 457/334
605 SPA 457/334
Modera - SPA 309/338
606 SPA 309/338
Modera - SPA 310/338
607 SPA 310/338
Modera - SPA 127/340
608 SPA 127/340
Modera - SPA 179/340
609 SPA 179/340
Modera - SPA 225/340
610 SPA 225/340
Modera - SPA 239/340
611 SPA 239/340
Modera - SPA 279/340
612 SPA 279/340
Modera - SPA 280/340
613 SPA 280/340
Modera - SPA 208/342
614 SPA 208/342
Modera - SPA 212/342
615 SPA 212/342
Modera - SPA 226/342
616 SPA 226/342
Modera - SPA 230/342
617 SPA 230/342
Modera - SPA 233/342
618 SPA 233/342
Modera - SPA 251/342
619 SPA 251/342
Modera - SPA 223/344
620 SPA 223/344
Modera - SPA 228/344
621 SPA 228/344
Modera - SPA 238/344
622 SPA 238/344
Modera - SPA 244/344
623 SPA 244/344
Modera - SPA 264/344
624 SPA 264/344
Modera - SPA 274/344
625 SPA 274/344
Modera - SPA 277/344
626 SPA 277/344
Modera - SPA 011/345
627 SPA 011/345
Modera - SPA 020/345
628 SPA 020/345
M

Modera - SPA 160/250
879 SPA 160/250
Modera - SPA 037/099
880 SPA 037/099
Modera - SPA 092/060
881 SPA 092/060
Modera - SPA 133/062
882 SPA 133/062
Modera - SPA 016/421
883 SPA 016/421
Modera - SPA 029/421
884 SPA 029/421
Modera - SPA 086/270
885 SPA 086/270
Modera - SPA 091/270
886 SPA 091/270
Modera - SPA 132/421
887 SPA 132/421
Modera - SPA 477/425
888 SPA 477/425
Modera - SPA 568/320
889 SPA 568/320
Modera - 01 SPA 018/461
890 01 SPA 018/461
Modera - SPI 065/099
891 SPI 065/099


In [5]:
from tqdm.notebook import tqdm
from helpers.signals import DisableSignals

In [8]:
reps = Reporting.objects.filter(company__name='Entrevias', 
                               occurrence_type='a5314341-194f-4cdb-8169-a5177f46fc87', 
                               status='3067707f-1eab-44af-b344-b346fda654ac', 
                               firm__in=['0dc0862e-579b-40af-aa51-8cff6de82d89', '97833756-3387-4e1e-9610-ea4f2d3793cd'])
reps.count()

0

In [7]:
for _ in tqdm(range(63)):
    rep_uuids = reps.values_list('uuid', flat=True)[:1000]
    reportings = Reporting.objects.filter(uuid__in=rep_uuids)
    with DisableSignals():
        reportings.delete()

  0%|          | 0/63 [00:00<?, ?it/s]

In [37]:
from helpers.strings import get_obj_from_path
from helpers.apps.reportings import get_lane
from tqdm.notebook import tqdm
import pandas as pd

In [38]:
companies = Company.objects.filter(name__in=['Arteris Litoral Sul','Arteris Régis Bittencourt','Arteris Planalto Sul'])
companies

<QuerySet [<Company: acd47ed7-4f96-4a27-b6a3-268b4e30779b: Arteris Planalto Sul>, <Company: 9763a1c5-3be6-4ddb-90bc-e6b96cc60b8d: Arteris Litoral Sul>, <Company: 9f11c666-9324-4d2c-8378-4648157b838b: Arteris Régis Bittencourt>]>

In [78]:
def get_connected_reference(company, resource, key, value):
    select_options = get_obj_from_path(company.custom_options, "{}__fields__{}__selectoptions__options".format(resource, key))
    return next(a['name'] for a in select_options if a['value'] == value)

In [79]:
my_dict=[]
unidades=[]
for company in tqdm(companies):
    if company.name not in unidades:
        unidades.append(company.name)
        pd.DataFrame().to_excel(company.name[:14]+'_arvore.xlsx')
    for form in tqdm(OccurrenceType.objects.filter(company=company).order_by('occurrence_kind', 'name')):
        try:
            campos=[]
            for field in form.form_fields['fields']:
                try:
                    campos.append(field['displayName'])
                except:
                    campos.append(field['display_name'])
            my_dict.append({get_connected_reference(company,
                                                    "reporting",
                                                    "occurrence_kind",
                                                    form.occurrence_kind):{form.name:campos}})
            reformed_dict = {}
            for a in my_dict:
                for outerKey, innerDict in a.items():
                    for innerKey, values in innerDict.items():
                        reformed_dict[(outerKey,
                                       innerKey)] = values
            max_length = max(len(v) for v in reformed_dict.values())
            for k, v in reformed_dict.items():
                reformed_dict[k] = v + [None] * (max_length - len(v))
            df2 = pd.DataFrame(reformed_dict)
            df2= df2.transpose()
            with pd.ExcelWriter(company.name[:14]+'_arvore.xlsx', mode='a',if_sheet_exists='replace') as writer:
                df2.to_excel(writer,sheet_name=natureza,na_rep="-")  
        except StopIteration:
            continue

In [80]:
df2

Levantamento                                                  \
            Acúmulo de Água              Bombeamento Buracos no acostamento   
0           Comprimento (m)          Comprimento (m)        Comprimento (m)   
1               Largura (m)              Largura (m)            Largura (m)   
2             Espessura (m)            Espessura (m)   Altura/Espessura (m)   
3                 Area (m²)                Area (m²)              Area (m²)   
4               Volume (m³)              Volume (m³)            Volume (m³)   
5               Observações              Observações            Observações   
6                    Origem                   Origem                   None   
7                    É TRO?                   É TRO?                   None   
8             Código da TRO            Código da TRO                   None   
9                   Solução                  Solução                   None   
10       Parâmetro Atendido       Parâmetro Atendido                   None   
11              Código KCOR              Código KCOR                   None   
12           Código Interno           Código Interno                   None   
13  Código Solicitação ANTT  Código Solicitação ANTT                   None   

                                                                               \
      Degrau de Acostamento                Depressão            Desplacamento   
0           Comprimento (m)          Comprimento (m)          Comprimento (m)   
1               Largura (m)              Largura (m)              Largura (m)   
2             Espessura (m)            Espessura (m)            Espessura (m)   
3                 Area (m²)                Area (m²)                Area (m²)   
4               Volume (m³)              Volume (m³)              Volume (m³)   
5               Observações              Observações              Observações   
6                    Origem                   Origem                   Origem   
7                    É TRO?                   É TRO?                   É TRO?   
8             Código da TRO            Código da TRO            Código da TRO   
9                   Solução                  Solução                  Solução   
10       Parâmetro Atendido       Parâmetro Atendido       Parâmetro Atendido   
11              Código KCOR              Código KCOR              Código KCOR   
12           Código Interno           Código Interno           Código Interno   
13  Código Solicitação ANTT  Código Solicitação ANTT  Código Solicitação ANTT   

                                                                            \
                Detritos  Escorregamento de Massa                Exsudação   
0        Comprimento (m)          Comprimento (m)          Comprimento (m)   
1            Largura (m)              Largura (m)              Largura (m)   
2   Altura/Espessura (m)            Espessura (m)            Espessura (m)   
3              Area (m²)                Area (m²)                Area (m²)   
4            Volume (m³)              Volume (m³)              Volume (m³)   
5            Observações              Observações              Observações   
6                   None                   Origem                   Origem   
7                   None                   É TRO?                   É TRO?   
8                   None            Código da TRO            Código da TRO   
9                   None                  Solução                  Solução   
10                  None       Parâmetro Atendido       Parâmetro Atendido   
11                  None              Código KCOR              Código KCOR   
12                  None           Código Interno           Código Interno   
13                  None  Código Solicitação ANTT  Código Solicitação ANTT   

                             ...                                              \
                       FC-3  ...           PRO-07 2023       Projeto. RP-152   
0           Comprimento (m)  ...       Compr

In [59]:
reformed_dict = {}
for a in my_dict:
    for outerKey, innerDict in a.items():
        for innerKey, values in innerDict.items():
            reformed_dict[(outerKey,
                           innerKey)] = values

# Multiindex dataframe
reformed_dict

{('Levantamento', 'Acúmulo de Água'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Observações',
  'Origem',
  'É TRO?',
  'Código da TRO',
  'Solução',
  'Parâmetro Atendido',
  'Código KCOR',
  'Código Interno',
  'Código Solicitação ANTT'],
 ('Levantamento', 'Bombeamento'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Observações',
  'Origem',
  'É TRO?',
  'Código da TRO',
  'Solução',
  'Parâmetro Atendido',
  'Código KCOR',
  'Código Interno',
  'Código Solicitação ANTT'],
 ('Levantamento', 'Degrau de Acostamento'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Observações',
  'Origem',
  'É TRO?',
  'Código da TRO',
  'Solução',
  'Parâmetro Atendido',
  'Código KCOR',
  'Código Interno',
  'Código Solicitação ANTT'],
 ('Levantamento', 'Depressão'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Obser

In [72]:
max_length = max(len(v) for v in reformed_dict.values())
for k, v in reformed_dict.items():
    reformed_dict[k] = v + [None] * (max_length - len(v))


In [74]:
reformed_dict

{('Levantamento', 'Acúmulo de Água'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Observações',
  'Origem',
  'É TRO?',
  'Código da TRO',
  'Solução',
  'Parâmetro Atendido',
  'Código KCOR',
  'Código Interno',
  'Código Solicitação ANTT'],
 ('Levantamento', 'Bombeamento'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Observações',
  'Origem',
  'É TRO?',
  'Código da TRO',
  'Solução',
  'Parâmetro Atendido',
  'Código KCOR',
  'Código Interno',
  'Código Solicitação ANTT'],
 ('Levantamento', 'Degrau de Acostamento'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Observações',
  'Origem',
  'É TRO?',
  'Código da TRO',
  'Solução',
  'Parâmetro Atendido',
  'Código KCOR',
  'Código Interno',
  'Código Solicitação ANTT'],
 ('Levantamento', 'Depressão'): ['Comprimento (m)',
  'Largura (m)',
  'Espessura (m)',
  'Area (m²)',
  'Volume (m³)',
  'Obser

In [75]:
df = pd.DataFrame(reformed_dict)

In [77]:
df.transpose()

0   \
Levantamento    Acúmulo de Água                 Comprimento (m)   
                Bombeamento                     Comprimento (m)   
                Degrau de Acostamento           Comprimento (m)   
                Depressão                       Comprimento (m)   
                Desplacamento                   Comprimento (m)   
                DRENO COM AFUNDAMENTO           Comprimento (m)   
                Dreno Existente                 Comprimento (m)   
                Escorregamento de Massa         Comprimento (m)   
                Exsudação                       Comprimento (m)   
                FC-3                            Comprimento (m)   
                Fresagem 2022                   Comprimento (m)   
                Imtraff                         Comprimento (m)   
                Inativo                         Comprimento (m)   
                Multifuncional                  Comprimento (m)   
                PA Buraco                       Comprimento (m)   
                Pintura Manual                  Comprimento (m)   
                Pintura Mecânica                Comprimento (m)   
                Pintura Termoplastica a Frio    Comprimento (m)   
                Pintura Termoplastica a Quente  Comprimento (m)   
                PM Buraco                       Comprimento (m)   
                Ponto de ensaio                 Comprimento (m)   
                Prioridade 1 Fresagem           Comprimento (m)   
                PRO-07 2023                     Comprimento (m)   
                Projeto. RP-152                 Comprimento (m)   
                Projeto RP (157)                Comprimento (m)   
                Remendo                         Comprimento (m)   
                Sela Trinca                     Comprimento (m)   
                Tachão                          Comprimento (m)   
                Tachas                          Comprimento (m)   
                TESTE                           Comprimento (m)   
                Trincamento                     Comprimento (m)   
                Verificação de PRO-07           Comprimento (m)   
Inventário      Dreno Longitudinal              Comprimento (m)   
                Dreno Transversal               Comprimento (m)   
                Espessura Estrutura                Revestimento   
                Pavimento                       Comprimento (m)   
                Pista com Faixa Adicional       Comprimento (m)   
                Pista Dupla                     Comprimento (m)   
                Pista Simples                   Comprimento (m)   
                PRO-07 2022                     Comprimento (m)   
                Saída de dreno                  Comprimento (m)   
                TRO                             Comprimento (m)   
Diário          DIÁRIO                                    Clima   
Monitoração     Aderência                       Comprimento (m)   
                ATR                             Comprimento (m)   
                Flecha                          Comprimento (m)   
                FWD                             Comprimento (m)   
                ICP                             Comprimento (m)   
                IRI                             Comprimento (m)   
                PRO-07 2022 R1                  Comprimento (m)   
                TR                              Comprimento (m)   
SGP             SGP                             Comprimento (m)   
Outros Serviços PRO 07                          Comprimento (m)   
                Retrorrefletância                       Valores   
                Sondagem                        Comprimento (m)   

                                                                     1   \
Levantamento    Acúmulo de Água                             Largura (m)   
                Bombeamento                                 Largura (m)   
                Degrau de Acostamento                       Largura (m)   
        

In [6]:
for a in ActionLog.objects.filter(user__username='leonardo.bruxel',company__name='Via Araucária'):
    print(a.__dict__,a.content_type.model)

{'_state': <django.db.models.base.ModelState object at 0x7692aaa56460>, 'uuid': UUID('7c162fc7-b4e9-46ec-86ea-eed48494eb67'), 'company_id': UUID('af087987-e1cd-47e0-8885-d306d5da6821'), 'company_group_id': UUID('e247f3b2-bd48-4849-92c6-301a7381f7e8'), 'user_id': UUID('eff91b1a-55a3-4504-850d-e6467fd99e49'), 'created_at': datetime.datetime(2024, 4, 3, 13, 25, 27, 501801, tzinfo=<UTC>), 'action': 'Update', 'user_ip': '161.22.58.178', 'user_port': None, 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'content_type_id': 12, 'object_id': UUID('af087987-e1cd-47e0-8885-d306d5da6821')} company
{'_state': <django.db.models.base.ModelState object at 0x76928b5e0790>, 'uuid': UUID('bba1a6e0-ee97-417d-9af1-733f9a122dce'), 'company_id': UUID('af087987-e1cd-47e0-8885-d306d5da6821'), 'company_group_id': UUID('e247f3b2-bd48-4849-92c6-301a7381f7e8'), 'user_id': UUID('eff91b1a-55a3-4504-850d-e6467fd99e49'), 'created_at': dat